In [ ]:
pip install sktime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.0/37.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.5/145.5 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.1
    Uninstalling joblib-1.5.1:
      Successfully uninstalled joblib-1.5.1


In [ ]:
pip install scikeras

In [ ]:
pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [ ]:
from keras.src import optimizers
from keras.src.utils import plot_model
from keras.src.models import Sequential, Model
from keras.src.layers.convolutional.conv1d import Conv1D
from keras.src.layers.pooling.max_pooling1d import MaxPooling1D
from keras.src.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from keras.src import metrics
from keras.layers import Input
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date
import sklearn
from sklearn.model_selection import train_test_split

import math
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime as dt, date, timedelta
# from meteostat import Point, Daily

In [ ]:
# data = pd.read_csv('/content/preprocessed_main_data_dki1_kemayoran.csv')
data = pd.read_csv('/content/preprocessed_main_data_dki1_kemayoran_v2.csv', sep=';')
data = data.set_index('tanggal')
data

,pm10,pm25,so2,co,o3,no2,Tavg,RH_avg,RR,ss,ff_avg,DDD_CAR_cos,DDD_CAR_sin
tanggal,,,,,,,,,,,,,
2023-01-01,44.0,55.0,47.0,10.0,24.0,9.0,26.3,87.0,14.5,0.0,1.0,0.000000,0.000000
2023-01-02,32.0,43.0,52.0,9.0,24.0,8.0,27.5,81.0,31.5,0.7,2.0,0.707107,-0.707107
2023-01-03,31.0,35.0,49.0,9.0,12.0,7.0,26.6,82.0,0.5,0.0,1.0,0.000000,0.000000
2023-01-04,30.0,47.0,53.0,11.0,15.0,9.0,26.4,86.0,2.4,2.1,1.0,0.000000,0.000000
2023-01-05,38.0,50.0,50.0,13.0,26.0,11.0,27.4,83.0,35.3,4.9,2.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-27,54.0,78.0,19.0,23.0,34.0,31.0,30.1,71.0,0.0,8.0,2.0,0.000000,0.000000
2024-10-28,60.0,83.0,10.0,26.0,37.0,36.0,30.2,67.0,0.0,8.0,1.0,0.000000,0.000000
2024-10-29,59.0,75.0,11.0,28.0,29.0,41.0,30.8,69.0,0.0,0.6,1.0,0.000000,0.000000


In [ ]:
WINDOW_SIZE = 7
HORIZON = 7

scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

In [ ]:
# train = 92
# val   = 4
# test  = 4
# feats = polutan only

features = ['pm10', 'pm25', 'so2', 'co', 'o3', 'no2']

targets = ['pm25']

data_x = data[features]
data_y = data[targets]

data_x_scaled = scaler_x.fit_transform(data_x)
data_y_scaled = scaler_y.fit_transform(data_y)

X = []
y = []

for i in range(len(data) - WINDOW_SIZE - HORIZON + 1):
    X.append(data_x_scaled[i:i+WINDOW_SIZE])
    y.append(data_y_scaled[i+WINDOW_SIZE:i+WINDOW_SIZE + HORIZON])

training_data_len = math.ceil(len(X)* 0.92)

#splitting train
x_train = X[:training_data_len]
y_train = y[:training_data_len]

# Val set
val_test_len = len(X[training_data_len:])
val_len = math.ceil(val_test_len/2)

x_val = X[training_data_len: training_data_len+val_len]
y_val = y[training_data_len: training_data_len+val_len]

# Test set
x_test = X[training_data_len+val_len: ]
y_test = y[training_data_len+val_len: ]

# Reshaping
x_train, y_train = np.array(x_train), np.array(y_train)

x_val, y_val = np.array(x_val), np.array(y_val)

x_test, y_test = np.array(x_test), np.array(y_test)

y_train = y_train.reshape((y_train.shape[0], y_train.shape[1]))
y_val = y_val.reshape((y_val.shape[0], y_val.shape[1]))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1]))

In [ ]:
x_train.shape, x_val.shape, x_test.shape
# ((575, 7, 11), (72, 7, 11), (71, 7, 11))

((661, 7, 6), (29, 7, 6), (28, 7, 6))

In [ ]:
y_train.shape, y_val.shape, y_test.shape
# ((575, 7, 1), (72, 7, 1), (71, 7, 1))

((661, 7), (29, 7), (28, 7))

In [ ]:
y_train

array([[0.34821429, 0.25892857, 0.30357143, ..., 0.52678571, 0.60714286,
        0.53571429],
       [0.25892857, 0.30357143, 0.49107143, ..., 0.60714286, 0.53571429,
        0.41964286],
       [0.30357143, 0.49107143, 0.52678571, ..., 0.53571429, 0.41964286,
        0.26785714],
       ...,
       [0.50892857, 0.22321429, 0.32142857, ..., 0.625     , 0.60714286,
        0.40178571],
       [0.22321429, 0.32142857, 0.38392857, ..., 0.60714286, 0.40178571,
        0.61607143],
       [0.32142857, 0.38392857, 0.625     , ..., 0.40178571, 0.61607143,
        0.46428571]])

In [ ]:
scaler_y.inverse_transform(y_train)

array([[49., 39., 44., ..., 69., 78., 70.],
       [39., 44., 65., ..., 78., 70., 57.],
       [44., 65., 69., ..., 70., 57., 40.],
       ...,
       [67., 35., 46., ..., 80., 78., 55.],
       [35., 46., 53., ..., 78., 55., 79.],
       [46., 53., 80., ..., 55., 79., 62.]])

# Define Evaluate Error

In [ ]:
from math import sqrt
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error

# MASE implemented courtesy of sktime - https://github.com/alan-turing-institute/sktime/blob/ee7a06843a44f4aaec7582d847e36073a9ab0566/sktime/performance_metrics/forecasting/_functions.py#L16
# def mean_absolute_scaled_error(y_true, y_pred):
#   """
#   Implement MASE (assuming no seasonality of data).
#   """
#   mae = tf.reduce_mean(tf.abs(y_true - y_pred))

#   # Find MAE of naive forecast (no seasonality)
#   mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1])) # our seasonality is 1 day (hence the shifting of 1 day)

#   return mae / mae_naive_no_season

def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  # y_true = tf.cast(y_true, dtype=tf.float32)
  # y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mape = mean_absolute_percentage_error(y_true, y_pred)

  y_pred = y_pred.reshape(y_pred.shape[0], -1)
  y_true = y_true.reshape(y_true.shape[0], -1)

  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
  rmse = sqrt(mse)
  # mase = mean_absolute_scaled_error(y_true, y_pred)
  # print(mae, mse, rmse, mape, mase)

  return {
          "Mean Squared Error: ": mse,
          "Mean Absolute Error: ": mae,
          "Root Mean Absolute Error: ": rmse,
          "Mean Absolute Percentage Error: ": mape
          # "mase": mase.numpy()
          }

# CNN Only

## model 1 (64)

In [ ]:
lr = 0.0003

model_cnn_1 = Sequential()
model_cnn_1.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_1.add(Flatten())
model_cnn_1.add(Dense(7))
model_cnn_1.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_1.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 7, 64)          │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,591 (14.03 KB)

 Trainable params: 3,591 (14.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_1.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.1583 - mean_absolute_error: 0.3491 - mean_absolute_percentage_error: 62.3506 - root_mean_squared_error: 0.3948 - val_loss: 0.0605 - val_mean_absolute_error: 0.1981 - val_mean_absolute_percentage_error: 18052466.0000 - val_root_mean_squared_error: 0.2460
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0300 - mean_absolute_error: 0.1334 - mean_absolute_percentage_error: 35.2644 - root_mean_squared_error: 0.1732 - val_loss: 0.0587 - val_mean_absolute_error: 0.1971 - val_mean_absolute_percentage_error: 17601498.0000 - val_root_mean_squared_error: 0.2423
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0255 - mean_absolute_error: 0.1246 - mean_absolute_percentage_error: 32.2389 - root_mean_squared_error: 0.1597 - val_loss: 0.0567 - val_mean_absolute_error: 0.1976 - val_mean_absolute_percentage_error: 16690315.0000 - val_root_mean_squared_error: 0.2382
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss

In [ ]:
y_pred = model_cnn_1.predict(x_test)
evaluation_cnn_1 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step


{'Mean Squared Error: ': np.float64(308.99187142125805),
 'Mean Absolute Error: ': np.float64(13.681984979279187),
 'Root Mean Absolute Error: ': 17.578164620382246,
 'Mean Absolute Percentage Error: ': np.float64(24.58539616498866)}

In [ ]:
loss, mae, mape, rmse = model_cnn_1.evaluate(x_test, y_test)
print("model cnn 1")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0246 - mean_absolute_error: 0.1222 - mean_absolute_percentage_error: 31.0923 - root_mean_squared_error: 0.1569
model cnn 1
Mean Absolute Error: 0.12216057628393173
Mean Square Error: 0.024632642045617104
Mean Absolute Percentage Error: 31.092294692993164
Root Mean Squared Error: 0.15694789588451385


## model 2 (128 & 32)
 erornya tinggi

In [ ]:
lr = 0.0003

model_cnn_2 = Sequential()
model_cnn_2.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_2.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model_cnn_2.add(Flatten())
model_cnn_2.add(Dense(7))
model_cnn_2.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_2.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 7, 128)         │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 7, 32)          │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,599 (25.78 KB)

 Trainable params: 6,599 (25.78 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_2.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.2313 - mean_absolute_error: 0.4330 - mean_absolute_percentage_error: 76.6534 - root_mean_squared_error: 0.4771 - val_loss: 0.0687 - val_mean_absolute_error: 0.2127 - val_mean_absolute_percentage_error: 17182568.0000 - val_root_mean_squared_error: 0.2620
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0372 - mean_absolute_error: 0.1514 - mean_absolute_percentage_error: 37.7572 - root_mean_squared_error: 0.1927 - val_loss: 0.0599 - val_mean_absolute_error: 0.2030 - val_mean_absolute_percentage_error: 15622445.0000 - val_root_mean_squared_error: 0.2448
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0262 - mean_absolute_error: 0.1298 - mean_absolute_percentage_error: 30.1337 - root_mean_squared_error: 0.1618 - val_loss: 0.0570 - val_mean_absolute_error: 0.1972 - val_mean_absolute_percentage_error: 16886262.0000 - val_root_mean_squared_error: 0.2387
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - lo

In [ ]:
y_pred = model_cnn_2.predict(x_test)

evaluation_cnn_2 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_2

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


{'Mean Squared Error: ': np.float64(308.7728270587435),
 'Mean Absolute Error: ': np.float64(13.709325790405273),
 'Root Mean Absolute Error: ': 17.571932934618875,
 'Mean Absolute Percentage Error: ': np.float64(24.22289758217828)}

In [ ]:
loss, mae, mape, rmse = model_cnn_2.evaluate(x_test, y_test)
print("model cnn 2")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0246 - mean_absolute_error: 0.1224 - mean_absolute_percentage_error: 30.5350 - root_mean_squared_error: 0.1569
model cnn 2
Mean Absolute Error: 0.12240468710660934
Mean Square Error: 0.024615181609988213
Mean Absolute Percentage Error: 30.53500747680664
Root Mean Squared Error: 0.15689225494861603


## model 3 (64 64)

In [ ]:
lr = 0.0003

model_cnn_3 = Sequential()
model_cnn_3.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_3.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_3.add(Flatten())
model_cnn_3.add(Dense(7))
model_cnn_3.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_3.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 7, 64)          │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 7, 64)          │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,751 (30.28 KB)

 Trainable params: 7,751 (30.28 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_3.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 0.2732 - mean_absolute_error: 0.4739 - mean_absolute_percentage_error: 86.2499 - root_mean_squared_error: 0.5185 - val_loss: 0.0610 - val_mean_absolute_error: 0.2022 - val_mean_absolute_percentage_error: 16936624.0000 - val_root_mean_squared_error: 0.2470
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0338 - mean_absolute_error: 0.1444 - mean_absolute_percentage_error: 36.9854 - root_mean_squared_error: 0.1835 - val_loss: 0.0573 - val_mean_absolute_error: 0.1991 - val_mean_absolute_percentage_error: 15920376.0000 - val_root_mean_squared_error: 0.2394
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0255 - mean_absolute_error: 0.1264 - mean_absolute_percentage_error: 30.5112 - root_mean_squared_error: 0.1596 - val_loss: 0.0569 - val_mean_absolute_error: 0.1981 - val_mean_absolute_percentage_error: 16898604.0000 - val_root_mean_squared_error: 0.2385
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - lo

In [ ]:
y_pred = model_cnn_3.predict(x_test)

evaluation_cnn_3 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_3

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


{'Mean Squared Error: ': np.float64(323.158230720397),
 'Mean Absolute Error: ': np.float64(14.176996386781031),
 'Root Mean Absolute Error: ': 17.976602313017803,
 'Mean Absolute Percentage Error: ': np.float64(25.360192021301597)}

In [ ]:
loss, mae, mape, rmse = model_cnn_3.evaluate(x_test, y_test)
print("model cnn 3")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0258 - mean_absolute_error: 0.1266 - mean_absolute_percentage_error: 32.0247 - root_mean_squared_error: 0.1605
model cnn 3
Mean Absolute Error: 0.12658031284809113
Mean Square Error: 0.025761974975466728
Mean Absolute Percentage Error: 32.02472686767578
Root Mean Squared Error: 0.16050536930561066


## model 4 (256 64)

In [ ]:
lr = 0.0003

model_cnn_4 = Sequential()
model_cnn_4.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_4.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_4.add(Flatten())
model_cnn_4.add(Dense(7))
model_cnn_4.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_4.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 7, 64)          │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,383 (83.53 KB)

 Trainable params: 21,383 (83.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_4.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.1699 - mean_absolute_error: 0.3553 - mean_absolute_percentage_error: 64.8792 - root_mean_squared_error: 0.4051 - val_loss: 0.0570 - val_mean_absolute_error: 0.1920 - val_mean_absolute_percentage_error: 18386642.0000 - val_root_mean_squared_error: 0.2388
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0292 - mean_absolute_error: 0.1361 - mean_absolute_percentage_error: 34.7164 - root_mean_squared_error: 0.1709 - val_loss: 0.0534 - val_mean_absolute_error: 0.1922 - val_mean_absolute_percentage_error: 15137688.0000 - val_root_mean_squared_error: 0.2311
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0242 - mean_absolute_error: 0.1206 - mean_absolute_percentage_error: 31.4427 - root_mean_squared_error: 0.1556 - val_loss: 0.0570 - val_mean_absolute_error: 0.1988 - val_mean_absolute_percentage_error: 14233034.0000 - val_root_mean_squared_error: 0.2387
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - lo

In [ ]:
y_pred = model_cnn_4.predict(x_test)

evaluation_cnn_4 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


{'Mean Squared Error: ': np.float64(313.0978174914591),
 'Mean Absolute Error: ': np.float64(13.376624593929368),
 'Root Mean Absolute Error: ': 17.694570282757905,
 'Mean Absolute Percentage Error: ': np.float64(24.57933044941292)}

In [ ]:
loss, mae, mape, rmse = model_cnn_4.evaluate(x_test, y_test)
print("model cnn 4")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0250 - mean_absolute_error: 0.1194 - mean_absolute_percentage_error: 31.2132 - root_mean_squared_error: 0.1580
model cnn 4
Mean Square Error: 0.02495996467769146
Mean Absolute Error: 0.11943414062261581
Mean Absolute Percentage Error: 31.213199615478516
Root Mean Squared Error: 0.15798722207546234


## model 5 (256 256)

In [ ]:
lr = 0.0003

model_cnn_5 = Sequential()
model_cnn_5.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_5.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_5.add(Flatten())
model_cnn_5.add(Dense(7))
model_cnn_5.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_5.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_7 (Conv1D)               │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 7, 256)         │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 7)              │        12,551 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,135 (313.03 KB)

 Trainable params: 80,135 (313.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_5.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1417 - mean_absolute_error: 0.3103 - mean_absolute_percentage_error: 60.4543 - root_mean_squared_error: 0.3668 - val_loss: 0.0678 - val_mean_absolute_error: 0.2205 - val_mean_absolute_percentage_error: 12332160.0000 - val_root_mean_squared_error: 0.2604
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0285 - mean_absolute_error: 0.1373 - mean_absolute_percentage_error: 29.9086 - root_mean_squared_error: 0.1687 - val_loss: 0.0551 - val_mean_absolute_error: 0.1987 - val_mean_absolute_percentage_error: 14878694.0000 - val_root_mean_squared_error: 0.2348
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0214 - mean_absolute_error: 0.1154 - mean_absolute_percentage_error: 26.6933 - root_mean_squared_error: 0.1461 - val_loss: 0.0542 - val_mean_absolute_error: 0.1966 - val_mean_absolute_percentage_error: 15635992.0000 - val_root_mean_squared_error: 0.2328
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - lo

In [ ]:
y_pred = model_cnn_5.predict(x_test)
evaluation_cnn_5 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_5

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


{'Mean Squared Error: ': np.float64(355.96332819572086),
 'Mean Absolute Error: ': np.float64(14.09022483047174),
 'Root Mean Absolute Error: ': 18.86699043821565,
 'Mean Absolute Percentage Error: ': np.float64(26.77810370094048)}

In [ ]:
loss, mae, mape, rmse = model_cnn_5.evaluate(x_test, y_test)
print("model cnn 5")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0284 - mean_absolute_error: 0.1258 - mean_absolute_percentage_error: 34.1703 - root_mean_squared_error: 0.1685
model cnn 5
Mean Square Error: 0.028377173468470573
Mean Absolute Error: 0.1258055716753006
Mean Absolute Percentage Error: 34.170265197753906
Root Mean Squared Error: 0.16845525801181793


## model 6 (128 256)

In [ ]:
lr = 0.0003

model_cnn_6 = Sequential()
model_cnn_6.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_6.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_6.add(Flatten())
model_cnn_6.add(Dense(7))
model_cnn_6.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_6.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_9 (Conv1D)               │ (None, 7, 128)         │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 7, 256)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │        12,551 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,471 (181.53 KB)

 Trainable params: 46,471 (181.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_6.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.1791 - mean_absolute_error: 0.3525 - mean_absolute_percentage_error: 66.5076 - root_mean_squared_error: 0.4126 - val_loss: 0.0572 - val_mean_absolute_error: 0.2035 - val_mean_absolute_percentage_error: 14315307.0000 - val_root_mean_squared_error: 0.2392
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0290 - mean_absolute_error: 0.1372 - mean_absolute_percentage_error: 30.0903 - root_mean_squared_error: 0.1702 - val_loss: 0.0559 - val_mean_absolute_error: 0.1951 - val_mean_absolute_percentage_error: 17221068.0000 - val_root_mean_squared_error: 0.2364
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0225 - mean_absolute_error: 0.1155 - mean_absolute_percentage_error: 29.2166 - root_mean_squared_error: 0.1500 - val_loss: 0.0558 - val_mean_absolute_error: 0.1989 - val_mean_absolute_percentage_error: 14718574.0000 - val_root_mean_squared_error: 0.2362
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss:

In [ ]:
y_pred = model_cnn_6.predict(x_test)
evaluation_cnn_6 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_6

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


{'Mean Squared Error: ': np.float64(297.0202223430667),
 'Mean Absolute Error: ': np.float64(13.155551307055415),
 'Root Mean Absolute Error: ': 17.23427463930718,
 'Mean Absolute Percentage Error: ': np.float64(24.11833917255041)}

In [ ]:
# y_pred = model_cnn_6.predict(x_test)
loss, mae, mape, rmse = model_cnn_6.evaluate(x_test, y_test)
print("model cnn 6")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0237 - mean_absolute_error: 0.1175 - mean_absolute_percentage_error: 30.6376 - root_mean_squared_error: 0.1539
model cnn 6
Mean Square Error: 0.023678269237279892
Mean Absolute Error: 0.11746027320623398
Mean Absolute Percentage Error: 30.63763999938965
Root Mean Squared Error: 0.15387745201587677


## model 7 (256 128)

In [ ]:
lr = 0.0003

model_cnn_7 = Sequential()
model_cnn_7.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_7.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_7.add(Flatten())
model_cnn_7.add(Dense(7))
model_cnn_7.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_7.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_11 (Conv1D)              │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_12 (Conv1D)              │ (None, 7, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 7)              │         6,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,967 (160.03 KB)

 Trainable params: 40,967 (160.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_7.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.1958 - mean_absolute_error: 0.3777 - mean_absolute_percentage_error: 69.3925 - root_mean_squared_error: 0.4329 - val_loss: 0.0549 - val_mean_absolute_error: 0.1958 - val_mean_absolute_percentage_error: 15031019.0000 - val_root_mean_squared_error: 0.2343
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0271 - mean_absolute_error: 0.1326 - mean_absolute_percentage_error: 30.1458 - root_mean_squared_error: 0.1645 - val_loss: 0.0542 - val_mean_absolute_error: 0.1949 - val_mean_absolute_percentage_error: 15482912.0000 - val_root_mean_squared_error: 0.2329
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0221 - mean_absolute_error: 0.1147 - mean_absolute_percentage_error: 29.7923 - root_mean_squared_error: 0.1487 - val_loss: 0.0551 - val_mean_absolute_error: 0.1970 - val_mean_absolute_percentage_error: 14136838.0000 - val_root_mean_squared_error: 0.2347
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - los

In [ ]:
y_pred = model_cnn_7.predict(x_test)
evaluation_cnn_7 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_7

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


{'Mean Squared Error: ': np.float64(356.9247375561596),
 'Mean Absolute Error: ': np.float64(14.048584607182718),
 'Root Mean Absolute Error: ': 18.89245186724475,
 'Mean Absolute Percentage Error: ': np.float64(26.568739400143286)}

In [ ]:
# y_pred = model_cnn_7.predict(x_test)
loss, mae, mape, rmse = model_cnn_7.evaluate(x_test, y_test)
print("model cnn 7")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0285 - mean_absolute_error: 0.1254 - mean_absolute_percentage_error: 33.8671 - root_mean_squared_error: 0.1687
model cnn 7
Mean Square Error: 0.028453821316361427
Mean Absolute Error: 0.1254337877035141
Mean Absolute Percentage Error: 33.867061614990234
Root Mean Squared Error: 0.16868260502815247


## model 8 (512 256)

In [ ]:
lr = 0.0003

model_cnn_8 = Sequential()
model_cnn_8.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_8.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_8.add(Flatten())
model_cnn_8.add(Dense(7))
model_cnn_8.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_8.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_13 (Conv1D)              │ (None, 7, 512)         │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_14 (Conv1D)              │ (None, 7, 256)         │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 7)              │        12,551 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147,463 (576.03 KB)

 Trainable params: 147,463 (576.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_8.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.1481 - mean_absolute_error: 0.3180 - mean_absolute_percentage_error: 61.1290 - root_mean_squared_error: 0.3751 - val_loss: 0.0707 - val_mean_absolute_error: 0.2279 - val_mean_absolute_percentage_error: 12335875.0000 - val_root_mean_squared_error: 0.2660
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0275 - mean_absolute_error: 0.1318 - mean_absolute_percentage_error: 29.6426 - root_mean_squared_error: 0.1655 - val_loss: 0.0571 - val_mean_absolute_error: 0.2011 - val_mean_absolute_percentage_error: 14662526.0000 - val_root_mean_squared_error: 0.2390
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0225 - mean_absolute_error: 0.1191 - mean_absolute_percentage_error: 28.1638 - root_mean_squared_error: 0.1500 - val_loss: 0.0553 - val_mean_absolute_error: 0.1975 - val_mean_absolute_percentage_error: 15691881.0000 - val_root_mean_squared_error: 0.2352
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - lo

In [ ]:
y_pred = model_cnn_8.predict(x_test)
evaluation_cnn_8 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_8

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


{'Mean Squared Error: ': np.float64(322.05384081145246),
 'Mean Absolute Error: ': np.float64(13.293174685264121),
 'Root Mean Absolute Error: ': 17.945858597778276,
 'Mean Absolute Percentage Error: ': np.float64(25.151665031733785)}

In [ ]:
# y_pred = model_cnn_8.predict(x_test)
loss, mae, mape, rmse = model_cnn_8.evaluate(x_test, y_test)
print("model cnn 8")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0257 - mean_absolute_error: 0.1187 - mean_absolute_percentage_error: 32.1079 - root_mean_squared_error: 0.1602
model cnn 8
Mean Square Error: 0.025673935189843178
Mean Absolute Error: 0.11868906021118164
Mean Absolute Percentage Error: 32.10792922973633
Root Mean Squared Error: 0.1602308750152588


## model 9 (512 512)

In [ ]:
lr = 0.0003

model_cnn_9 = Sequential()
model_cnn_9.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_9.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_9.add(Flatten())
model_cnn_9.add(Dense(7))
model_cnn_9.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_9.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_15 (Conv1D)              │ (None, 7, 512)         │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 7, 512)         │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 7)              │        25,095 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 291,335 (1.11 MB)

 Trainable params: 291,335 (1.11 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_9.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.1377 - mean_absolute_error: 0.3015 - mean_absolute_percentage_error: 58.6110 - root_mean_squared_error: 0.3598 - val_loss: 0.0590 - val_mean_absolute_error: 0.2052 - val_mean_absolute_percentage_error: 13940334.0000 - val_root_mean_squared_error: 0.2429
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0251 - mean_absolute_error: 0.1253 - mean_absolute_percentage_error: 30.7049 - root_mean_squared_error: 0.1585 - val_loss: 0.0557 - val_mean_absolute_error: 0.1992 - val_mean_absolute_percentage_error: 14832501.0000 - val_root_mean_squared_error: 0.2359
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0205 - mean_absolute_error: 0.1107 - mean_absolute_percentage_error: 27.9148 - root_mean_squared_error: 0.1433 - val_loss: 0.0571 - val_mean_absolute_error: 0.2022 - val_mean_absolute_percentage_error: 14534118.0000 - val_root_mean_squared_error: 0.2390
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - lo

In [ ]:
y_pred = model_cnn_9.predict(x_test)
evaluation_cnn_9 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_9

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


{'Mean Squared Error: ': np.float64(248.00585235838474),
 'Mean Absolute Error: ': np.float64(12.346905435834612),
 'Root Mean Absolute Error: ': 15.748201559491951,
 'Mean Absolute Percentage Error: ': np.float64(21.917377992616142)}

In [ ]:
# y_pred = model_cnn_9.predict(x_test)
loss, mae, mape, rmse = model_cnn_9.evaluate(x_test, y_test)
print("model cnn 9")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0198 - mean_absolute_error: 0.1102 - mean_absolute_percentage_error: 27.7148 - root_mean_squared_error: 0.1406
model cnn 9
Mean Square Error: 0.019770875573158264
Mean Absolute Error: 0.11024022847414017
Mean Absolute Percentage Error: 27.714767456054688
Root Mean Squared Error: 0.14060895144939423


## model 10 (1024 512)

In [ ]:
lr = 0.0003

model_cnn_10 = Sequential()
model_cnn_10.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_10.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_10.add(Flatten())
model_cnn_10.add(Dense(7))
model_cnn_10.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_10.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_17 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_18 (Conv1D)              │ (None, 7, 512)         │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 7)              │        25,095 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 557,063 (2.13 MB)

 Trainable params: 557,063 (2.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_10.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - loss: 0.1098 - mean_absolute_error: 0.2637 - mean_absolute_percentage_error: 53.8423 - root_mean_squared_error: 0.3210 - val_loss: 0.0534 - val_mean_absolute_error: 0.1897 - val_mean_absolute_percentage_error: 17035918.0000 - val_root_mean_squared_error: 0.2310
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - loss: 0.0226 - mean_absolute_error: 0.1187 - mean_absolute_percentage_error: 29.4617 - root_mean_squared_error: 0.1504 - val_loss: 0.0534 - val_mean_absolute_error: 0.1938 - val_mean_absolute_percentage_error: 15329679.0000 - val_root_mean_squared_error: 0.2311
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - loss: 0.0199 - mean_absolute_error: 0.1091 - mean_absolute_percentage_error: 26.7458 - root_mean_squared_error: 0.1412 - val_loss: 0.0564 - val_mean_absolute_error: 0.2020 - val_mean_absolute_percentage_error: 14785302.0000 - val_root_mean_squared_error: 0.2375
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - 

In [ ]:
y_pred = model_cnn_10.predict(x_test)
evaluation_cnn_10 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_10

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


{'Mean Squared Error: ': np.float64(309.8490671182868),
 'Mean Absolute Error: ': np.float64(12.92501332808514),
 'Root Mean Absolute Error: ': 17.60253013399741,
 'Mean Absolute Percentage Error: ': np.float64(24.48451157723588)}

In [ ]:
# y_pred = model_cnn_10.predict(x_test)
loss, mae, mape, rmse = model_cnn_10.evaluate(x_test, y_test)
print("model cnn 10")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0247 - mean_absolute_error: 0.1154 - mean_absolute_percentage_error: 31.2801 - root_mean_squared_error: 0.1572
model cnn 10
Mean Square Error: 0.02470097504556179
Mean Absolute Error: 0.11540190130472183
Mean Absolute Percentage Error: 31.28013801574707
Root Mean Squared Error: 0.15716543793678284


## model 11 (128 256 128)

In [ ]:
lr = 0.0003

model_cnn_11 = Sequential()
model_cnn_11.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_11.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_11.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_11.add(Flatten())
model_cnn_11.add(Dense(7))
model_cnn_11.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_11.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_19 (Conv1D)              │ (None, 7, 128)         │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 7, 256)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 7, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 7)              │         6,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,095 (285.53 KB)

 Trainable params: 73,095 (285.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_11.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.1300 - mean_absolute_error: 0.2989 - mean_absolute_percentage_error: 58.2481 - root_mean_squared_error: 0.3525 - val_loss: 0.0716 - val_mean_absolute_error: 0.2276 - val_mean_absolute_percentage_error: 12242217.0000 - val_root_mean_squared_error: 0.2676
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0341 - mean_absolute_error: 0.1515 - mean_absolute_percentage_error: 31.0968 - root_mean_squared_error: 0.1838 - val_loss: 0.0557 - val_mean_absolute_error: 0.1970 - val_mean_absolute_percentage_error: 15374989.0000 - val_root_mean_squared_error: 0.2359
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0214 - mean_absolute_error: 0.1152 - mean_absolute_percentage_error: 27.1955 - root_mean_squared_error: 0.1464 - val_loss: 0.0544 - val_mean_absolute_error: 0.1935 - val_mean_absolute_percentage_error: 16176538.0000 - val_root_mean_squared_error: 0.2331
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - lo

In [ ]:
y_pred = model_cnn_11.predict(x_test)
evaluation_cnn_11 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_11

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


{'Mean Squared Error: ': np.float64(407.9019675863541),
 'Mean Absolute Error: ': np.float64(15.284369468688965),
 'Root Mean Absolute Error: ': 20.196583067102072,
 'Mean Absolute Percentage Error: ': np.float64(29.218351574776964)}

In [ ]:
# y_pred = model_cnn_11.predict(x_test)
loss, mae, mape, rmse = model_cnn_11.evaluate(x_test, y_test)
print("model cnn 11")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0325 - mean_absolute_error: 0.1365 - mean_absolute_percentage_error: 37.2841 - root_mean_squared_error: 0.1803
model cnn 11
Mean Square Error: 0.0325176902115345
Mean Absolute Error: 0.1364675760269165
Mean Absolute Percentage Error: 37.284122467041016
Root Mean Squared Error: 0.18032662570476532


## model 12 (256 128 64)

In [ ]:
lr = 0.0003

model_cnn_12 = Sequential()
model_cnn_12.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_12.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_12.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_12.add(Flatten())
model_cnn_12.add(Dense(7))
model_cnn_12.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_12.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_22 (Conv1D)              │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_23 (Conv1D)              │ (None, 7, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_24 (Conv1D)              │ (None, 7, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,087 (180.03 KB)

 Trainable params: 46,087 (180.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_12.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.2551 - mean_absolute_error: 0.4508 - mean_absolute_percentage_error: 81.1069 - root_mean_squared_error: 0.4987 - val_loss: 0.0802 - val_mean_absolute_error: 0.2239 - val_mean_absolute_percentage_error: 20097286.0000 - val_root_mean_squared_error: 0.2831
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0312 - mean_absolute_error: 0.1410 - mean_absolute_percentage_error: 34.4680 - root_mean_squared_error: 0.1762 - val_loss: 0.0535 - val_mean_absolute_error: 0.1909 - val_mean_absolute_percentage_error: 15784778.0000 - val_root_mean_squared_error: 0.2314
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0231 - mean_absolute_error: 0.1181 - mean_absolute_percentage_error: 31.5569 - root_mean_squared_error: 0.1519 - val_loss: 0.0548 - val_mean_absolute_error: 0.1917 - val_mean_absolute_percentage_error: 14917540.0000 - val_root_mean_squared_error: 0.2340
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - lo

In [ ]:
y_pred = model_cnn_12.predict(x_test)
evaluation_cnn_12 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_12

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step


{'Mean Squared Error: ': np.float64(320.5368921308092),
 'Mean Absolute Error: ': np.float64(14.266113845669494),
 'Root Mean Absolute Error: ': 17.90354412206726,
 'Mean Absolute Percentage Error: ': np.float64(25.274283338217508)}

In [ ]:
# y_pred = model_cnn_12.predict(x_test)
loss, mae, mape, rmse = model_cnn_12.evaluate(x_test, y_test)
print("model cnn 12")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0256 - mean_absolute_error: 0.1274 - mean_absolute_percentage_error: 31.8379 - root_mean_squared_error: 0.1599
model cnn 12
Mean Square Error: 0.025553002953529358
Mean Absolute Error: 0.1273760050535202
Mean Absolute Percentage Error: 31.837936401367188
Root Mean Squared Error: 0.15985307097434998


## model 13 (256 128 256)

In [ ]:
lr = 0.0003

model_cnn_13 = Sequential()
model_cnn_13.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_13.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_13.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_13.add(Flatten())
model_cnn_13.add(Dense(7))
model_cnn_13.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_13.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_25 (Conv1D)              │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_26 (Conv1D)              │ (None, 7, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_27 (Conv1D)              │ (None, 7, 256)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 7)              │        12,551 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,263 (313.53 KB)

 Trainable params: 80,263 (313.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_13.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=50,
    batch_size=42
)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.1751 - mean_absolute_error: 0.3575 - mean_absolute_percentage_error: 67.9682 - root_mean_squared_error: 0.4097 - val_loss: 0.0711 - val_mean_absolute_error: 0.2268 - val_mean_absolute_percentage_error: 12554454.0000 - val_root_mean_squared_error: 0.2666
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0321 - mean_absolute_error: 0.1461 - mean_absolute_percentage_error: 31.1026 - root_mean_squared_error: 0.1788 - val_loss: 0.0559 - val_mean_absolute_error: 0.1972 - val_mean_absolute_percentage_error: 14944821.0000 - val_root_mean_squared_error: 0.2365
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0220 - mean_absolute_error: 0.1179 - mean_absolute_percentage_error: 27.2318 - root_mean_squared_error: 0.1482 - val_loss: 0.0552 - val_mean_absolute_error: 0.1964 - val_mean_absolute_percentage_error: 15005233.0000 - val_root_mean_squared_error: 0.2349
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - lo

In [ ]:
y_pred = model_cnn_13.predict(x_test)
evaluation_cnn_13 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_13

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step


{'Mean Squared Error: ': np.float64(623.7250012213093),
 'Mean Absolute Error: ': np.float64(19.00586840571189),
 'Root Mean Absolute Error: ': 24.974487006169102,
 'Mean Absolute Percentage Error: ': np.float64(35.72098593041443)}

In [ ]:
# y_pred = model_cnn_13.predict(x_test)
loss, mae, mape, rmse = model_cnn_13.evaluate(x_test, y_test)
print("model cnn 13")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0497 - mean_absolute_error: 0.1697 - mean_absolute_percentage_error: 45.4759 - root_mean_squared_error: 0.2230
model cnn 13
Mean Square Error: 0.04972296953201294
Mean Absolute Error: 0.16969525814056396
Mean Absolute Percentage Error: 45.47587203979492
Root Mean Squared Error: 0.22298647463321686


## model 14 (256 128 64, epoch 50)

In [ ]:
lr = 0.0003

model_cnn_14 = Sequential()
model_cnn_14.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_14.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_14.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_14.add(Flatten())
model_cnn_14.add(Dense(7))
model_cnn_14.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_14.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_28 (Conv1D)              │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_29 (Conv1D)              │ (None, 7, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_30 (Conv1D)              │ (None, 7, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,087 (180.03 KB)

 Trainable params: 46,087 (180.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_14.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=30,
    batch_size=42
)

Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.2276 - mean_absolute_error: 0.4219 - mean_absolute_percentage_error: 75.0125 - root_mean_squared_error: 0.4708 - val_loss: 0.0557 - val_mean_absolute_error: 0.1963 - val_mean_absolute_percentage_error: 16083540.0000 - val_root_mean_squared_error: 0.2361
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0273 - mean_absolute_error: 0.1316 - mean_absolute_percentage_error: 31.1771 - root_mean_squared_error: 0.1650 - val_loss: 0.0541 - val_mean_absolute_error: 0.1939 - val_mean_absolute_percentage_error: 15635285.0000 - val_root_mean_squared_error: 0.2327
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0215 - mean_absolute_error: 0.1143 - mean_absolute_percentage_error: 29.1047 - root_mean_squared_error: 0.1465 - val_loss: 0.0548 - val_mean_absolute_error: 0.1969 - val_mean_absolute_percentage_error: 14301085.0000 - val_root_mean_squared_error: 0.2341
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - los

In [ ]:
y_pred = model_cnn_14.predict(x_test)
evaluation_cnn_14 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_14

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step


{'Mean Squared Error: ': np.float64(326.3614521036704),
 'Mean Absolute Error: ': np.float64(13.501906550660424),
 'Root Mean Absolute Error: ': 18.065476802555487,
 'Mean Absolute Percentage Error: ': np.float64(25.21498846913357)}

In [ ]:
# y_pred = model_cnn_14.predict(x_test)
loss, mae, mape, rmse = model_cnn_14.evaluate(x_test, y_test)
print("model cnn 14")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0260 - mean_absolute_error: 0.1206 - mean_absolute_percentage_error: 32.1010 - root_mean_squared_error: 0.1613
model cnn 14
Mean Square Error: 0.026017332449555397
Mean Absolute Error: 0.12055272608995438
Mean Absolute Percentage Error: 32.100975036621094
Root Mean Squared Error: 0.16129888594150543


## model 15 (1024 1024)

In [ ]:
lr = 0.0003

model_cnn_15 = Sequential()
model_cnn_15.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_15.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_15.add(Flatten())
model_cnn_15.add(Dense(7))
model_cnn_15.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_15.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_31 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_32 (Conv1D)              │ (None, 7, 1024)        │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 7168)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 7)              │        50,183 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,106,951 (4.22 MB)

 Trainable params: 1,106,951 (4.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_15.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 257ms/step - loss: 0.1209 - mean_absolute_error: 0.2743 - mean_absolute_percentage_error: 54.2968 - root_mean_squared_error: 0.3357 - val_loss: 0.0608 - val_mean_absolute_error: 0.1975 - val_mean_absolute_percentage_error: 18729472.0000 - val_root_mean_squared_error: 0.2466
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 240ms/step - loss: 0.0251 - mean_absolute_error: 0.1252 - mean_absolute_percentage_error: 29.1384 - root_mean_squared_error: 0.1582 - val_loss: 0.0556 - val_mean_absolute_error: 0.2000 - val_mean_absolute_percentage_error: 15195482.0000 - val_root_mean_squared_error: 0.2359
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 296ms/step - loss: 0.0216 - mean_absolute_error: 0.1148 - mean_absolute_percentage_error: 28.2844 - root_mean_squared_error: 0.1470 - val_loss: 0.0576 - val_mean_absolute_error: 0.2047 - val_mean_absolute_percentage_error: 14712259.0000 - val_root_mean_squared_error: 0.2400
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 252ms/step 

In [ ]:
y_pred = model_cnn_15.predict(x_test)
evaluation_cnn_15 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_15

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


{'Mean Squared Error: ': np.float64(262.3581179722084),
 'Mean Absolute Error: ': np.float64(12.571553716854172),
 'Root Mean Absolute Error: ': 16.197472579764053,
 'Mean Absolute Percentage Error: ': np.float64(22.314289338570518)}

In [ ]:
# y_pred = model_cnn_15.predict(x_test)
loss, mae, mape, rmse = model_cnn_15.evaluate(x_test, y_test)
print("model cnn 15")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0209 - mean_absolute_error: 0.1122 - mean_absolute_percentage_error: 28.2311 - root_mean_squared_error: 0.1446
model cnn 15
Mean Square Error: 0.02091502957046032
Mean Absolute Error: 0.11224602162837982
Mean Absolute Percentage Error: 28.231054306030273
Root Mean Squared Error: 0.14462029933929443


## model 16 (2048 512)

In [ ]:
lr = 0.0003

model_cnn_16 = Sequential()
model_cnn_16.add(Conv1D(filters=2048, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_16.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_16.add(Flatten())
model_cnn_16.add(Dense(7))
model_cnn_16.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_16.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_33 (Conv1D)              │ (None, 7, 2048)        │        14,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_34 (Conv1D)              │ (None, 7, 512)         │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 7)              │        25,095 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,088,519 (4.15 MB)

 Trainable params: 1,088,519 (4.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_16.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 275ms/step - loss: 0.1277 - mean_absolute_error: 0.2885 - mean_absolute_percentage_error: 55.8747 - root_mean_squared_error: 0.3451 - val_loss: 0.0609 - val_mean_absolute_error: 0.1995 - val_mean_absolute_percentage_error: 18423906.0000 - val_root_mean_squared_error: 0.2467
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 271ms/step - loss: 0.0277 - mean_absolute_error: 0.1305 - mean_absolute_percentage_error: 35.1175 - root_mean_squared_error: 0.1660 - val_loss: 0.0544 - val_mean_absolute_error: 0.1954 - val_mean_absolute_percentage_error: 16122424.0000 - val_root_mean_squared_error: 0.2333
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 248ms/step - loss: 0.0206 - mean_absolute_error: 0.1114 - mean_absolute_percentage_error: 28.3482 - root_mean_squared_error: 0.1435 - val_loss: 0.0567 - val_mean_absolute_error: 0.2021 - val_mean_absolute_percentage_error: 14852440.0000 - val_root_mean_squared_error: 0.2382
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 274ms/step 

In [ ]:
y_pred = model_cnn_16.predict(x_test)
evaluation_cnn_16 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_16

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


{'Mean Squared Error: ': np.float64(355.5798910350058),
 'Mean Absolute Error: ': np.float64(14.188494098429777),
 'Root Mean Absolute Error: ': 18.856826112445482,
 'Mean Absolute Percentage Error: ': np.float64(26.8978685202793)}

In [ ]:
# y_pred = model_cnn_15.predict(x_test)
loss, mae, mape, rmse = model_cnn_16.evaluate(x_test, y_test)
print("model cnn 16")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0283 - mean_absolute_error: 0.1267 - mean_absolute_percentage_error: 34.3115 - root_mean_squared_error: 0.1684
model cnn 16
Mean Square Error: 0.028346609324216843
Mean Absolute Error: 0.12668298184871674
Mean Absolute Percentage Error: 34.31153106689453
Root Mean Squared Error: 0.1683645099401474


## model 17 (1024 512)

In [ ]:
lr = 0.0003

model_cnn_17 = Sequential()
model_cnn_17.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_17.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_17.add(Flatten())
model_cnn_17.add(Dense(7))
model_cnn_17.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_17.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_35 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_36 (Conv1D)              │ (None, 7, 512)         │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 7)              │        25,095 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 557,063 (2.13 MB)

 Trainable params: 557,063 (2.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_17.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=40,
    batch_size=42
)

Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - loss: 0.1279 - mean_absolute_error: 0.2878 - mean_absolute_percentage_error: 57.0378 - root_mean_squared_error: 0.3450 - val_loss: 0.0544 - val_mean_absolute_error: 0.1926 - val_mean_absolute_percentage_error: 17343620.0000 - val_root_mean_squared_error: 0.2333
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - loss: 0.0225 - mean_absolute_error: 0.1166 - mean_absolute_percentage_error: 28.2473 - root_mean_squared_error: 0.1499 - val_loss: 0.0549 - val_mean_absolute_error: 0.1977 - val_mean_absolute_percentage_error: 15828556.0000 - val_root_mean_squared_error: 0.2343
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step - loss: 0.0201 - mean_absolute_error: 0.1101 - mean_absolute_percentage_error: 28.3008 - root_mean_squared_error: 0.1419 - val_loss: 0.0566 - val_mean_absolute_error: 0.2026 - val_mean_absolute_percentage_error: 15208618.0000 - val_root_mean_squared_error: 0.2379
Epoch 4/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step -

In [ ]:
y_pred = model_cnn_17.predict(x_test)
evaluation_cnn_17 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_17

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


{'Mean Squared Error: ': np.float64(386.56729889748334),
 'Mean Absolute Error: ': np.float64(14.146349303576411),
 'Root Mean Absolute Error: ': 19.661314780489207,
 'Mean Absolute Percentage Error: ': np.float64(27.17288654530953)}

In [ ]:
# y_pred = model_cnn_17.predict(x_test)
loss, mae, mape, rmse = model_cnn_17.evaluate(x_test, y_test)
print("model cnn 17")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0308 - mean_absolute_error: 0.1263 - mean_absolute_percentage_error: 34.8105 - root_mean_squared_error: 0.1755
model cnn 17
Mean Square Error: 0.030816907063126564
Mean Absolute Error: 0.1263066828250885
Mean Absolute Percentage Error: 34.81047439575195
Root Mean Squared Error: 0.17554745078086853


## model 18 (1024 512)

In [ ]:
lr = 0.0003

model_cnn_18 = Sequential()
model_cnn_18.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_18.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_18.add(Flatten())
model_cnn_18.add(Dense(7))
model_cnn_18.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_18.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_37 (Conv1D)              │ (None, 7, 128)         │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_38 (Conv1D)              │ (None, 7, 256)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 7)              │        12,551 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,471 (181.53 KB)

 Trainable params: 46,471 (181.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_18.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=15,
    batch_size=64
)

Epoch 1/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.1938 - mean_absolute_error: 0.3818 - mean_absolute_percentage_error: 67.7230 - root_mean_squared_error: 0.4323 - val_loss: 0.0973 - val_mean_absolute_error: 0.2466 - val_mean_absolute_percentage_error: 20138704.0000 - val_root_mean_squared_error: 0.3119
Epoch 2/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0386 - mean_absolute_error: 0.1567 - mean_absolute_percentage_error: 39.5126 - root_mean_squared_error: 0.1958 - val_loss: 0.0617 - val_mean_absolute_error: 0.2121 - val_mean_absolute_percentage_error: 12361111.0000 - val_root_mean_squared_error: 0.2484
Epoch 3/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0271 - mean_absolute_error: 0.1332 - mean_absolute_percentage_error: 29.3987 - root_mean_squared_error: 0.1647 - val_loss: 0.0553 - val_mean_absolute_error: 0.1949 - val_mean_absolute_percentage_error: 16686713.0000 - val_root_mean_squared_error: 0.2352
Epoch 4/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - lo

In [ ]:
y_pred = model_cnn_18.predict(x_test)
evaluation_cnn_18 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_18

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


{'Mean Squared Error: ': np.float64(313.784171152303),
 'Mean Absolute Error: ': np.float64(13.686381456803302),
 'Root Mean Absolute Error: ': 17.713954136564286,
 'Mean Absolute Percentage Error: ': np.float64(24.85618467718476)}

In [ ]:
# y_pred = model_cnn_18.predict(x_test)
loss, mae, mape, rmse = model_cnn_18.evaluate(x_test, y_test)
print("model cnn 18")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0250 - mean_absolute_error: 0.1222 - mean_absolute_percentage_error: 31.4906 - root_mean_squared_error: 0.1582
model cnn 18
Mean Square Error: 0.025014681741595268
Mean Absolute Error: 0.12219983339309692
Mean Absolute Percentage Error: 31.490585327148438
Root Mean Squared Error: 0.15816029906272888


## model 19 (256 128 512, epoch 50)

In [ ]:
lr = 0.0003

model_cnn_19 = Sequential()
model_cnn_19.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_19.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_19.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_19.add(Flatten())
model_cnn_19.add(Dense(7))
model_cnn_19.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_19.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_39 (Conv1D)              │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_40 (Conv1D)              │ (None, 7, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_41 (Conv1D)              │ (None, 7, 512)         │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 7)              │        25,095 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 125,831 (491.53 KB)

 Trainable params: 125,831 (491.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_19.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=50,
    batch_size=42
)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.1412 - mean_absolute_error: 0.3094 - mean_absolute_percentage_error: 59.2217 - root_mean_squared_error: 0.3656 - val_loss: 0.0585 - val_mean_absolute_error: 0.2011 - val_mean_absolute_percentage_error: 14237769.0000 - val_root_mean_squared_error: 0.2419
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0245 - mean_absolute_error: 0.1218 - mean_absolute_percentage_error: 30.1596 - root_mean_squared_error: 0.1565 - val_loss: 0.0538 - val_mean_absolute_error: 0.1921 - val_mean_absolute_percentage_error: 15849830.0000 - val_root_mean_squared_error: 0.2321
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0222 - mean_absolute_error: 0.1146 - mean_absolute_percentage_error: 30.9396 - root_mean_squared_error: 0.1489 - val_loss: 0.0533 - val_mean_absolute_error: 0.1919 - val_mean_absolute_percentage_error: 16415479.0000 - val_root_mean_squared_error: 0.2309
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - lo

In [ ]:
y_pred = model_cnn_19.predict(x_test)
evaluation_cnn_19 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_19

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step


{'Mean Squared Error: ': np.float64(600.5764414323583),
 'Mean Absolute Error: ': np.float64(18.096356450294962),
 'Root Mean Absolute Error: ': 24.506661164515215,
 'Mean Absolute Percentage Error: ': np.float64(34.04964176969604)}

In [ ]:
# y_pred = model_cnn_13.predict(x_test)
loss, mae, mape, rmse = model_cnn_19.evaluate(x_test, y_test)
print("model cnn 19")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0479 - mean_absolute_error: 0.1616 - mean_absolute_percentage_error: 43.3706 - root_mean_squared_error: 0.2188
model cnn 19
Mean Square Error: 0.04787757992744446
Mean Absolute Error: 0.1615746021270752
Mean Absolute Percentage Error: 43.370609283447266
Root Mean Squared Error: 0.21880945563316345


## model 20 (256 128 512, epoch 50)

In [ ]:
lr = 0.0003

model_cnn_20 = Sequential()
model_cnn_20.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_20.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_20.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_20.add(Flatten())
model_cnn_20.add(Dense(7))
model_cnn_20.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_20.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_42 (Conv1D)              │ (None, 7, 512)         │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_43 (Conv1D)              │ (None, 7, 128)         │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_44 (Conv1D)              │ (None, 7, 512)         │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 7)              │        25,095 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,391 (626.53 KB)

 Trainable params: 160,391 (626.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_20.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=50,
    batch_size=42
)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1574 - mean_absolute_error: 0.3329 - mean_absolute_percentage_error: 63.8871 - root_mean_squared_error: 0.3871 - val_loss: 0.0581 - val_mean_absolute_error: 0.2009 - val_mean_absolute_percentage_error: 14588245.0000 - val_root_mean_squared_error: 0.2411
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0249 - mean_absolute_error: 0.1230 - mean_absolute_percentage_error: 30.4118 - root_mean_squared_error: 0.1577 - val_loss: 0.0526 - val_mean_absolute_error: 0.1871 - val_mean_absolute_percentage_error: 16703263.0000 - val_root_mean_squared_error: 0.2293
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0216 - mean_absolute_error: 0.1130 - mean_absolute_percentage_error: 29.7650 - root_mean_squared_error: 0.1470 - val_loss: 0.0533 - val_mean_absolute_error: 0.1889 - val_mean_absolute_percentage_error: 17448888.0000 - val_root_mean_squared_error: 0.2309
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - lo

In [ ]:
y_pred = model_cnn_20.predict(x_test)
evaluation_cnn_20 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_cnn_20

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


{'Mean Squared Error: ': np.float64(307.8782494393687),
 'Mean Absolute Error: ': np.float64(13.23758481473339),
 'Root Mean Absolute Error: ': 17.546459740909807,
 'Mean Absolute Percentage Error: ': np.float64(24.71139895319035)}

In [ ]:
# y_pred = model_cnn_13.predict(x_test)
loss, mae, mape, rmse = model_cnn_20.evaluate(x_test, y_test)
print("model cnn 20")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0245 - mean_absolute_error: 0.1182 - mean_absolute_percentage_error: 31.5259 - root_mean_squared_error: 0.1567
model cnn 20
Mean Square Error: 0.024543864652514458
Mean Absolute Error: 0.11819272488355637
Mean Absolute Percentage Error: 31.525876998901367
Root Mean Squared Error: 0.15666481852531433


# CNN Hyperparameter

## gridsearch

In [ ]:
# from keras.callbacks import EarlyStopping
# from sklearn.model_selection import GridSearchCV
# from scikeras.wrappers import KerasRegressor

# # 1. Define your model builder function
# def build_model(filters1=64, filters2=32, kernel_size=1, lr=0.001):
#     model = Sequential()
#     model.add(Input(shape=(7, 11)))
#     model.add(Conv1D(filters=filters1, kernel_size=kernel_size, activation='relu'))
#     model.add(Conv1D(filters=filters2, kernel_size=kernel_size, activation='relu'))
#     model.add(Flatten())
#     model.add(Dense(7))
#     optimizer = Adam(learning_rate=lr)
#     model.compile(loss='mse', optimizer=optimizer,
#                   metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
#     return model

# # 2. Set up KerasRegressor
# model = KerasRegressor(model=build_model, verbose=0)

# # 3. Define early stopping
# early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # 4. Grid of parameters
# param_grid = {
#     'model__filters1': [256, 512, 1024],
#     'model__filters2': [32, 64, 128],
#     'model__kernel_size': [1],
#     'model__lr': [0.0003],
#     'batch_size': [42],
#     'epochs': [50]  # set high epochs so early stopping can interrupt early
# }

# # 5. GridSearchCV with early stopping passed via fit_params
# grid = GridSearchCV(estimator=model, param_grid=param_grid,
#                     scoring='neg_mean_squared_error',
#                     cv=3,
#                     refit=True)

# # 6. Fit with early stopping
# grid_result_cnn = grid.fit(x_train, y_train, callbacks=[early_stop], validation_data=(x_val, y_val))


In [ ]:
# print("Best parameters found: ", grid_result_cnn.best_params_)
# # print("Best MSE: ", -grid_result.best_score_)

In [ ]:
# best_model = grid_result_cnn.best_estimator_
# loss, mae, mape, rmse = best_model.model_.evaluate(x_test, y_test)
# print("Test MSE:", loss)
# print("Test MAE:", mae)
# print("Test MAPE:", mape)
# print("Test RMSE:", rmse)


# LSTM Only

## model 1 (LSTM=32)

In [ ]:
lr = 0.0003

model_lstm_1 = Sequential()
model_lstm_1.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_1.add(LSTM(32, activation='relu'))
model_lstm_1.add(Flatten())
model_lstm_1.add(Dense(7))
model_lstm_1.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_1.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         4,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_20 (Flatten)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,223 (20.40 KB)

 Trainable params: 5,223 (20.40 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_1.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.2419 - mean_absolute_error: 0.4501 - mean_absolute_percentage_error: 78.2251 - root_mean_squared_error: 0.4914 - val_loss: 0.1751 - val_mean_absolute_error: 0.3577 - val_mean_absolute_percentage_error: 6443156.0000 - val_root_mean_squared_error: 0.4185
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1329 - mean_absolute_error: 0.3050 - mean_absolute_percentage_error: 53.9187 - root_mean_squared_error: 0.3640 - val_loss: 0.1012 - val_mean_absolute_error: 0.2646 - val_mean_absolute_percentage_error: 12391855.0000 - val_root_mean_squared_error: 0.3181
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0554 - mean_absolute_error: 0.1822 - mean_absolute_percentage_error: 37.7273 - root_mean_squared_error: 0.2349 - val_loss: 0.0639 - val_mean_absolute_error: 0.2120 - val_mean_absolute_percentage_error: 15050977.0000 - val_root_mean_squared_error: 0.2528
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss

In [ ]:
y_pred = model_lstm_1.predict(x_test)

evaluation_lstm_1 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step


{'Mean Squared Error: ': np.float64(340.11774836036926),
 'Mean Absolute Error: ': np.float64(14.84447903535804),
 'Root Mean Absolute Error: ': 18.44228153890861,
 'Mean Absolute Percentage Error: ': np.float64(23.803440264229536)}

In [ ]:
loss, mae, mape, rmse = model_lstm_1.evaluate(x_test, y_test)
print("model lstm 1")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0271 - mean_absolute_error: 0.1325 - mean_absolute_percentage_error: 29.3235 - root_mean_squared_error: 0.1647
model lstm 1
Mean Square Error: 0.027113977819681168
Mean Absolute Error: 0.13253998756408691
Mean Absolute Percentage Error: 29.32354736328125
Root Mean Squared Error: 0.16466322541236877


## model 2 (LSTM=32. 32)

In [ ]:
lr = 0.0003

model_lstm_2 = Sequential()
model_lstm_2.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_2.add(LSTM(32, activation='relu', return_sequences=True))
model_lstm_2.add(LSTM(32, activation='relu'))
model_lstm_2.add(Flatten())
model_lstm_2.add(Dense(7))
model_lstm_2.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_2.summary()

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 7, 32)          │         4,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,543 (52.90 KB)

 Trainable params: 13,543 (52.90 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_2.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - loss: 0.3571 - mean_absolute_error: 0.5685 - mean_absolute_percentage_error: 101.3269 - root_mean_squared_error: 0.5974 - val_loss: 0.2886 - val_mean_absolute_error: 0.4923 - val_mean_absolute_percentage_error: 1922210.6250 - val_root_mean_squared_error: 0.5372
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.2435 - mean_absolute_error: 0.4577 - mean_absolute_percentage_error: 79.4178 - root_mean_squared_error: 0.4931 - val_loss: 0.1382 - val_mean_absolute_error: 0.3142 - val_mean_absolute_percentage_error: 8029880.5000 - val_root_mean_squared_error: 0.3717
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0891 - mean_absolute_error: 0.2350 - mean_absolute_percentage_error: 48.0718 - root_mean_squared_error: 0.2981 - val_loss: 0.0783 - val_mean_absolute_error: 0.2329 - val_mean_absolute_percentage_error: 11527804.0000 - val_root_mean_squared_error: 0.2798
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - los

In [ ]:
y_pred = model_lstm_2.predict(x_test)

evaluation_lstm_2 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_2

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step


{'Mean Squared Error: ': np.float64(392.0863969822591),
 'Mean Absolute Error: ': np.float64(16.510735317152374),
 'Root Mean Absolute Error: ': 19.801171606302972,
 'Mean Absolute Percentage Error: ': np.float64(27.492623217690426)}

In [ ]:
loss, mae, mape, rmse = model_lstm_2.evaluate(x_test, y_test)
print("model lstm 2")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0313 - mean_absolute_error: 0.1474 - mean_absolute_percentage_error: 34.1190 - root_mean_squared_error: 0.1768
model lstm 2
Mean Square Error: 0.031256888061761856
Mean Absolute Error: 0.14741727709770203
Mean Absolute Percentage Error: 34.11897659301758
Root Mean Squared Error: 0.17679618299007416


## model 3 (LSTM=256. 32)

In [ ]:
lr = 0.0003

model_lstm_3 = Sequential()
model_lstm_3.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_3.add(LSTM(256, activation='relu', return_sequences=True))
model_lstm_3.add(LSTM(32, activation='relu'))
model_lstm_3.add(Flatten())
model_lstm_3.add(Dense(7))
model_lstm_3.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_3.summary()

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 7, 256)         │       269,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 32)             │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 306,535 (1.17 MB)

 Trainable params: 306,535 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_3.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - loss: 0.2753 - mean_absolute_error: 0.4878 - mean_absolute_percentage_error: 86.6109 - root_mean_squared_error: 0.5232 - val_loss: 0.1044 - val_mean_absolute_error: 0.2619 - val_mean_absolute_percentage_error: 15200788.0000 - val_root_mean_squared_error: 0.3230
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 0.0629 - mean_absolute_error: 0.1990 - mean_absolute_percentage_error: 41.3669 - root_mean_squared_error: 0.2492 - val_loss: 0.0519 - val_mean_absolute_error: 0.1917 - val_mean_absolute_percentage_error: 15372832.0000 - val_root_mean_squared_error: 0.2278
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0251 - mean_absolute_error: 0.1239 - mean_absolute_percentage_error: 29.9047 - root_mean_squared_error: 0.1584 - val_loss: 0.0533 - val_mean_absolute_error: 0.1926 - val_mean_absolute_percentage_error: 14787811.0000 - val_root_mean_squared_error: 0.2308
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - lo

In [ ]:
y_pred = model_lstm_3.predict(x_test)

evaluation_lstm_3 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_3

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step


{'Mean Squared Error: ': np.float64(395.8629015229882),
 'Mean Absolute Error: ': np.float64(15.105303316700216),
 'Root Mean Absolute Error: ': 19.896303715087086,
 'Mean Absolute Percentage Error: ': np.float64(28.91552608709616)}

In [ ]:
loss, mae, mape, rmse = model_lstm_3.evaluate(x_test, y_test)
print("model lstm 3")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0316 - mean_absolute_error: 0.1349 - mean_absolute_percentage_error: 36.8759 - root_mean_squared_error: 0.1776
model lstm 3
Mean Square Error: 0.031557947397232056
Mean Absolute Error: 0.1348687708377838
Mean Absolute Percentage Error: 36.87588119506836
Root Mean Squared Error: 0.17764556407928467


## model 4 (LSTM=256. 64)

In [ ]:
lr = 0.0003

model_lstm_4 = Sequential()
model_lstm_4.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_4.add(LSTM(256, activation='relu', return_sequences=True))
model_lstm_4.add(LSTM(64, activation='relu'))
model_lstm_4.add(Flatten())
model_lstm_4.add(Dense(7))
model_lstm_4.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_4.summary()

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 7, 256)         │       269,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 64)             │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 351,943 (1.34 MB)

 Trainable params: 351,943 (1.34 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_4.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.2793 - mean_absolute_error: 0.4904 - mean_absolute_percentage_error: 87.0582 - root_mean_squared_error: 0.5268 - val_loss: 0.0757 - val_mean_absolute_error: 0.2259 - val_mean_absolute_percentage_error: 15547745.0000 - val_root_mean_squared_error: 0.2751
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0366 - mean_absolute_error: 0.1491 - mean_absolute_percentage_error: 32.7817 - root_mean_squared_error: 0.1909 - val_loss: 0.0564 - val_mean_absolute_error: 0.1968 - val_mean_absolute_percentage_error: 14603779.0000 - val_root_mean_squared_error: 0.2374
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0248 - mean_absolute_error: 0.1239 - mean_absolute_percentage_error: 30.8335 - root_mean_squared_error: 0.1575 - val_loss: 0.0534 - val_mean_absolute_error: 0.1915 - val_mean_absolute_percentage_error: 14955815.0000 - val_root_mean_squared_error: 0.2311
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - lo

In [ ]:
y_pred = model_lstm_4.predict(x_test)

evaluation_lstm_4 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step


{'Mean Squared Error: ': np.float64(385.0995006265619),
 'Mean Absolute Error: ': np.float64(14.782673660589724),
 'Root Mean Absolute Error: ': 19.623952217292057,
 'Mean Absolute Percentage Error: ': np.float64(28.017642150445106)}

In [ ]:
loss, mae, mape, rmse = model_lstm_4.evaluate(x_test, y_test)
print("model lstm 4")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0307 - mean_absolute_error: 0.1320 - mean_absolute_percentage_error: 35.6844 - root_mean_squared_error: 0.1752
model lstm 4
Mean Square Error: 0.030699891969561577
Mean Absolute Error: 0.13198815286159515
Mean Absolute Percentage Error: 35.6844482421875
Root Mean Squared Error: 0.17521384358406067


## model 5 (LSTM=512. 128)

In [ ]:
lr = 0.0003

model_lstm_5 = Sequential()
model_lstm_5.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_5.add(LSTM(512, activation='relu', return_sequences=True))
model_lstm_5.add(LSTM(128, activation='relu'))
model_lstm_5.add(Flatten())
model_lstm_5.add(Dense(7))
model_lstm_5.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_5.summary()

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                   │ (None, 7, 512)         │     1,062,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)            │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_24 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,392,007 (5.31 MB)

 Trainable params: 1,392,007 (5.31 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_5.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 155ms/step - loss: 0.2501 - mean_absolute_error: 0.4508 - mean_absolute_percentage_error: 81.3857 - root_mean_squared_error: 0.4958 - val_loss: 0.0778 - val_mean_absolute_error: 0.2382 - val_mean_absolute_percentage_error: 10982953.0000 - val_root_mean_squared_error: 0.2789
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - loss: 0.0367 - mean_absolute_error: 0.1577 - mean_absolute_percentage_error: 33.2824 - root_mean_squared_error: 0.1914 - val_loss: 0.0570 - val_mean_absolute_error: 0.1995 - val_mean_absolute_percentage_error: 13899095.0000 - val_root_mean_squared_error: 0.2388
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - loss: 0.0228 - mean_absolute_error: 0.1191 - mean_absolute_percentage_error: 29.5821 - root_mean_squared_error: 0.1511 - val_loss: 0.0532 - val_mean_absolute_error: 0.1901 - val_mean_absolute_percentage_error: 14698691.0000 - val_root_mean_squared_error: 0.2306
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step 

In [ ]:
y_pred = model_lstm_5.predict(x_test)

evaluation_lstm_5 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_5

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step


{'Mean Squared Error: ': np.float64(324.7721261971295),
 'Mean Absolute Error: ': np.float64(13.977943595574827),
 'Root Mean Absolute Error: ': 18.021435186941396,
 'Mean Absolute Percentage Error: ': np.float64(25.840499978600956)}

In [ ]:
loss, mae, mape, rmse = model_lstm_5.evaluate(x_test, y_test)
print("model lstm 5")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.0259 - mean_absolute_error: 0.1248 - mean_absolute_percentage_error: 32.8091 - root_mean_squared_error: 0.1609
model lstm 5
Mean Square Error: 0.025890633463859558
Mean Absolute Error: 0.12480305880308151
Mean Absolute Percentage Error: 32.80914306640625
Root Mean Squared Error: 0.160905659198761


## model 6 (LSTM=16)

In [ ]:
lr = 0.0003

model_lstm_6 = Sequential()
model_lstm_6.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_6.add(LSTM(16, activation='relu'))
model_lstm_6.add(Flatten())
model_lstm_6.add(Dense(7))
model_lstm_6.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_6.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                   │ (None, 16)             │         1,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_25 (Flatten)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,591 (6.21 KB)

 Trainable params: 1,591 (6.21 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_6.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.2960 - mean_absolute_error: 0.4932 - mean_absolute_percentage_error: 86.8988 - root_mean_squared_error: 0.5439 - val_loss: 0.2619 - val_mean_absolute_error: 0.4610 - val_mean_absolute_percentage_error: 3951509.7500 - val_root_mean_squared_error: 0.5117
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2179 - mean_absolute_error: 0.4181 - mean_absolute_percentage_error: 72.7865 - root_mean_squared_error: 0.4667 - val_loss: 0.1955 - val_mean_absolute_error: 0.3908 - val_mean_absolute_percentage_error: 6515204.0000 - val_root_mean_squared_error: 0.4421
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1440 - mean_absolute_error: 0.3179 - mean_absolute_percentage_error: 55.7881 - root_mean_squared_error: 0.3792 - val_loss: 0.1221 - val_mean_absolute_error: 0.2972 - val_mean_absolute_percentage_error: 10175105.0000 - val_root_mean_squared_error: 0.3495
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss:

In [ ]:
y_pred = model_lstm_6.predict(x_test)

evaluation_lstm_6 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_6

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step


{'Mean Squared Error: ': np.float64(340.19115233959553),
 'Mean Absolute Error: ': np.float64(14.633661114439672),
 'Root Mean Absolute Error: ': 18.444271531822437,
 'Mean Absolute Percentage Error: ': np.float64(22.943267627097597)}

In [ ]:
loss, mae, mape, rmse = model_lstm_6.evaluate(x_test, y_test)
print("model lstm 6")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0271 - mean_absolute_error: 0.1307 - mean_absolute_percentage_error: 28.1392 - root_mean_squared_error: 0.1647
model lstm 6
Mean Square Error: 0.027119828388094902
Mean Absolute Error: 0.13065767288208008
Mean Absolute Percentage Error: 28.139184951782227
Root Mean Squared Error: 0.16468098759651184


## model 7 (LSTM=16 16)

In [ ]:
lr = 0.0003

model_lstm_7 = Sequential()
model_lstm_7.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_7.add(LSTM(16, activation='relu', return_sequences=True))
model_lstm_7.add(LSTM(16, activation='relu'))
model_lstm_7.add(Flatten())
model_lstm_7.add(Dense(7))
model_lstm_7.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_7.summary()

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 7, 16)          │         1,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 16)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_26 (Flatten)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,703 (14.46 KB)

 Trainable params: 3,703 (14.46 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_7.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - loss: 0.3237 - mean_absolute_error: 0.5402 - mean_absolute_percentage_error: 95.6427 - root_mean_squared_error: 0.5688 - val_loss: 0.2711 - val_mean_absolute_error: 0.4761 - val_mean_absolute_percentage_error: 2272626.0000 - val_root_mean_squared_error: 0.5207
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2421 - mean_absolute_error: 0.4573 - mean_absolute_percentage_error: 79.4868 - root_mean_squared_error: 0.4919 - val_loss: 0.1790 - val_mean_absolute_error: 0.3733 - val_mean_absolute_percentage_error: 5443273.0000 - val_root_mean_squared_error: 0.4231
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1422 - mean_absolute_error: 0.3257 - mean_absolute_percentage_error: 56.6110 - root_mean_squared_error: 0.3763 - val_loss: 0.1017 - val_mean_absolute_error: 0.2681 - val_mean_absolute_percentage_error: 11802382.0000 - val_root_mean_squared_error: 0.3190
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss

In [ ]:
y_pred = model_lstm_7.predict(x_test)

evaluation_lstm_7 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_7

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step


{'Mean Squared Error: ': np.float64(388.7641676932649),
 'Mean Absolute Error: ': np.float64(16.76678328611413),
 'Root Mean Absolute Error: ': 19.717103430607267,
 'Mean Absolute Percentage Error: ': np.float64(27.62630369072231)}

In [ ]:
loss, mae, mape, rmse = model_lstm_7.evaluate(x_test, y_test)
print("model lstm 7")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0310 - mean_absolute_error: 0.1497 - mean_absolute_percentage_error: 34.2039 - root_mean_squared_error: 0.1760
model lstm 7
Mean Square Error: 0.030992036685347557
Mean Absolute Error: 0.14970341324806213
Mean Absolute Percentage Error: 34.20393753051758
Root Mean Squared Error: 0.17604555189609528


## model 8 (LSTM=16 16 16)

In [ ]:
lr = 0.0003

model_lstm_8 = Sequential()
model_lstm_8.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_8.add(LSTM(16, activation='relu', return_sequences=True))
model_lstm_8.add(LSTM(16, activation='relu', return_sequences=True))
model_lstm_8.add(LSTM(16, activation='relu'))
model_lstm_8.add(Flatten())
model_lstm_8.add(Dense(7))
model_lstm_8.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_8.summary()

Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 7, 16)          │         1,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 7, 16)          │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 16)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_27 (Flatten)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,815 (22.71 KB)

 Trainable params: 5,815 (22.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_8.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step - loss: 0.3344 - mean_absolute_error: 0.5502 - mean_absolute_percentage_error: 98.1905 - root_mean_squared_error: 0.5782 - val_loss: 0.3083 - val_mean_absolute_error: 0.5100 - val_mean_absolute_percentage_error: 1319317.1250 - val_root_mean_squared_error: 0.5552
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.2847 - mean_absolute_error: 0.5032 - mean_absolute_percentage_error: 88.4289 - root_mean_squared_error: 0.5335 - val_loss: 0.2358 - val_mean_absolute_error: 0.4423 - val_mean_absolute_percentage_error: 3849289.5000 - val_root_mean_squared_error: 0.4856
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1860 - mean_absolute_error: 0.3894 - mean_absolute_percentage_error: 67.2974 - root_mean_squared_error: 0.4304 - val_loss: 0.1011 - val_mean_absolute_error: 0.2710 - val_mean_absolute_percentage_error: 13073612.0000 - val_root_mean_squared_error: 0.3180
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss

In [ ]:
y_pred = model_lstm_8.predict(x_test)

evaluation_lstm_8 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_8

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step


{'Mean Squared Error: ': np.float64(316.5355930347165),
 'Mean Absolute Error: ': np.float64(14.23902614749208),
 'Root Mean Absolute Error: ': 17.79144718775616,
 'Mean Absolute Percentage Error: ': np.float64(25.42743128179052)}

In [ ]:
loss, mae, mape, rmse = model_lstm_8.evaluate(x_test, y_test)
print("model lstm 8")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0252 - mean_absolute_error: 0.1271 - mean_absolute_percentage_error: 32.1044 - root_mean_squared_error: 0.1589
model lstm 8
Mean Square Error: 0.025234024971723557
Mean Absolute Error: 0.12713415920734406
Mean Absolute Percentage Error: 32.10444641113281
Root Mean Squared Error: 0.1588522046804428


## model 9 (LSTM=32 16)

In [ ]:
lr = 0.0003

model_lstm_9 = Sequential()
model_lstm_9.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_9.add(LSTM(32, activation='relu', return_sequences=True))
model_lstm_9.add(LSTM(16, activation='relu'))
model_lstm_9.add(Flatten())
model_lstm_9.add(Dense(7))
model_lstm_9.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_9.summary()

Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_15 (LSTM)                  │ (None, 7, 32)          │         4,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_16 (LSTM)                  │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_28 (Flatten)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,247 (32.21 KB)

 Trainable params: 8,247 (32.21 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_9.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.3442 - mean_absolute_error: 0.5602 - mean_absolute_percentage_error: 98.8299 - root_mean_squared_error: 0.5866 - val_loss: 0.2991 - val_mean_absolute_error: 0.5023 - val_mean_absolute_percentage_error: 1605935.6250 - val_root_mean_squared_error: 0.5469
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.2632 - mean_absolute_error: 0.4825 - mean_absolute_percentage_error: 85.1442 - root_mean_squared_error: 0.5129 - val_loss: 0.2004 - val_mean_absolute_error: 0.4037 - val_mean_absolute_percentage_error: 5404966.5000 - val_root_mean_squared_error: 0.4476
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1353 - mean_absolute_error: 0.3098 - mean_absolute_percentage_error: 54.8920 - root_mean_squared_error: 0.3656 - val_loss: 0.0656 - val_mean_absolute_error: 0.2193 - val_mean_absolute_percentage_error: 14624310.0000 - val_root_mean_squared_error: 0.2562
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss

In [ ]:
y_pred = model_lstm_9.predict(x_test)

evaluation_lstm_9 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_9

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step


{'Mean Squared Error: ': np.float64(379.76451502489266),
 'Mean Absolute Error: ': np.float64(15.208665283358826),
 'Root Mean Absolute Error: ': 19.48754769140778,
 'Mean Absolute Percentage Error: ': np.float64(27.620088164244827)}

In [ ]:
loss, mae, mape, rmse = model_lstm_9.evaluate(x_test, y_test)
print("model lstm 9")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0303 - mean_absolute_error: 0.1358 - mean_absolute_percentage_error: 34.9018 - root_mean_squared_error: 0.1740
model lstm 9
Mean Square Error: 0.030274594202637672
Mean Absolute Error: 0.13579164445400238
Mean Absolute Percentage Error: 34.901817321777344
Root Mean Squared Error: 0.1739959567785263


## model 10 (LSTM=128. 64)

In [ ]:
lr = 0.0003

model_lstm_10 = Sequential()
model_lstm_10.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_10.add(LSTM(128, activation='relu', return_sequences=True))
model_lstm_10.add(LSTM(64, activation='relu'))
model_lstm_10.add(Flatten())
model_lstm_10.add(Dense(7))
model_lstm_10.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_10.summary()

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_17 (LSTM)                  │ (None, 7, 128)         │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_18 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_29 (Flatten)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,983 (464.78 KB)

 Trainable params: 118,983 (464.78 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_10.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.2739 - mean_absolute_error: 0.4865 - mean_absolute_percentage_error: 86.6694 - root_mean_squared_error: 0.5217 - val_loss: 0.0883 - val_mean_absolute_error: 0.2406 - val_mean_absolute_percentage_error: 15276545.0000 - val_root_mean_squared_error: 0.2971
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0486 - mean_absolute_error: 0.1748 - mean_absolute_percentage_error: 41.9468 - root_mean_squared_error: 0.2189 - val_loss: 0.0564 - val_mean_absolute_error: 0.2023 - val_mean_absolute_percentage_error: 13640981.0000 - val_root_mean_squared_error: 0.2375
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0255 - mean_absolute_error: 0.1249 - mean_absolute_percentage_error: 29.4566 - root_mean_squared_error: 0.1596 - val_loss: 0.0597 - val_mean_absolute_error: 0.2087 - val_mean_absolute_percentage_error: 12851906.0000 - val_root_mean_squared_error: 0.2444
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - lo

In [ ]:
y_pred = model_lstm_10.predict(x_test)

evaluation_lstm_10 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_10

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step


{'Mean Squared Error: ': np.float64(407.9362081373832),
 'Mean Absolute Error: ': np.float64(15.009092642336475),
 'Root Mean Absolute Error: ': 20.19743073109506,
 'Mean Absolute Percentage Error: ': np.float64(28.816592599477563)}

In [ ]:
loss, mae, mape, rmse = model_lstm_10.evaluate(x_test, y_test)
print("model lstm 10")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0325 - mean_absolute_error: 0.1340 - mean_absolute_percentage_error: 36.7682 - root_mean_squared_error: 0.1803
model lstm 10
Mean Square Error: 0.03252042084932327
Mean Absolute Error: 0.13400974869728088
Mean Absolute Percentage Error: 36.76820755004883
Root Mean Squared Error: 0.1803341954946518


## model 11 (LSTM=128 128)

In [ ]:
lr = 0.0003

model_lstm_11 = Sequential()
model_lstm_11.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_11.add(LSTM(128, activation='relu', return_sequences=True))
model_lstm_11.add(LSTM(128, activation='relu'))
model_lstm_11.add(Flatten())
model_lstm_11.add(Dense(7))
model_lstm_11.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_11.summary()

Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_19 (LSTM)                  │ (None, 7, 128)         │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_30 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,607 (787.53 KB)

 Trainable params: 201,607 (787.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_11.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - loss: 0.2977 - mean_absolute_error: 0.5131 - mean_absolute_percentage_error: 90.9418 - root_mean_squared_error: 0.5447 - val_loss: 0.1005 - val_mean_absolute_error: 0.2653 - val_mean_absolute_percentage_error: 12748996.0000 - val_root_mean_squared_error: 0.3170
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0606 - mean_absolute_error: 0.1933 - mean_absolute_percentage_error: 40.6529 - root_mean_squared_error: 0.2444 - val_loss: 0.0550 - val_mean_absolute_error: 0.1961 - val_mean_absolute_percentage_error: 14540527.0000 - val_root_mean_squared_error: 0.2344
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0268 - mean_absolute_error: 0.1291 - mean_absolute_percentage_error: 32.4556 - root_mean_squared_error: 0.1636 - val_loss: 0.0511 - val_mean_absolute_error: 0.1870 - val_mean_absolute_percentage_error: 15157250.0000 - val_root_mean_squared_error: 0.2260
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - lo

In [ ]:
y_pred = model_lstm_11.predict(x_test)

evaluation_lstm_11 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_11

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step


{'Mean Squared Error: ': np.float64(301.9662657849719),
 'Mean Absolute Error: ': np.float64(13.733981658001335),
 'Root Mean Absolute Error: ': 17.377176576906038,
 'Mean Absolute Percentage Error: ': np.float64(24.452698227640965)}

In [ ]:
loss, mae, mape, rmse = model_lstm_11.evaluate(x_test, y_test)
print("model lstm 11")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0241 - mean_absolute_error: 0.1226 - mean_absolute_percentage_error: 30.8456 - root_mean_squared_error: 0.1552
model lstm 11
Mean Square Error: 0.024072567000985146
Mean Absolute Error: 0.12262482941150665
Mean Absolute Percentage Error: 30.8455810546875
Root Mean Squared Error: 0.15515336394309998


## model 12 (LSTM=64 128)

In [ ]:
lr = 0.0003

model_lstm_12 = Sequential()
model_lstm_12.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_12.add(LSTM(64, activation='relu', return_sequences=True))
model_lstm_12.add(LSTM(128, activation='relu'))
model_lstm_12.add(Flatten())
model_lstm_12.add(Dense(7))
model_lstm_12.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_12.summary()

Model: "sequential_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_21 (LSTM)                  │ (None, 7, 64)          │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_31 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,895 (460.53 KB)

 Trainable params: 117,895 (460.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_12.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - loss: 0.2989 - mean_absolute_error: 0.5147 - mean_absolute_percentage_error: 90.1031 - root_mean_squared_error: 0.5455 - val_loss: 0.0687 - val_mean_absolute_error: 0.2223 - val_mean_absolute_percentage_error: 12746359.0000 - val_root_mean_squared_error: 0.2620
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0413 - mean_absolute_error: 0.1613 - mean_absolute_percentage_error: 38.1608 - root_mean_squared_error: 0.2030 - val_loss: 0.0637 - val_mean_absolute_error: 0.2106 - val_mean_absolute_percentage_error: 12914355.0000 - val_root_mean_squared_error: 0.2524
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0257 - mean_absolute_error: 0.1275 - mean_absolute_percentage_error: 30.5147 - root_mean_squared_error: 0.1603 - val_loss: 0.0521 - val_mean_absolute_error: 0.1883 - val_mean_absolute_percentage_error: 15141922.0000 - val_root_mean_squared_error: 0.2282
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - lo

In [ ]:
y_pred = model_lstm_12.predict(x_test)

evaluation_lstm_12 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_12

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step


{'Mean Squared Error: ': np.float64(271.98071853461926),
 'Mean Absolute Error: ': np.float64(13.213419174661443),
 'Root Mean Absolute Error: ': 16.491837936828606,
 'Mean Absolute Percentage Error: ': np.float64(23.168551997855385)}

In [ ]:
loss, mae, mape, rmse = model_lstm_12.evaluate(x_test, y_test)
print("model lstm 12")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0217 - mean_absolute_error: 0.1180 - mean_absolute_percentage_error: 29.2195 - root_mean_squared_error: 0.1472
model lstm 12
Mean Square Error: 0.0216821376234293
Mean Absolute Error: 0.11797695606946945
Mean Absolute Percentage Error: 29.219472885131836
Root Mean Squared Error: 0.1472485512495041


## model 13 (LSTM=128)

In [ ]:
lr = 0.0003

model_lstm_13 = Sequential()
model_lstm_13.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_13.add(LSTM(128, activation='relu', return_sequences=True))
model_lstm_13.add(Flatten())
model_lstm_13.add(Dense(7))
model_lstm_13.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_13.summary()

Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_23 (LSTM)                  │ (None, 7, 128)         │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_32 (Flatten)            │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 7)              │         6,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,399 (294.53 KB)

 Trainable params: 75,399 (294.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_13.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.2459 - mean_absolute_error: 0.4510 - mean_absolute_percentage_error: 78.1502 - root_mean_squared_error: 0.4912 - val_loss: 0.0643 - val_mean_absolute_error: 0.1997 - val_mean_absolute_percentage_error: 19353830.0000 - val_root_mean_squared_error: 0.2535
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0392 - mean_absolute_error: 0.1564 - mean_absolute_percentage_error: 39.2980 - root_mean_squared_error: 0.1968 - val_loss: 0.0604 - val_mean_absolute_error: 0.2026 - val_mean_absolute_percentage_error: 13659873.0000 - val_root_mean_squared_error: 0.2457
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0243 - mean_absolute_error: 0.1214 - mean_absolute_percentage_error: 30.0521 - root_mean_squared_error: 0.1559 - val_loss: 0.0520 - val_mean_absolute_error: 0.1885 - val_mean_absolute_percentage_error: 15341995.0000 - val_root_mean_squared_error: 0.2280
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - lo

In [ ]:
y_pred = model_lstm_13.predict(x_test)

evaluation_lstm_13 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_13

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step


{'Mean Squared Error: ': np.float64(321.657629135667),
 'Mean Absolute Error: ': np.float64(13.947566986083984),
 'Root Mean Absolute Error: ': 17.934816116583605,
 'Mean Absolute Percentage Error: ': np.float64(25.885039068633287)}

In [ ]:
loss, mae, mape, rmse = model_lstm_13.evaluate(x_test, y_test)
print("model lstm 13")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0256 - mean_absolute_error: 0.1245 - mean_absolute_percentage_error: 32.9022 - root_mean_squared_error: 0.1601
model lstm 13
Mean Square Error: 0.02564234845340252
Mean Absolute Error: 0.12453185021877289
Mean Absolute Percentage Error: 32.902217864990234
Root Mean Squared Error: 0.1601322889328003


## model 14 (LSTM=64)

In [ ]:
lr = 0.0003

model_lstm_14 = Sequential()
model_lstm_14.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_14.add(LSTM(64, activation='relu', return_sequences=True))
model_lstm_14.add(Flatten())
model_lstm_14.add(Dense(7))
model_lstm_14.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_14.summary()

Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_24 (LSTM)                  │ (None, 7, 64)          │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_33 (Flatten)            │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,319 (83.28 KB)

 Trainable params: 21,319 (83.28 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_14.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.2969 - mean_absolute_error: 0.5119 - mean_absolute_percentage_error: 90.8411 - root_mean_squared_error: 0.5441 - val_loss: 0.1269 - val_mean_absolute_error: 0.3155 - val_mean_absolute_percentage_error: 7749652.5000 - val_root_mean_squared_error: 0.3563
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0634 - mean_absolute_error: 0.2059 - mean_absolute_percentage_error: 40.7759 - root_mean_squared_error: 0.2494 - val_loss: 0.0541 - val_mean_absolute_error: 0.1903 - val_mean_absolute_percentage_error: 17057244.0000 - val_root_mean_squared_error: 0.2326
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0251 - mean_absolute_error: 0.1243 - mean_absolute_percentage_error: 31.6883 - root_mean_squared_error: 0.1583 - val_loss: 0.0541 - val_mean_absolute_error: 0.1984 - val_mean_absolute_percentage_error: 13986243.0000 - val_root_mean_squared_error: 0.2327
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - los

In [ ]:
y_pred = model_lstm_14.predict(x_test)

evaluation_lstm_14 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_14

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step


{'Mean Squared Error: ': np.float64(336.93133774970903),
 'Mean Absolute Error: ': np.float64(15.04144080804319),
 'Root Mean Absolute Error: ': 18.35568951986574,
 'Mean Absolute Percentage Error: ': np.float64(25.902786676580003)}

In [ ]:
loss, mae, mape, rmse = model_lstm_14.evaluate(x_test, y_test)
print("model lstm 14")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0269 - mean_absolute_error: 0.1343 - mean_absolute_percentage_error: 32.4453 - root_mean_squared_error: 0.1639
model lstm 14
Mean Square Error: 0.026859959587454796
Mean Absolute Error: 0.13429856300354004
Mean Absolute Percentage Error: 32.44533157348633
Root Mean Squared Error: 0.163890078663826


## model 15 (LSTM=256)

In [ ]:
lr = 0.0003

model_lstm_15 = Sequential()
model_lstm_15.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_15.add(LSTM(256, activation='relu', return_sequences=True))
model_lstm_15.add(Flatten())
model_lstm_15.add(Dense(7))
model_lstm_15.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_15.summary()

Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_25 (LSTM)                  │ (None, 7, 256)         │       269,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_34 (Flatten)            │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 7)              │        12,551 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 281,863 (1.08 MB)

 Trainable params: 281,863 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_15.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.1996 - mean_absolute_error: 0.3972 - mean_absolute_percentage_error: 73.8006 - root_mean_squared_error: 0.4420 - val_loss: 0.0507 - val_mean_absolute_error: 0.1862 - val_mean_absolute_percentage_error: 16600281.0000 - val_root_mean_squared_error: 0.2251
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0278 - mean_absolute_error: 0.1341 - mean_absolute_percentage_error: 31.2032 - root_mean_squared_error: 0.1668 - val_loss: 0.0509 - val_mean_absolute_error: 0.1864 - val_mean_absolute_percentage_error: 16342879.0000 - val_root_mean_squared_error: 0.2256
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0220 - mean_absolute_error: 0.1136 - mean_absolute_percentage_error: 30.5618 - root_mean_squared_error: 0.1483 - val_loss: 0.0552 - val_mean_absolute_error: 0.1944 - val_mean_absolute_percentage_error: 14703558.0000 - val_root_mean_squared_error: 0.2350
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - lo

In [ ]:
y_pred = model_lstm_15.predict(x_test)

evaluation_lstm_15 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_15

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step


{'Mean Squared Error: ': np.float64(321.99200746362897),
 'Mean Absolute Error: ': np.float64(13.920665118159082),
 'Root Mean Absolute Error: ': 17.944135740225246,
 'Mean Absolute Percentage Error: ': np.float64(25.754616371782816)}

In [ ]:
loss, mae, mape, rmse = model_lstm_15.evaluate(x_test, y_test)
print("model lstm 15")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0257 - mean_absolute_error: 0.1243 - mean_absolute_percentage_error: 32.7161 - root_mean_squared_error: 0.1602
model lstm 15
Mean Square Error: 0.025669004768133163
Mean Absolute Error: 0.12429165095090866
Mean Absolute Percentage Error: 32.716068267822266
Root Mean Squared Error: 0.16021549701690674


## model 16 (LSTM=512)

In [ ]:
lr = 0.0003

model_lstm_16 = Sequential()
model_lstm_16.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_16.add(LSTM(512, activation='relu', return_sequences=True))
model_lstm_16.add(Flatten())
model_lstm_16.add(Dense(7))
model_lstm_16.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_16.summary()

Model: "sequential_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_26 (LSTM)                  │ (None, 7, 512)         │     1,062,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_35 (Flatten)            │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 7)              │        25,095 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,088,007 (4.15 MB)

 Trainable params: 1,088,007 (4.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_16.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - loss: 0.1844 - mean_absolute_error: 0.3656 - mean_absolute_percentage_error: 66.6131 - root_mean_squared_error: 0.4196 - val_loss: 0.0723 - val_mean_absolute_error: 0.2292 - val_mean_absolute_percentage_error: 11407193.0000 - val_root_mean_squared_error: 0.2690
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - loss: 0.0320 - mean_absolute_error: 0.1440 - mean_absolute_percentage_error: 31.0393 - root_mean_squared_error: 0.1786 - val_loss: 0.0573 - val_mean_absolute_error: 0.1999 - val_mean_absolute_percentage_error: 13480289.0000 - val_root_mean_squared_error: 0.2394
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - loss: 0.0255 - mean_absolute_error: 0.1282 - mean_absolute_percentage_error: 29.7092 - root_mean_squared_error: 0.1596 - val_loss: 0.0523 - val_mean_absolute_error: 0.1897 - val_mean_absolute_percentage_error: 14647306.0000 - val_root_mean_squared_error: 0.2287
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 128ms/step - 

In [ ]:
y_pred = model_lstm_16.predict(x_test)

evaluation_lstm_16 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_16

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step


{'Mean Squared Error: ': np.float64(466.82663771646855),
 'Mean Absolute Error: ': np.float64(16.250607957645336),
 'Root Mean Absolute Error: ': 21.606171287770273,
 'Mean Absolute Percentage Error: ': np.float64(31.161397248350614)}

In [ ]:
loss, mae, mape, rmse = model_lstm_16.evaluate(x_test, y_test)
print("model lstm 16")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0372 - mean_absolute_error: 0.1451 - mean_absolute_percentage_error: 39.7344 - root_mean_squared_error: 0.1929
model lstm 16
Mean Square Error: 0.037215132266283035
Mean Absolute Error: 0.14509472250938416
Mean Absolute Percentage Error: 39.734397888183594
Root Mean Squared Error: 0.1929122358560562


## model 17 (LSTM=512 128)

In [ ]:
lr = 0.0003

model_lstm_17 = Sequential()
model_lstm_17.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_17.add(LSTM(512, activation='relu', return_sequences=True))
model_lstm_17.add(LSTM(128, activation='relu'))
model_lstm_17.add(Flatten())
model_lstm_17.add(Dense(7))
model_lstm_17.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_17.summary()

Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_27 (LSTM)                  │ (None, 7, 512)         │     1,062,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_28 (LSTM)                  │ (None, 128)            │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_36 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,392,007 (5.31 MB)

 Trainable params: 1,392,007 (5.31 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_17.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 208ms/step - loss: 0.2168 - mean_absolute_error: 0.4135 - mean_absolute_percentage_error: 75.9879 - root_mean_squared_error: 0.4603 - val_loss: 0.0759 - val_mean_absolute_error: 0.2351 - val_mean_absolute_percentage_error: 11297524.0000 - val_root_mean_squared_error: 0.2756
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - loss: 0.0337 - mean_absolute_error: 0.1503 - mean_absolute_percentage_error: 31.8493 - root_mean_squared_error: 0.1832 - val_loss: 0.0557 - val_mean_absolute_error: 0.1959 - val_mean_absolute_percentage_error: 14064612.0000 - val_root_mean_squared_error: 0.2361
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - loss: 0.0225 - mean_absolute_error: 0.1184 - mean_absolute_percentage_error: 29.3283 - root_mean_squared_error: 0.1499 - val_loss: 0.0513 - val_mean_absolute_error: 0.1861 - val_mean_absolute_percentage_error: 15290746.0000 - val_root_mean_squared_error: 0.2264
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step 

In [ ]:
y_pred = model_lstm_17.predict(x_test)

evaluation_lstm_17 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_17

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step


{'Mean Squared Error: ': np.float64(353.5499715529387),
 'Mean Absolute Error: ': np.float64(14.611112711380937),
 'Root Mean Absolute Error: ': 18.802924547871235,
 'Mean Absolute Percentage Error: ': np.float64(27.375874587516353)}

In [ ]:
loss, mae, mape, rmse = model_lstm_17.evaluate(x_test, y_test)
print("model lstm 17")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0282 - mean_absolute_error: 0.1305 - mean_absolute_percentage_error: 34.8169 - root_mean_squared_error: 0.1679
model lstm 17
Mean Square Error: 0.028184786438941956
Mean Absolute Error: 0.13045634329319
Mean Absolute Percentage Error: 34.816864013671875
Root Mean Squared Error: 0.1678832471370697


## model 18 (LSTM=16 16 16)

In [ ]:
from keras.layers import LeakyReLU, ELU
lr = 0.0003

model_lstm_18 = Sequential()
model_lstm_18.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_18.add(LSTM(16, return_sequences=True))
model_lstm_18.add(LeakyReLU(alpha=0.1))  # or ELU(alpha=1.0)

model_lstm_18.add(LSTM(16, return_sequences=True))
model_lstm_18.add(LeakyReLU(alpha=0.1))  # or ELU(alpha=1.0)

model_lstm_18.add(LSTM(16))
model_lstm_18.add(Flatten())
model_lstm_18.add(Dense(7))
model_lstm_18.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_18.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_29 (LSTM)                  │ (None, 7, 16)          │         1,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_30 (LSTM)                  │ (None, 7, 16)          │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_31 (LSTM)                  │ (None, 16)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_37 (Flatten)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,815 (22.71 KB)

 Trainable params: 5,815 (22.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_18.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - loss: 0.3134 - mean_absolute_error: 0.5290 - mean_absolute_percentage_error: 95.5103 - root_mean_squared_error: 0.5598 - val_loss: 0.2692 - val_mean_absolute_error: 0.4734 - val_mean_absolute_percentage_error: 3280095.2500 - val_root_mean_squared_error: 0.5188
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.2264 - mean_absolute_error: 0.4313 - mean_absolute_percentage_error: 74.8317 - root_mean_squared_error: 0.4754 - val_loss: 0.1452 - val_mean_absolute_error: 0.3049 - val_mean_absolute_percentage_error: 10870090.0000 - val_root_mean_squared_error: 0.3810
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1037 - mean_absolute_error: 0.2383 - mean_absolute_percentage_error: 49.7401 - root_mean_squared_error: 0.3217 - val_loss: 0.0770 - val_mean_absolute_error: 0.2183 - val_mean_absolute_percentage_error: 15128644.0000 - val_root_mean_squared_error: 0.2774
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - los

In [ ]:
y_pred = model_lstm_18.predict(x_test)

evaluation_lstm_18 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_18

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step


{'Mean Squared Error: ': np.float64(430.8832270191488),
 'Mean Absolute Error: ': np.float64(17.273792111143774),
 'Root Mean Absolute Error: ': 20.757726923224247,
 'Mean Absolute Percentage Error: ': np.float64(29.58385066397657)}

In [ ]:
loss, mae, mape, rmse = model_lstm_18.evaluate(x_test, y_test)
print("model lstm 18")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0343 - mean_absolute_error: 0.1542 - mean_absolute_percentage_error: 36.9139 - root_mean_squared_error: 0.1853
model lstm 18
Mean Square Error: 0.034349747002124786
Mean Absolute Error: 0.1542302817106247
Mean Absolute Percentage Error: 36.913917541503906
Root Mean Squared Error: 0.18533684313297272


## model 19 (LSTM=16 16 16)

In [ ]:
lr = 0.0003

model_lstm_19 = Sequential()
model_lstm_19.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_19.add(LSTM(16, return_sequences=True))
model_lstm_19.add(ELU(alpha=1.0))  # or ELU(alpha=1.0)

model_lstm_19.add(LSTM(16, return_sequences=True))
model_lstm_19.add(ELU(alpha=1.0))  # or ELU(alpha=1.0)

model_lstm_19.add(LSTM(16))
model_lstm_19.add(Flatten())
model_lstm_19.add(Dense(7))
model_lstm_19.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_19.summary()

Model: "sequential_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_32 (LSTM)                  │ (None, 7, 16)          │         1,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu (ELU)                       │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_33 (LSTM)                  │ (None, 7, 16)          │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_1 (ELU)                     │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_34 (LSTM)                  │ (None, 16)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_38 (Flatten)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,815 (22.71 KB)

 Trainable params: 5,815 (22.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_19.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - loss: 0.3249 - mean_absolute_error: 0.5426 - mean_absolute_percentage_error: 95.1254 - root_mean_squared_error: 0.5697 - val_loss: 0.2460 - val_mean_absolute_error: 0.4515 - val_mean_absolute_percentage_error: 3413283.7500 - val_root_mean_squared_error: 0.4960
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.1961 - mean_absolute_error: 0.4003 - mean_absolute_percentage_error: 68.6410 - root_mean_squared_error: 0.4418 - val_loss: 0.1078 - val_mean_absolute_error: 0.2758 - val_mean_absolute_percentage_error: 12105600.0000 - val_root_mean_squared_error: 0.3284
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0724 - mean_absolute_error: 0.2167 - mean_absolute_percentage_error: 45.9522 - root_mean_squared_error: 0.2686 - val_loss: 0.0594 - val_mean_absolute_error: 0.1964 - val_mean_absolute_percentage_error: 15927159.0000 - val_root_mean_squared_error: 0.2438
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - los

In [ ]:
y_pred = model_lstm_19.predict(x_test)

evaluation_lstm_19 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_19

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step


{'Mean Squared Error: ': np.float64(401.08763956785367),
 'Mean Absolute Error: ': np.float64(16.457443937963372),
 'Root Mean Absolute Error: ': 20.02717253053595,
 'Mean Absolute Percentage Error: ': np.float64(27.461652576503997)}

In [ ]:
loss, mae, mape, rmse = model_lstm_19.evaluate(x_test, y_test)
print("model lstm 19")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0320 - mean_absolute_error: 0.1469 - mean_absolute_percentage_error: 34.1232 - root_mean_squared_error: 0.1788
model lstm 19
Mean Square Error: 0.03197445720434189
Mean Absolute Error: 0.14694145321846008
Mean Absolute Percentage Error: 34.12320327758789
Root Mean Squared Error: 0.17881402373313904


## model 20 (LSTM=128 200)

In [ ]:
lr = 0.0003

model_lstm_20 = Sequential()
model_lstm_20.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_20.add(LSTM(128, activation='relu', return_sequences=True))
model_lstm_20.add(LSTM(200, activation='relu'))
model_lstm_20.add(Flatten())
model_lstm_20.add(Dense(7))
model_lstm_20.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_20.summary()

Model: "sequential_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_35 (LSTM)                  │ (None, 7, 128)         │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_36 (LSTM)                  │ (None, 200)            │       263,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_39 (Flatten)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 7)              │         1,407 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 333,727 (1.27 MB)

 Trainable params: 333,727 (1.27 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_20.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - loss: 0.2799 - mean_absolute_error: 0.4912 - mean_absolute_percentage_error: 87.8308 - root_mean_squared_error: 0.5274 - val_loss: 0.0776 - val_mean_absolute_error: 0.2209 - val_mean_absolute_percentage_error: 18553908.0000 - val_root_mean_squared_error: 0.2785
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0435 - mean_absolute_error: 0.1633 - mean_absolute_percentage_error: 39.3433 - root_mean_squared_error: 0.2074 - val_loss: 0.0541 - val_mean_absolute_error: 0.1923 - val_mean_absolute_percentage_error: 15039214.0000 - val_root_mean_squared_error: 0.2325
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0252 - mean_absolute_error: 0.1239 - mean_absolute_percentage_error: 31.6077 - root_mean_squared_error: 0.1587 - val_loss: 0.0537 - val_mean_absolute_error: 0.1917 - val_mean_absolute_percentage_error: 14362261.0000 - val_root_mean_squared_error: 0.2318
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - lo

In [ ]:
y_pred = model_lstm_20.predict(x_test)

evaluation_lstm_20 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_20

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step


{'Mean Squared Error: ': np.float64(505.48421103464335),
 'Mean Absolute Error: ': np.float64(16.546811006507095),
 'Root Mean Absolute Error: ': 22.48297602708866,
 'Mean Absolute Percentage Error: ': np.float64(32.24402781983183)}

In [ ]:
loss, mae, mape, rmse = model_lstm_20.evaluate(x_test, y_test)
print("model lstm 20")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0403 - mean_absolute_error: 0.1477 - mean_absolute_percentage_error: 41.2078 - root_mean_squared_error: 0.2007
model lstm 20
Mean Square Error: 0.04029688611626625
Mean Absolute Error: 0.14773938059806824
Mean Absolute Percentage Error: 41.207828521728516
Root Mean Squared Error: 0.20074084401130676


## model 21 (LSTM=128 256)

In [ ]:
lr = 0.0003

model_lstm_21 = Sequential()
model_lstm_21.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_21.add(LSTM(128, activation='relu', return_sequences=True))
model_lstm_21.add(LSTM(256, activation='relu'))
model_lstm_21.add(Flatten())
model_lstm_21.add(Dense(7))
model_lstm_21.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_21.summary()

Model: "sequential_40"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_37 (LSTM)                  │ (None, 7, 128)         │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_38 (LSTM)                  │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_40 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 465,159 (1.77 MB)

 Trainable params: 465,159 (1.77 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_21.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - loss: 0.2673 - mean_absolute_error: 0.4780 - mean_absolute_percentage_error: 85.4492 - root_mean_squared_error: 0.5147 - val_loss: 0.0749 - val_mean_absolute_error: 0.2147 - val_mean_absolute_percentage_error: 19452040.0000 - val_root_mean_squared_error: 0.2736
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0386 - mean_absolute_error: 0.1569 - mean_absolute_percentage_error: 35.2619 - root_mean_squared_error: 0.1958 - val_loss: 0.0529 - val_mean_absolute_error: 0.1891 - val_mean_absolute_percentage_error: 16721612.0000 - val_root_mean_squared_error: 0.2299
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0236 - mean_absolute_error: 0.1205 - mean_absolute_percentage_error: 31.6352 - root_mean_squared_error: 0.1538 - val_loss: 0.0521 - val_mean_absolute_error: 0.1889 - val_mean_absolute_percentage_error: 15193671.0000 - val_root_mean_squared_error: 0.2283
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - lo

In [ ]:
y_pred = model_lstm_21.predict(x_test)

evaluation_lstm_21 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_21

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step


{'Mean Squared Error: ': np.float64(295.41889165762313),
 'Mean Absolute Error: ': np.float64(13.529825093794841),
 'Root Mean Absolute Error: ': 17.187754119070448,
 'Mean Absolute Percentage Error: ': np.float64(24.716833615875313)}

In [ ]:
loss, mae, mape, rmse = model_lstm_21.evaluate(x_test, y_test)
print("model lstm 21")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0236 - mean_absolute_error: 0.1208 - mean_absolute_percentage_error: 31.3613 - root_mean_squared_error: 0.1535
model lstm 21
Mean Square Error: 0.02355061285197735
Mean Absolute Error: 0.12080200761556625
Mean Absolute Percentage Error: 31.361284255981445
Root Mean Squared Error: 0.15346208214759827


## model 22 (LSTM=128 512)

In [ ]:
lr = 0.0003

model_lstm_22 = Sequential()
model_lstm_22.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_22.add(LSTM(128, activation='relu', return_sequences=True))
model_lstm_22.add(LSTM(512, activation='relu'))
model_lstm_22.add(Flatten())
model_lstm_22.add(Dense(7))
model_lstm_22.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_22.summary()

Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_39 (LSTM)                  │ (None, 7, 128)         │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_40 (LSTM)                  │ (None, 512)            │     1,312,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_41 (Flatten)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,385,479 (5.29 MB)

 Trainable params: 1,385,479 (5.29 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_22.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 208ms/step - loss: 0.2326 - mean_absolute_error: 0.4304 - mean_absolute_percentage_error: 78.4560 - root_mean_squared_error: 0.4770 - val_loss: 0.0736 - val_mean_absolute_error: 0.2304 - val_mean_absolute_percentage_error: 12236343.0000 - val_root_mean_squared_error: 0.2713
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - loss: 0.0361 - mean_absolute_error: 0.1569 - mean_absolute_percentage_error: 33.5979 - root_mean_squared_error: 0.1897 - val_loss: 0.0516 - val_mean_absolute_error: 0.1836 - val_mean_absolute_percentage_error: 15497199.0000 - val_root_mean_squared_error: 0.2272
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - loss: 0.0233 - mean_absolute_error: 0.1218 - mean_absolute_percentage_error: 32.5490 - root_mean_squared_error: 0.1526 - val_loss: 0.0506 - val_mean_absolute_error: 0.1835 - val_mean_absolute_percentage_error: 15956980.0000 - val_root_mean_squared_error: 0.2250
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step 

In [ ]:
y_pred = model_lstm_22.predict(x_test)

evaluation_lstm_22 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_22

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step


{'Mean Squared Error: ': np.float64(371.5288416056039),
 'Mean Absolute Error: ': np.float64(14.506794637563276),
 'Root Mean Absolute Error: ': 19.275083439653482,
 'Mean Absolute Percentage Error: ': np.float64(27.581209227944818)}

In [ ]:
loss, mae, mape, rmse = model_lstm_22.evaluate(x_test, y_test)
print("model lstm 22")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0296 - mean_absolute_error: 0.1295 - mean_absolute_percentage_error: 35.1706 - root_mean_squared_error: 0.1721
model lstm 22
Mean Square Error: 0.029618050903081894
Mean Absolute Error: 0.12952494621276855
Mean Absolute Percentage Error: 35.17060089111328
Root Mean Squared Error: 0.17209894955158234


## model 23 (LSTM=500 250)

In [ ]:
lr = 0.0003

model_lstm_23 = Sequential()
model_lstm_23.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_23.add(LSTM(500, activation='relu', return_sequences=True))
model_lstm_23.add(LSTM(250, activation='relu'))
model_lstm_23.add(Flatten())
model_lstm_23.add(Dense(7))
model_lstm_23.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_23.summary()

Model: "sequential_42"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_41 (LSTM)                  │ (None, 7, 500)         │     1,014,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_42 (LSTM)                  │ (None, 250)            │       751,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_42 (Flatten)            │ (None, 250)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 7)              │         1,757 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,766,757 (6.74 MB)

 Trainable params: 1,766,757 (6.74 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_23.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 157ms/step - loss: 0.2081 - mean_absolute_error: 0.3981 - mean_absolute_percentage_error: 72.8778 - root_mean_squared_error: 0.4491 - val_loss: 0.0800 - val_mean_absolute_error: 0.2425 - val_mean_absolute_percentage_error: 11103959.0000 - val_root_mean_squared_error: 0.2829
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 156ms/step - loss: 0.0351 - mean_absolute_error: 0.1524 - mean_absolute_percentage_error: 33.5648 - root_mean_squared_error: 0.1868 - val_loss: 0.0589 - val_mean_absolute_error: 0.2013 - val_mean_absolute_percentage_error: 13586617.0000 - val_root_mean_squared_error: 0.2427
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - loss: 0.0236 - mean_absolute_error: 0.1222 - mean_absolute_percentage_error: 29.8501 - root_mean_squared_error: 0.1536 - val_loss: 0.0535 - val_mean_absolute_error: 0.1909 - val_mean_absolute_percentage_error: 14704398.0000 - val_root_mean_squared_error: 0.2312
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step 

In [ ]:
y_pred = model_lstm_23.predict(x_test)

evaluation_lstm_23 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_23

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step


{'Mean Squared Error: ': np.float64(419.47515755431897),
 'Mean Absolute Error: ': np.float64(15.502510946624133),
 'Root Mean Absolute Error: ': 20.481092684579085,
 'Mean Absolute Percentage Error: ': np.float64(29.37209014553771)}

In [ ]:
loss, mae, mape, rmse = model_lstm_23.evaluate(x_test, y_test)
print("model lstm 23")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0334 - mean_absolute_error: 0.1384 - mean_absolute_percentage_error: 37.4073 - root_mean_squared_error: 0.1829
model lstm 23
Mean Square Error: 0.033440299332141876
Mean Absolute Error: 0.13841526210308075
Mean Absolute Percentage Error: 37.40731430053711
Root Mean Squared Error: 0.1828668862581253


## model 24 (LSTM=512 256)

In [ ]:
lr = 0.0003

model_lstm_24 = Sequential()
model_lstm_24.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_24.add(LSTM(512, activation='relu', return_sequences=True))
model_lstm_24.add(LSTM(256, activation='relu'))
model_lstm_24.add(Flatten())
model_lstm_24.add(Dense(7))
model_lstm_24.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_24.summary()

Model: "sequential_43"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_43 (LSTM)                  │ (None, 7, 512)         │     1,062,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_44 (LSTM)                  │ (None, 256)            │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_43 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,852,167 (7.07 MB)

 Trainable params: 1,852,167 (7.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_24.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 181ms/step - loss: 0.1976 - mean_absolute_error: 0.3872 - mean_absolute_percentage_error: 70.4908 - root_mean_squared_error: 0.4377 - val_loss: 0.0745 - val_mean_absolute_error: 0.2303 - val_mean_absolute_percentage_error: 11899053.0000 - val_root_mean_squared_error: 0.2729
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 158ms/step - loss: 0.0326 - mean_absolute_error: 0.1451 - mean_absolute_percentage_error: 31.6840 - root_mean_squared_error: 0.1803 - val_loss: 0.0576 - val_mean_absolute_error: 0.2007 - val_mean_absolute_percentage_error: 14319693.0000 - val_root_mean_squared_error: 0.2401
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 184ms/step - loss: 0.0240 - mean_absolute_error: 0.1226 - mean_absolute_percentage_error: 29.8976 - root_mean_squared_error: 0.1548 - val_loss: 0.0534 - val_mean_absolute_error: 0.1884 - val_mean_absolute_percentage_error: 14981454.0000 - val_root_mean_squared_error: 0.2310
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step 

In [ ]:
y_pred = model_lstm_24.predict(x_test)

evaluation_lstm_24 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_24

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step


{'Mean Squared Error: ': np.float64(298.2663526506554),
 'Mean Absolute Error: ': np.float64(13.852951497447734),
 'Root Mean Absolute Error: ': 17.270389475939893,
 'Mean Absolute Percentage Error: ': np.float64(24.072552732877288)}

In [ ]:
loss, mae, mape, rmse = model_lstm_24.evaluate(x_test, y_test)
print("model lstm 24")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0238 - mean_absolute_error: 0.1237 - mean_absolute_percentage_error: 30.2591 - root_mean_squared_error: 0.1542
model lstm 24
Mean Square Error: 0.023777609691023827
Mean Absolute Error: 0.12368706613779068
Mean Absolute Percentage Error: 30.259063720703125
Root Mean Squared Error: 0.15419989824295044


## model 25 (LSTM=1024 512)

In [ ]:
lr = 0.0003

model_lstm_25 = Sequential()
model_lstm_25.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_25.add(LSTM(1024, activation='relu', return_sequences=True))
model_lstm_25.add(LSTM(512, activation='relu'))
model_lstm_25.add(Flatten())
model_lstm_25.add(Dense(7))
model_lstm_25.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_25.summary()

Model: "sequential_44"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_45 (LSTM)                  │ (None, 7, 1024)        │     4,222,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_46 (LSTM)                  │ (None, 512)            │     3,147,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_44 (Flatten)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,374,343 (28.13 MB)

 Trainable params: 7,374,343 (28.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_25.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 720ms/step - loss: 0.1864 - mean_absolute_error: 0.3681 - mean_absolute_percentage_error: 67.2716 - root_mean_squared_error: 0.4229 - val_loss: 0.0592 - val_mean_absolute_error: 0.1969 - val_mean_absolute_percentage_error: 15989226.0000 - val_root_mean_squared_error: 0.2432
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 673ms/step - loss: 0.0281 - mean_absolute_error: 0.1306 - mean_absolute_percentage_error: 34.0955 - root_mean_squared_error: 0.1672 - val_loss: 0.0520 - val_mean_absolute_error: 0.1883 - val_mean_absolute_percentage_error: 15597169.0000 - val_root_mean_squared_error: 0.2280
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 666ms/step - loss: 0.0227 - mean_absolute_error: 0.1180 - mean_absolute_percentage_error: 30.4885 - root_mean_squared_error: 0.1506 - val_loss: 0.0523 - val_mean_absolute_error: 0.1878 - val_mean_absolute_percentage_error: 15163625.0000 - val_root_mean_squared_error: 0.2286
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 668ms/s

In [ ]:
y_pred = model_lstm_25.predict(x_test)

evaluation_lstm_25 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_lstm_25

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step


{'Mean Squared Error: ': np.float64(316.91277572113967),
 'Mean Absolute Error: ': np.float64(14.169352570358587),
 'Root Mean Absolute Error: ': 17.802044144455426,
 'Mean Absolute Percentage Error: ': np.float64(25.577514425098176)}

In [ ]:
loss, mae, mape, rmse = model_lstm_25.evaluate(x_test, y_test)
print("model lstm 25")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 0.0253 - mean_absolute_error: 0.1265 - mean_absolute_percentage_error: 32.3677 - root_mean_squared_error: 0.1589
model lstm 25
Mean Square Error: 0.025264089927077293
Mean Absolute Error: 0.1265120655298233
Mean Absolute Percentage Error: 32.3676872253418
Root Mean Squared Error: 0.15894681215286255


# LSTM Hyperparameter

## Gridsearch

In [ ]:
# from keras.callbacks import EarlyStopping
# from sklearn.model_selection import GridSearchCV
# from scikeras.wrappers import KerasRegressor
# from keras.optimizers import Adam

# # 1. Define your model builder function
# def build_model(units1=64, units2=32, lr=0.001):
#     model = Sequential()
#     model.add(Input(shape=(7, 11)))
#     model.add(LSTM(units=units1, activation='relu', return_sequences=True))
#     model.add(LSTM(units=units2, activation='relu'))
#     model.add(Flatten())
#     model.add(Dense(7))
#     optimizer = Adam(learning_rate=lr)
#     model.compile(loss='mse', optimizer=optimizer,
#                   metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
#     return model

# # 2. Set up KerasRegressor
# model = KerasRegressor(model=build_model, verbose=0)

# # 3. Define early stopping
# early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # 4. Grid of parameters
# param_grid = {
#     'model__units1': [16, 32, 64],
#     'model__units2': [32, 64, 128],
#     'model__lr': [0.0003, 0.0002],
#     'batch_size': [42],
#     'epochs': [50]  # set high epochs so early stopping can interrupt early
# }

# # 5. GridSearchCV with early stopping passed via fit_params
# grid = GridSearchCV(estimator=model, param_grid=param_grid,
#                     scoring='neg_mean_squared_error',
#                     cv=3,
#                     refit=True)

# # 6. Fit with early stopping
# grid_result_cnn = grid.fit(x_train, y_train, callbacks=[early_stop], validation_data=(x_val, y_val))


In [ ]:
# print("Best parameters found: ", grid_result_cnn.best_params_)
# print("Best MSE: ", -grid_result.best_score_)

In [ ]:
# best_model = grid_result_cnn.best_estimator_
# loss, mae, mape, rmse = best_model.model_.evaluate(x_test, y_test)
# print("Test MSE:", loss)
# print("Test MAE:", mae)
# print("Test MAPE:", mape)
# print("Test RMSE:", rmse)

# Hybrid

# Model 2 layer CNN > LSTM

## model 1 (CNN=64 LSTM=64)

In [ ]:
lr = 0.0003

model_hybrid_1 = Sequential()
model_hybrid_1.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_1.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_1.add(Flatten())
model_hybrid_1.add(Dense(7))
model_hybrid_1.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_1.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_45 (Conv1D)              │ (None, 7, 64)          │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_47 (LSTM)                  │ (None, 7, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_45 (Flatten)            │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,615 (143.03 KB)

 Trainable params: 36,615 (143.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_1.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.2420 - mean_absolute_error: 0.4445 - mean_absolute_percentage_error: 78.2582 - root_mean_squared_error: 0.4882 - val_loss: 0.0590 - val_mean_absolute_error: 0.1942 - val_mean_absolute_percentage_error: 18847812.0000 - val_root_mean_squared_error: 0.2429
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0415 - mean_absolute_error: 0.1640 - mean_absolute_percentage_error: 43.1687 - root_mean_squared_error: 0.2034 - val_loss: 0.0652 - val_mean_absolute_error: 0.2141 - val_mean_absolute_percentage_error: 13230013.0000 - val_root_mean_squared_error: 0.2553
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0259 - mean_absolute_error: 0.1291 - mean_absolute_percentage_error: 30.4045 - root_mean_squared_error: 0.1610 - val_loss: 0.0508 - val_mean_absolute_error: 0.1857 - val_mean_absolute_percentage_error: 16036192.0000 - val_root_mean_squared_error: 0.2255
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - lo

In [ ]:
y_pred = model_hybrid_1.predict(x_test)

evaluation_hybrid_1 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step


{'Mean Squared Error: ': np.float64(368.07719860158966),
 'Mean Absolute Error: ': np.float64(14.585360215634719),
 'Root Mean Absolute Error: ': 19.185338115383573,
 'Mean Absolute Percentage Error: ': np.float64(27.464497393851005)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_1.evaluate(x_test, y_test)
print("model hybrid 1")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0293 - mean_absolute_error: 0.1302 - mean_absolute_percentage_error: 34.9834 - root_mean_squared_error: 0.1713
model hybrid 1
Mean Square Error: 0.029342887923121452
Mean Absolute Error: 0.13022644817829132
Mean Absolute Percentage Error: 34.98341751098633
Root Mean Squared Error: 0.17129765450954437


## model 2 (CNN=64 LSTM=32)

In [ ]:
lr = 0.0003

model_hybrid_2 = Sequential()
model_hybrid_2.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_2.add(LSTM(32, activation='tanh', return_sequences=True))
model_hybrid_2.add(Flatten())
model_hybrid_2.add(Dense(7))
model_hybrid_2.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_2.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_46"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_46 (Conv1D)              │ (None, 7, 64)          │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_48 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_46 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,439 (56.40 KB)

 Trainable params: 14,439 (56.40 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_2.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - loss: 0.2069 - mean_absolute_error: 0.4031 - mean_absolute_percentage_error: 71.7821 - root_mean_squared_error: 0.4523 - val_loss: 0.0682 - val_mean_absolute_error: 0.2112 - val_mean_absolute_percentage_error: 16631683.0000 - val_root_mean_squared_error: 0.2611
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0334 - mean_absolute_error: 0.1439 - mean_absolute_percentage_error: 35.8161 - root_mean_squared_error: 0.1828 - val_loss: 0.0590 - val_mean_absolute_error: 0.2052 - val_mean_absolute_percentage_error: 15237593.0000 - val_root_mean_squared_error: 0.2430
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0263 - mean_absolute_error: 0.1276 - mean_absolute_percentage_error: 30.2861 - root_mean_squared_error: 0.1619 - val_loss: 0.0532 - val_mean_absolute_error: 0.1901 - val_mean_absolute_percentage_error: 16756487.0000 - val_root_mean_squared_error: 0.2307
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - los

In [ ]:
y_pred = model_hybrid_2.predict(x_test)

evaluation_hybrid_2 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_2

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step


{'Mean Squared Error: ': np.float64(327.6028594153596),
 'Mean Absolute Error: ': np.float64(14.298462225466361),
 'Root Mean Absolute Error: ': 18.099802745205807,
 'Mean Absolute Percentage Error: ': np.float64(25.68796351144772)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_2.evaluate(x_test, y_test)
print("model hybrid 2")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0261 - mean_absolute_error: 0.1277 - mean_absolute_percentage_error: 32.4508 - root_mean_squared_error: 0.1616
model hybrid 2
Mean Square Error: 0.026116298511624336
Mean Absolute Error: 0.12766484916210175
Mean Absolute Percentage Error: 32.45075607299805
Root Mean Squared Error: 0.1616053730249405


## model 3 (CNN=32 LSTM=32)

In [ ]:
lr = 0.0003

model_hybrid_3 = Sequential()
model_hybrid_3.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_3.add(LSTM(32, activation='tanh', return_sequences=True))
model_hybrid_3.add(Flatten())
model_hybrid_3.add(Dense(7))
model_hybrid_3.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_3.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_47"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_47 (Conv1D)              │ (None, 7, 32)          │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_49 (LSTM)                  │ (None, 7, 32)          │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_47 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,119 (39.53 KB)

 Trainable params: 10,119 (39.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_3.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.2757 - mean_absolute_error: 0.4819 - mean_absolute_percentage_error: 85.9436 - root_mean_squared_error: 0.5233 - val_loss: 0.1069 - val_mean_absolute_error: 0.2818 - val_mean_absolute_percentage_error: 10622810.0000 - val_root_mean_squared_error: 0.3269
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0519 - mean_absolute_error: 0.1859 - mean_absolute_percentage_error: 37.2742 - root_mean_squared_error: 0.2268 - val_loss: 0.0561 - val_mean_absolute_error: 0.1925 - val_mean_absolute_percentage_error: 17608444.0000 - val_root_mean_squared_error: 0.2368
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0267 - mean_absolute_error: 0.1267 - mean_absolute_percentage_error: 31.1805 - root_mean_squared_error: 0.1635 - val_loss: 0.0540 - val_mean_absolute_error: 0.1900 - val_mean_absolute_percentage_error: 15986420.0000 - val_root_mean_squared_error: 0.2324
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - los

In [ ]:
y_pred = model_hybrid_3.predict(x_test)

evaluation_hybrid_3 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_3

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step


{'Mean Squared Error: ': np.float64(315.60906852084565),
 'Mean Absolute Error: ': np.float64(14.145307190564212),
 'Root Mean Absolute Error: ': 17.76538962479702,
 'Mean Absolute Percentage Error: ': np.float64(25.504562203188776)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_3.evaluate(x_test, y_test)
print("model hybrid 3")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0252 - mean_absolute_error: 0.1263 - mean_absolute_percentage_error: 32.2317 - root_mean_squared_error: 0.1586
model hybrid 3
Mean Square Error: 0.025160159915685654
Mean Absolute Error: 0.12629738450050354
Mean Absolute Percentage Error: 32.23171615600586
Root Mean Squared Error: 0.15861953794956207


## model 4 (CNN=24 LSTM=12)

In [ ]:
lr = 0.0003

model_hybrid_4 = Sequential()
model_hybrid_4.add(Conv1D(filters=24, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_4.add(LSTM(12, activation='tanh', return_sequences=True))
model_hybrid_4.add(Flatten())
model_hybrid_4.add(Dense(7))
model_hybrid_4.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_4.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_48"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_48 (Conv1D)              │ (None, 7, 24)          │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_50 (LSTM)                  │ (None, 7, 12)          │         1,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_48 (Flatten)            │ (None, 84)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 7)              │           595 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,539 (9.92 KB)

 Trainable params: 2,539 (9.92 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_4.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.2007 - mean_absolute_error: 0.4088 - mean_absolute_percentage_error: 72.0425 - root_mean_squared_error: 0.4470 - val_loss: 0.1011 - val_mean_absolute_error: 0.2713 - val_mean_absolute_percentage_error: 10624344.0000 - val_root_mean_squared_error: 0.3180
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0477 - mean_absolute_error: 0.1753 - mean_absolute_percentage_error: 36.1722 - root_mean_squared_error: 0.2179 - val_loss: 0.0523 - val_mean_absolute_error: 0.1850 - val_mean_absolute_percentage_error: 17255814.0000 - val_root_mean_squared_error: 0.2288
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0256 - mean_absolute_error: 0.1253 - mean_absolute_percentage_error: 33.2101 - root_mean_squared_error: 0.1600 - val_loss: 0.0569 - val_mean_absolute_error: 0.1934 - val_mean_absolute_percentage_error: 14686146.0000 - val_root_mean_squared_error: 0.2386
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss:

In [ ]:
y_pred = model_hybrid_4.predict(x_test)

evaluation_hybrid_4 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step


{'Mean Squared Error: ': np.float64(409.30828164831024),
 'Mean Absolute Error: ': np.float64(16.674587288681344),
 'Root Mean Absolute Error: ': 20.231368753703006,
 'Mean Absolute Percentage Error: ': np.float64(27.921256943160245)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_4.evaluate(x_test, y_test)
print("model hybrid 4")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0326 - mean_absolute_error: 0.1489 - mean_absolute_percentage_error: 34.7342 - root_mean_squared_error: 0.1806
model hybrid 4
Mean Square Error: 0.03262980282306671
Mean Absolute Error: 0.1488802433013916
Mean Absolute Percentage Error: 34.734153747558594
Root Mean Squared Error: 0.1806372106075287


## model 5 (CNN=256 LSTM=128)

In [ ]:
lr = 0.0003

model_hybrid_5 = Sequential()
model_hybrid_5.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_5.add(LSTM(128, activation='tanh', return_sequences=True))
model_hybrid_5.add(Flatten())
model_hybrid_5.add(Dense(7))
model_hybrid_5.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_5.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_49"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_49 (Conv1D)              │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_51 (LSTM)                  │ (None, 7, 128)         │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_49 (Flatten)            │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 7)              │         6,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,191 (801.53 KB)

 Trainable params: 205,191 (801.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_5.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 136ms/step - loss: 0.1691 - mean_absolute_error: 0.3461 - mean_absolute_percentage_error: 65.4936 - root_mean_squared_error: 0.4016 - val_loss: 0.0768 - val_mean_absolute_error: 0.2369 - val_mean_absolute_percentage_error: 11611185.0000 - val_root_mean_squared_error: 0.2770
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0328 - mean_absolute_error: 0.1476 - mean_absolute_percentage_error: 31.1895 - root_mean_squared_error: 0.1807 - val_loss: 0.0545 - val_mean_absolute_error: 0.1927 - val_mean_absolute_percentage_error: 14813805.0000 - val_root_mean_squared_error: 0.2334
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0230 - mean_absolute_error: 0.1190 - mean_absolute_percentage_error: 28.5486 - root_mean_squared_error: 0.1517 - val_loss: 0.0517 - val_mean_absolute_error: 0.1880 - val_mean_absolute_percentage_error: 15612118.0000 - val_root_mean_squared_error: 0.2273
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - l

In [ ]:
y_pred = model_hybrid_5.predict(x_test)

evaluation_hybrid_5 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_5

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step


{'Mean Squared Error: ': np.float64(278.67293421442804),
 'Mean Absolute Error: ': np.float64(12.875939797381966),
 'Root Mean Absolute Error: ': 16.693499759320332,
 'Mean Absolute Percentage Error: ': np.float64(23.449496326932877)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_5.evaluate(x_test, y_test)
print("model hybrid 5")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0222 - mean_absolute_error: 0.1150 - mean_absolute_percentage_error: 29.7374 - root_mean_squared_error: 0.1490
model hybrid 5
Mean Square Error: 0.02221563458442688
Mean Absolute Error: 0.11496375501155853
Mean Absolute Percentage Error: 29.737369537353516
Root Mean Squared Error: 0.14904910326004028


## model 6 (CNN=256 LSTM=64)

In [ ]:
lr = 0.0003

model_hybrid_6 = Sequential()
model_hybrid_6.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_6.add(LSTM(64, activation='tanh', return_sequences=True))
model_hybrid_6.add(Flatten())
model_hybrid_6.add(Dense(7))
model_hybrid_6.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_6.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_50 (Conv1D)              │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_52 (LSTM)                  │ (None, 7, 64)          │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_50 (Flatten)            │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 87,111 (340.28 KB)

 Trainable params: 87,111 (340.28 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_6.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - loss: 0.1653 - mean_absolute_error: 0.3476 - mean_absolute_percentage_error: 64.8745 - root_mean_squared_error: 0.3987 - val_loss: 0.0558 - val_mean_absolute_error: 0.1951 - val_mean_absolute_percentage_error: 15014584.0000 - val_root_mean_squared_error: 0.2361
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0280 - mean_absolute_error: 0.1354 - mean_absolute_percentage_error: 28.9500 - root_mean_squared_error: 0.1673 - val_loss: 0.0511 - val_mean_absolute_error: 0.1837 - val_mean_absolute_percentage_error: 17232232.0000 - val_root_mean_squared_error: 0.2260
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0229 - mean_absolute_error: 0.1178 - mean_absolute_percentage_error: 32.4966 - root_mean_squared_error: 0.1512 - val_loss: 0.0513 - val_mean_absolute_error: 0.1847 - val_mean_absolute_percentage_error: 16590054.0000 - val_root_mean_squared_error: 0.2266
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - lo

In [ ]:
y_pred = model_hybrid_6.predict(x_test)

evaluation_hybrid_6 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_6

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step


{'Mean Squared Error: ': np.float64(281.63863413084306),
 'Mean Absolute Error: ': np.float64(13.404991266678792),
 'Root Mean Absolute Error: ': 16.782092662443592,
 'Mean Absolute Percentage Error: ': np.float64(23.61762003946376)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_6.evaluate(x_test, y_test)
print("model hybrid 6")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0225 - mean_absolute_error: 0.1197 - mean_absolute_percentage_error: 29.7793 - root_mean_squared_error: 0.1498
model hybrid 6
Mean Square Error: 0.022452058270573616
Mean Absolute Error: 0.11968743056058884
Mean Absolute Percentage Error: 29.779266357421875
Root Mean Squared Error: 0.1498401015996933


## model 7 (CNN=32 LSTM=64)

In [ ]:
lr = 0.0003

model_hybrid_7 = Sequential()
model_hybrid_7.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_7.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_7.add(Flatten())
model_hybrid_7.add(Dense(7))
model_hybrid_7.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_7.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_51"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_51 (Conv1D)              │ (None, 7, 32)          │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_53 (LSTM)                  │ (None, 7, 64)          │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_51 (Flatten)            │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,199 (110.15 KB)

 Trainable params: 28,199 (110.15 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_7.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.3026 - mean_absolute_error: 0.5155 - mean_absolute_percentage_error: 88.6593 - root_mean_squared_error: 0.5482 - val_loss: 0.0773 - val_mean_absolute_error: 0.2383 - val_mean_absolute_percentage_error: 10932278.0000 - val_root_mean_squared_error: 0.2780
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0424 - mean_absolute_error: 0.1643 - mean_absolute_percentage_error: 39.4755 - root_mean_squared_error: 0.2056 - val_loss: 0.0551 - val_mean_absolute_error: 0.1996 - val_mean_absolute_percentage_error: 13492381.0000 - val_root_mean_squared_error: 0.2346
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0273 - mean_absolute_error: 0.1347 - mean_absolute_percentage_error: 30.2617 - root_mean_squared_error: 0.1653 - val_loss: 0.0514 - val_mean_absolute_error: 0.1890 - val_mean_absolute_percentage_error: 15750573.0000 - val_root_mean_squared_error: 0.2268
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - lo

In [ ]:
y_pred = model_hybrid_7.predict(x_test)

evaluation_hybrid_7 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_7

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step


{'Mean Squared Error: ': np.float64(298.72962497714514),
 'Mean Absolute Error: ': np.float64(13.601172836459414),
 'Root Mean Absolute Error: ': 17.283796601937468,
 'Mean Absolute Percentage Error: ': np.float64(24.964750144738453)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_7.evaluate(x_test, y_test)
print("model hybrid 7")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0238 - mean_absolute_error: 0.1214 - mean_absolute_percentage_error: 31.6904 - root_mean_squared_error: 0.1543
model hybrid 7
Mean Square Error: 0.02381454035639763
Mean Absolute Error: 0.12143903970718384
Mean Absolute Percentage Error: 31.69041633605957
Root Mean Squared Error: 0.15431959927082062


## model 8 (CNN=1024 LSTM=128)

In [ ]:
lr = 0.0003

model_hybrid_8 = Sequential()
model_hybrid_8.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_8.add(LSTM(128, activation='relu', return_sequences=True))
model_hybrid_8.add(Flatten())
model_hybrid_8.add(Dense(7))
model_hybrid_8.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_8.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_52"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_52 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_54 (LSTM)                  │ (None, 7, 128)         │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_52 (Flatten)            │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 7)              │         6,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,783 (2.30 MB)

 Trainable params: 603,783 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_8.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - loss: 0.1960 - mean_absolute_error: 0.3779 - mean_absolute_percentage_error: 68.6088 - root_mean_squared_error: 0.4322 - val_loss: 0.0704 - val_mean_absolute_error: 0.2248 - val_mean_absolute_percentage_error: 12416438.0000 - val_root_mean_squared_error: 0.2654
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0292 - mean_absolute_error: 0.1367 - mean_absolute_percentage_error: 31.1849 - root_mean_squared_error: 0.1706 - val_loss: 0.0554 - val_mean_absolute_error: 0.1961 - val_mean_absolute_percentage_error: 14632225.0000 - val_root_mean_squared_error: 0.2353
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0224 - mean_absolute_error: 0.1171 - mean_absolute_percentage_error: 28.6269 - root_mean_squared_error: 0.1497 - val_loss: 0.0531 - val_mean_absolute_error: 0.1920 - val_mean_absolute_percentage_error: 15110131.0000 - val_root_mean_squared_error: 0.2304
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - lo

In [ ]:
y_pred = model_hybrid_8.predict(x_test)

evaluation_hybrid_8 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_8

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step


{'Mean Squared Error: ': np.float64(297.3300744137226),
 'Mean Absolute Error: ': np.float64(13.170095774592186),
 'Root Mean Absolute Error: ': 17.2432617104109,
 'Mean Absolute Percentage Error: ': np.float64(24.34830239326961)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_8.evaluate(x_test, y_test)
print("model hybrid 8")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0237 - mean_absolute_error: 0.1176 - mean_absolute_percentage_error: 30.9664 - root_mean_squared_error: 0.1540
model hybrid 8
Mean Square Error: 0.023702967911958694
Mean Absolute Error: 0.11759013682603836
Mean Absolute Percentage Error: 30.96636962890625
Root Mean Squared Error: 0.15395767986774445


# model 4 layer CNN > LSTM

## model 9 (CNN=1024, 512 LSTM=128, 64)

In [ ]:
lr = 0.0003

model_hybrid_9 = Sequential()
model_hybrid_9.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_9.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_9.add(LSTM(128, activation='relu', return_sequences=True))
model_hybrid_9.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_9.add(Flatten())
model_hybrid_9.add(Dense(7))
model_hybrid_9.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_9.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_53"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_53 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_54 (Conv1D)              │ (None, 7, 512)         │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_55 (LSTM)                  │ (None, 7, 128)         │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_56 (LSTM)                  │ (None, 7, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_53 (Flatten)            │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 912,711 (3.48 MB)

 Trainable params: 912,711 (3.48 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_9.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 162ms/step - loss: 0.2148 - mean_absolute_error: 0.4025 - mean_absolute_percentage_error: 72.4607 - root_mean_squared_error: 0.4553 - val_loss: 0.0625 - val_mean_absolute_error: 0.2081 - val_mean_absolute_percentage_error: 14610588.0000 - val_root_mean_squared_error: 0.2500
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - loss: 0.0287 - mean_absolute_error: 0.1337 - mean_absolute_percentage_error: 33.0033 - root_mean_squared_error: 0.1692 - val_loss: 0.0532 - val_mean_absolute_error: 0.1919 - val_mean_absolute_percentage_error: 14867209.0000 - val_root_mean_squared_error: 0.2306
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - loss: 0.0223 - mean_absolute_error: 0.1165 - mean_absolute_percentage_error: 29.4935 - root_mean_squared_error: 0.1494 - val_loss: 0.0544 - val_mean_absolute_error: 0.1953 - val_mean_absolute_percentage_error: 13861393.0000 - val_root_mean_squared_error: 0.2332
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step 

In [ ]:
y_pred = model_hybrid_9.predict(x_test)

evaluation_hybrid_9 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_9

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step


{'Mean Squared Error: ': np.float64(351.1029415015795),
 'Mean Absolute Error: ': np.float64(14.309362645051916),
 'Root Mean Absolute Error: ': 18.737741099224834,
 'Mean Absolute Percentage Error: ': np.float64(26.9330533094466)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_9.evaluate(x_test, y_test)
print("model hybrid 9")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0280 - mean_absolute_error: 0.1278 - mean_absolute_percentage_error: 34.3203 - root_mean_squared_error: 0.1673
model hybrid 9
Mean Square Error: 0.027989709749817848
Mean Absolute Error: 0.12776215374469757
Mean Absolute Percentage Error: 34.320281982421875
Root Mean Squared Error: 0.1673012524843216


## model 10 (CNN=1024, 512 LSTM=128, 128)

In [ ]:
lr = 0.0003

model_hybrid_10 = Sequential()
model_hybrid_10.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_10.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_10.add(LSTM(128, activation='relu', return_sequences=True))
model_hybrid_10.add(LSTM(128, activation='relu', return_sequences=True))
model_hybrid_10.add(Flatten())
model_hybrid_10.add(Dense(7))
model_hybrid_10.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_10.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_54"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_55 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_56 (Conv1D)              │ (None, 7, 512)         │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_57 (LSTM)                  │ (None, 7, 128)         │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_58 (LSTM)                  │ (None, 7, 128)         │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_54 (Flatten)            │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 7)              │         6,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 998,023 (3.81 MB)

 Trainable params: 998,023 (3.81 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_10.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 193ms/step - loss: 0.2380 - mean_absolute_error: 0.4346 - mean_absolute_percentage_error: 78.4062 - root_mean_squared_error: 0.4824 - val_loss: 0.1041 - val_mean_absolute_error: 0.2829 - val_mean_absolute_percentage_error: 9113855.0000 - val_root_mean_squared_error: 0.3227
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - loss: 0.0499 - mean_absolute_error: 0.1858 - mean_absolute_percentage_error: 37.8719 - root_mean_squared_error: 0.2218 - val_loss: 0.0604 - val_mean_absolute_error: 0.2045 - val_mean_absolute_percentage_error: 13269165.0000 - val_root_mean_squared_error: 0.2458
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - loss: 0.0262 - mean_absolute_error: 0.1288 - mean_absolute_percentage_error: 31.4432 - root_mean_squared_error: 0.1618 - val_loss: 0.0522 - val_mean_absolute_error: 0.1888 - val_mean_absolute_percentage_error: 15054039.0000 - val_root_mean_squared_error: 0.2284
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step -

In [ ]:
y_pred = model_hybrid_10.predict(x_test)

evaluation_hybrid_10 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_10

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step


{'Mean Squared Error: ': np.float64(336.56983218861296),
 'Mean Absolute Error: ': np.float64(14.560619198546119),
 'Root Mean Absolute Error: ': 18.345839642507862,
 'Mean Absolute Percentage Error: ': np.float64(26.094718172552483)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_10.evaluate(x_test, y_test)
print("model hybrid 10")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0268 - mean_absolute_error: 0.1300 - mean_absolute_percentage_error: 32.9690 - root_mean_squared_error: 0.1638
model hybrid 10
Mean Square Error: 0.026831140741705894
Mean Absolute Error: 0.13000552356243134
Mean Absolute Percentage Error: 32.96895980834961
Root Mean Squared Error: 0.1638021320104599


## model 11 (CNN=1024, 512 LSTM=256, 128)

In [ ]:
lr = 0.0003

model_hybrid_11 = Sequential()
model_hybrid_11.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_11.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_11.add(LSTM(256, activation='relu', return_sequences=True))
model_hybrid_11.add(LSTM(128, activation='relu', return_sequences=True))
model_hybrid_11.add(Flatten())
model_hybrid_11.add(Dense(7))
model_hybrid_11.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_11.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_55"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_57 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_58 (Conv1D)              │ (None, 7, 512)         │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_59 (LSTM)                  │ (None, 7, 256)         │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_60 (LSTM)                  │ (None, 7, 128)         │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_55 (Flatten)            │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 7)              │         6,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,522,823 (5.81 MB)

 Trainable params: 1,522,823 (5.81 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_11.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 205ms/step - loss: 0.2259 - mean_absolute_error: 0.4162 - mean_absolute_percentage_error: 74.8952 - root_mean_squared_error: 0.4687 - val_loss: 0.0600 - val_mean_absolute_error: 0.2047 - val_mean_absolute_percentage_error: 14503196.0000 - val_root_mean_squared_error: 0.2449
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 216ms/step - loss: 0.0292 - mean_absolute_error: 0.1356 - mean_absolute_percentage_error: 33.5384 - root_mean_squared_error: 0.1708 - val_loss: 0.0507 - val_mean_absolute_error: 0.1869 - val_mean_absolute_percentage_error: 15600011.0000 - val_root_mean_squared_error: 0.2252
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - loss: 0.0219 - mean_absolute_error: 0.1160 - mean_absolute_percentage_error: 29.3830 - root_mean_squared_error: 0.1480 - val_loss: 0.0535 - val_mean_absolute_error: 0.1932 - val_mean_absolute_percentage_error: 14234332.0000 - val_root_mean_squared_error: 0.2313
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 158ms/step 

In [ ]:
y_pred = model_hybrid_11.predict(x_test)

evaluation_hybrid_11 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_11

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step


{'Mean Squared Error: ': np.float64(357.2811472876997),
 'Mean Absolute Error: ': np.float64(14.423620379700955),
 'Root Mean Absolute Error: ': 18.901882109665685,
 'Mean Absolute Percentage Error: ': np.float64(27.190408651718467)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_11.evaluate(x_test, y_test)
print("model hybrid 11")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0285 - mean_absolute_error: 0.1288 - mean_absolute_percentage_error: 34.6417 - root_mean_squared_error: 0.1688
model hybrid 11
Mean Square Error: 0.028482230380177498
Mean Absolute Error: 0.12878231704235077
Mean Absolute Percentage Error: 34.641685485839844
Root Mean Squared Error: 0.1687667965888977


## model 12 (CNN=1024, 512 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_12 = Sequential()
model_hybrid_12.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_12.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_12.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_12.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_12.add(Flatten())
model_hybrid_12.add(Dense(7))
model_hybrid_12.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_12.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_56"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_59 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_60 (Conv1D)              │ (None, 7, 512)         │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_61 (LSTM)                  │ (None, 7, 64)          │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_62 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_56 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 693,671 (2.65 MB)

 Trainable params: 693,671 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_12.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - loss: 0.2561 - mean_absolute_error: 0.4597 - mean_absolute_percentage_error: 82.9061 - root_mean_squared_error: 0.5024 - val_loss: 0.0564 - val_mean_absolute_error: 0.1990 - val_mean_absolute_percentage_error: 14069816.0000 - val_root_mean_squared_error: 0.2376
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - loss: 0.0291 - mean_absolute_error: 0.1345 - mean_absolute_percentage_error: 32.2197 - root_mean_squared_error: 0.1706 - val_loss: 0.0523 - val_mean_absolute_error: 0.1916 - val_mean_absolute_percentage_error: 14798101.0000 - val_root_mean_squared_error: 0.2288
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - loss: 0.0244 - mean_absolute_error: 0.1246 - mean_absolute_percentage_error: 31.3124 - root_mean_squared_error: 0.1562 - val_loss: 0.0538 - val_mean_absolute_error: 0.1938 - val_mean_absolute_percentage_error: 14398817.0000 - val_root_mean_squared_error: 0.2320
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step 

In [ ]:
y_pred = model_hybrid_12.predict(x_test)

evaluation_hybrid_12 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_12

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step


{'Mean Squared Error: ': np.float64(266.5441483777805),
 'Mean Absolute Error: ': np.float64(12.834602122404137),
 'Root Mean Absolute Error: ': 16.32617984642398,
 'Mean Absolute Percentage Error: ': np.float64(23.03219316799609)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_12.evaluate(x_test, y_test)
print("model hybrid 12")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0212 - mean_absolute_error: 0.1146 - mean_absolute_percentage_error: 29.1675 - root_mean_squared_error: 0.1458
model hybrid 12
Mean Square Error: 0.02124873362481594
Mean Absolute Error: 0.11459465324878693
Mean Absolute Percentage Error: 29.167478561401367
Root Mean Squared Error: 0.14576944708824158


## model 13 (CNN=1024, 512 LSTM=32, 64)

In [ ]:
lr = 0.0003

model_hybrid_13 = Sequential()
model_hybrid_13.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_13.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_13.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_13.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_13.add(Flatten())
model_hybrid_13.add(Dense(7))
model_hybrid_13.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_13.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_57"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_61 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_62 (Conv1D)              │ (None, 7, 512)         │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_63 (LSTM)                  │ (None, 7, 32)          │        69,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_64 (LSTM)                  │ (None, 7, 64)          │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_57 (Flatten)            │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 7)              │         3,143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 629,703 (2.40 MB)

 Trainable params: 629,703 (2.40 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_13.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 194ms/step - loss: 0.2342 - mean_absolute_error: 0.4311 - mean_absolute_percentage_error: 77.4188 - root_mean_squared_error: 0.4786 - val_loss: 0.0759 - val_mean_absolute_error: 0.2308 - val_mean_absolute_percentage_error: 11887797.0000 - val_root_mean_squared_error: 0.2754
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - loss: 0.0367 - mean_absolute_error: 0.1547 - mean_absolute_percentage_error: 34.1553 - root_mean_squared_error: 0.1910 - val_loss: 0.0540 - val_mean_absolute_error: 0.1922 - val_mean_absolute_percentage_error: 15092725.0000 - val_root_mean_squared_error: 0.2324
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - loss: 0.0231 - mean_absolute_error: 0.1188 - mean_absolute_percentage_error: 30.5126 - root_mean_squared_error: 0.1520 - val_loss: 0.0521 - val_mean_absolute_error: 0.1894 - val_mean_absolute_percentage_error: 14771529.0000 - val_root_mean_squared_error: 0.2282
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step 

In [ ]:
y_pred = model_hybrid_13.predict(x_test)

evaluation_hybrid_13 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_13

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step


{'Mean Squared Error: ': np.float64(288.4177258759487),
 'Mean Absolute Error: ': np.float64(13.257998252401546),
 'Root Mean Absolute Error: ': 16.982865655593837,
 'Mean Absolute Percentage Error: ': np.float64(24.435759835557473)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_13.evaluate(x_test, y_test)
print("model hybrid 13")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0230 - mean_absolute_error: 0.1184 - mean_absolute_percentage_error: 31.0755 - root_mean_squared_error: 0.1516
model hybrid 13
Mean Square Error: 0.022992482408881187
Mean Absolute Error: 0.11837499588727951
Mean Absolute Percentage Error: 31.075515747070312
Root Mean Squared Error: 0.15163272619247437


## model 14 (CNN=1024, 512 LSTM=64, 128)

In [ ]:
lr = 0.0003

model_hybrid_14 = Sequential()
model_hybrid_14.add(Conv1D(filters=1024, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_14.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_14.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_14.add(LSTM(128, activation='relu', return_sequences=True))
model_hybrid_14.add(Flatten())
model_hybrid_14.add(Dense(7))
model_hybrid_14.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_14.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_58"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_63 (Conv1D)              │ (None, 7, 1024)        │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_64 (Conv1D)              │ (None, 7, 512)         │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_65 (LSTM)                  │ (None, 7, 64)          │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_66 (LSTM)                  │ (None, 7, 128)         │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_58 (Flatten)            │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 7)              │         6,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 784,775 (2.99 MB)

 Trainable params: 784,775 (2.99 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_14.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 195ms/step - loss: 0.2204 - mean_absolute_error: 0.4107 - mean_absolute_percentage_error: 74.6147 - root_mean_squared_error: 0.4626 - val_loss: 0.0661 - val_mean_absolute_error: 0.2177 - val_mean_absolute_percentage_error: 12951584.0000 - val_root_mean_squared_error: 0.2570
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - loss: 0.0292 - mean_absolute_error: 0.1340 - mean_absolute_percentage_error: 32.1113 - root_mean_squared_error: 0.1706 - val_loss: 0.0523 - val_mean_absolute_error: 0.1900 - val_mean_absolute_percentage_error: 15297116.0000 - val_root_mean_squared_error: 0.2286
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - loss: 0.0221 - mean_absolute_error: 0.1167 - mean_absolute_percentage_error: 29.4476 - root_mean_squared_error: 0.1486 - val_loss: 0.0530 - val_mean_absolute_error: 0.1918 - val_mean_absolute_percentage_error: 14666424.0000 - val_root_mean_squared_error: 0.2303
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step 

In [ ]:
y_pred = model_hybrid_14.predict(x_test)

evaluation_hybrid_14 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_14

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step


{'Mean Squared Error: ': np.float64(286.74235388195257),
 'Mean Absolute Error: ': np.float64(13.61697397426683),
 'Root Mean Absolute Error: ': 16.933468453980495,
 'Mean Absolute Percentage Error: ': np.float64(23.53350241163923)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_14.evaluate(x_test, y_test)
print("model hybrid 14")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0229 - mean_absolute_error: 0.1216 - mean_absolute_percentage_error: 29.5631 - root_mean_squared_error: 0.1512
model hybrid 14
Mean Square Error: 0.02285892330110073
Mean Absolute Error: 0.12158012390136719
Mean Absolute Percentage Error: 29.563093185424805
Root Mean Squared Error: 0.15119168162345886


# Model 3 layer CNN > LSTM

## model 15 (CNN=16 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_15 = Sequential()
model_hybrid_15.add(Conv1D(filters=16, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_15.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_15.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_15.add(Flatten())
model_hybrid_15.add(Dense(7))
model_hybrid_15.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_15.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_59"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_65 (Conv1D)              │ (None, 7, 16)          │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_67 (LSTM)                  │ (None, 7, 64)          │        20,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_68 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_59 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,839 (136.09 KB)

 Trainable params: 34,839 (136.09 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_15.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - loss: 0.2858 - mean_absolute_error: 0.5041 - mean_absolute_percentage_error: 90.9135 - root_mean_squared_error: 0.5344 - val_loss: 0.1560 - val_mean_absolute_error: 0.3515 - val_mean_absolute_percentage_error: 6017825.0000 - val_root_mean_squared_error: 0.3950
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0849 - mean_absolute_error: 0.2424 - mean_absolute_percentage_error: 45.7056 - root_mean_squared_error: 0.2889 - val_loss: 0.0534 - val_mean_absolute_error: 0.2012 - val_mean_absolute_percentage_error: 13307236.0000 - val_root_mean_squared_error: 0.2310
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0282 - mean_absolute_error: 0.1382 - mean_absolute_percentage_error: 29.0797 - root_mean_squared_error: 0.1679 - val_loss: 0.0519 - val_mean_absolute_error: 0.1921 - val_mean_absolute_percentage_error: 14961973.0000 - val_root_mean_squared_error: 0.2279
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - los

In [ ]:
y_pred = model_hybrid_15.predict(x_test)

evaluation_hybrid_15 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_15

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step


{'Mean Squared Error: ': np.float64(353.5128758538434),
 'Mean Absolute Error: ': np.float64(14.94358101669623),
 'Root Mean Absolute Error: ': 18.80193808770371,
 'Mean Absolute Percentage Error: ': np.float64(27.13061628263254)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_15.evaluate(x_test, y_test)
print("model hybrid 15")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0282 - mean_absolute_error: 0.1334 - mean_absolute_percentage_error: 34.3193 - root_mean_squared_error: 0.1679
model hybrid 15
Mean Square Error: 0.028181826695799828
Mean Absolute Error: 0.13342483341693878
Mean Absolute Percentage Error: 34.31930923461914
Root Mean Squared Error: 0.16787444055080414


## model 16 (CNN=32 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_16 = Sequential()
model_hybrid_16.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_16.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_16.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_16.add(Flatten())
model_hybrid_16.add(Dense(7))
model_hybrid_16.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_16.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_60"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_66 (Conv1D)              │ (None, 7, 32)          │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_69 (LSTM)                  │ (None, 7, 64)          │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_70 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_60 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,047 (152.53 KB)

 Trainable params: 39,047 (152.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_16.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 0.3133 - mean_absolute_error: 0.5300 - mean_absolute_percentage_error: 95.6103 - root_mean_squared_error: 0.5596 - val_loss: 0.2266 - val_mean_absolute_error: 0.4352 - val_mean_absolute_percentage_error: 4239079.0000 - val_root_mean_squared_error: 0.4760
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1321 - mean_absolute_error: 0.3140 - mean_absolute_percentage_error: 56.8090 - root_mean_squared_error: 0.3602 - val_loss: 0.0576 - val_mean_absolute_error: 0.1944 - val_mean_absolute_percentage_error: 14687711.0000 - val_root_mean_squared_error: 0.2400
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0311 - mean_absolute_error: 0.1445 - mean_absolute_percentage_error: 30.8400 - root_mean_squared_error: 0.1762 - val_loss: 0.0484 - val_mean_absolute_error: 0.1775 - val_mean_absolute_percentage_error: 16216994.0000 - val_root_mean_squared_error: 0.2201
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - los

In [ ]:
y_pred = model_hybrid_16.predict(x_test)

evaluation_hybrid_16 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_16

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step


{'Mean Squared Error: ': np.float64(351.9614834178192),
 'Mean Absolute Error: ': np.float64(14.976647630029797),
 'Root Mean Absolute Error: ': 18.76063654084848,
 'Mean Absolute Percentage Error: ': np.float64(26.773198328737035)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_16.evaluate(x_test, y_test)
print("model hybrid 16")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0281 - mean_absolute_error: 0.1337 - mean_absolute_percentage_error: 33.7933 - root_mean_squared_error: 0.1675
model hybrid 16
Mean Square Error: 0.02805815264582634
Mean Absolute Error: 0.13372007012367249
Mean Absolute Percentage Error: 33.79328155517578
Root Mean Squared Error: 0.16750568151474


## model 17 (CNN=64 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_17 = Sequential()
model_hybrid_17.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_17.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_17.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_17.add(Flatten())
model_hybrid_17.add(Dense(7))
model_hybrid_17.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_17.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_61"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_67 (Conv1D)              │ (None, 7, 64)          │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_71 (LSTM)                  │ (None, 7, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_72 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_61 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,463 (185.40 KB)

 Trainable params: 47,463 (185.40 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_17.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - loss: 0.3371 - mean_absolute_error: 0.5533 - mean_absolute_percentage_error: 97.9648 - root_mean_squared_error: 0.5805 - val_loss: 0.2556 - val_mean_absolute_error: 0.4610 - val_mean_absolute_percentage_error: 2744159.0000 - val_root_mean_squared_error: 0.5056
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.1800 - mean_absolute_error: 0.3760 - mean_absolute_percentage_error: 65.6055 - root_mean_squared_error: 0.4214 - val_loss: 0.0708 - val_mean_absolute_error: 0.2165 - val_mean_absolute_percentage_error: 15684413.0000 - val_root_mean_squared_error: 0.2661
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0330 - mean_absolute_error: 0.1411 - mean_absolute_percentage_error: 33.7478 - root_mean_squared_error: 0.1810 - val_loss: 0.0531 - val_mean_absolute_error: 0.1917 - val_mean_absolute_percentage_error: 15181515.0000 - val_root_mean_squared_error: 0.2305
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - los

In [ ]:
y_pred = model_hybrid_17.predict(x_test)

evaluation_hybrid_17 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_17

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step


{'Mean Squared Error: ': np.float64(422.32530408931717),
 'Mean Absolute Error: ': np.float64(15.953688563132772),
 'Root Mean Absolute Error: ': 20.550554836532203,
 'Mean Absolute Percentage Error: ': np.float64(29.503985368979198)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_17.evaluate(x_test, y_test)
print("model hybrid 17")
print("Mean Square E17rror:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0337 - mean_absolute_error: 0.1424 - mean_absolute_percentage_error: 37.4087 - root_mean_squared_error: 0.1835
model hybrid 17
Mean Square E17rror: 0.03366750851273537
Mean Absolute Error: 0.14244364202022552
Mean Absolute Percentage Error: 37.4086799621582
Root Mean Squared Error: 0.18348707258701324


## model 18 (CNN=128 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_18 = Sequential()
model_hybrid_18.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_18.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_18.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_18.add(Flatten())
model_hybrid_18.add(Dense(7))
model_hybrid_18.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_18.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_62"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_68 (Conv1D)              │ (None, 7, 128)         │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_73 (LSTM)                  │ (None, 7, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_74 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_62 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,295 (251.15 KB)

 Trainable params: 64,295 (251.15 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_18.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - loss: 0.3008 - mean_absolute_error: 0.5177 - mean_absolute_percentage_error: 90.5848 - root_mean_squared_error: 0.5475 - val_loss: 0.0886 - val_mean_absolute_error: 0.2538 - val_mean_absolute_percentage_error: 10945373.0000 - val_root_mean_squared_error: 0.2976
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0469 - mean_absolute_error: 0.1706 - mean_absolute_percentage_error: 39.3987 - root_mean_squared_error: 0.2161 - val_loss: 0.0621 - val_mean_absolute_error: 0.2118 - val_mean_absolute_percentage_error: 13009717.0000 - val_root_mean_squared_error: 0.2493
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0267 - mean_absolute_error: 0.1276 - mean_absolute_percentage_error: 29.4629 - root_mean_squared_error: 0.1633 - val_loss: 0.0530 - val_mean_absolute_error: 0.1908 - val_mean_absolute_percentage_error: 14656774.0000 - val_root_mean_squared_error: 0.2302
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - lo

In [ ]:
y_pred = model_hybrid_18.predict(x_test)

evaluation_hybrid_18 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_18

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step


{'Mean Squared Error: ': np.float64(360.5353105441474),
 'Mean Absolute Error: ': np.float64(15.216635840279713),
 'Root Mean Absolute Error: ': 18.987767392301482,
 'Mean Absolute Percentage Error: ': np.float64(27.356577334010773)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_18.evaluate(x_test, y_test)
print("model hybrid 18")
print("Mean Square E17rror:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0287 - mean_absolute_error: 0.1359 - mean_absolute_percentage_error: 34.5461 - root_mean_squared_error: 0.1695
model hybrid 18
Mean Square E17rror: 0.02874165214598179
Mean Absolute Error: 0.135862797498703
Mean Absolute Percentage Error: 34.546051025390625
Root Mean Squared Error: 0.1695336252450943


## model 19 (CNN=256 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_19 = Sequential()
model_hybrid_19.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_19.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_19.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_19.add(Flatten())
model_hybrid_19.add(Dense(7))
model_hybrid_19.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_19.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_63"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_69 (Conv1D)              │ (None, 7, 256)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_75 (LSTM)                  │ (None, 7, 64)          │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_76 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_63 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,959 (382.65 KB)

 Trainable params: 97,959 (382.65 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_19.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - loss: 0.3037 - mean_absolute_error: 0.5185 - mean_absolute_percentage_error: 91.8652 - root_mean_squared_error: 0.5504 - val_loss: 0.1145 - val_mean_absolute_error: 0.2856 - val_mean_absolute_percentage_error: 11757151.0000 - val_root_mean_squared_error: 0.3384
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0705 - mean_absolute_error: 0.2143 - mean_absolute_percentage_error: 46.7119 - root_mean_squared_error: 0.2632 - val_loss: 0.0551 - val_mean_absolute_error: 0.1977 - val_mean_absolute_percentage_error: 14522072.0000 - val_root_mean_squared_error: 0.2347
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0268 - mean_absolute_error: 0.1276 - mean_absolute_percentage_error: 31.3539 - root_mean_squared_error: 0.1636 - val_loss: 0.0514 - val_mean_absolute_error: 0.1882 - val_mean_absolute_percentage_error: 15011375.0000 - val_root_mean_squared_error: 0.2266
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - lo

In [ ]:
y_pred = model_hybrid_19.predict(x_test)

evaluation_hybrid_19 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_19

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step


{'Mean Squared Error: ': np.float64(320.39316870466865),
 'Mean Absolute Error: ': np.float64(14.109604563031876),
 'Root Mean Absolute Error: ': 17.899529845911278,
 'Mean Absolute Percentage Error: ': np.float64(25.83768973632542)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_19.evaluate(x_test, y_test)
print("model hybrid 19")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0255 - mean_absolute_error: 0.1260 - mean_absolute_percentage_error: 32.7662 - root_mean_squared_error: 0.1598
model hybrid 19
Mean Square Error: 0.02554154582321644
Mean Absolute Error: 0.12597860395908356
Mean Absolute Percentage Error: 32.76620864868164
Root Mean Squared Error: 0.15981723368167877


## model 20 (CNN=512 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_20 = Sequential()
model_hybrid_20.add(Conv1D(filters=512, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_20.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_20.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_20.add(Flatten())
model_hybrid_20.add(Dense(7))
model_hybrid_20.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_20.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_64"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_70 (Conv1D)              │ (None, 7, 512)         │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_77 (LSTM)                  │ (None, 7, 64)          │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_78 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_64 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 165,287 (645.65 KB)

 Trainable params: 165,287 (645.65 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_20.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - loss: 0.2951 - mean_absolute_error: 0.5085 - mean_absolute_percentage_error: 89.8562 - root_mean_squared_error: 0.5418 - val_loss: 0.0741 - val_mean_absolute_error: 0.2172 - val_mean_absolute_percentage_error: 17206364.0000 - val_root_mean_squared_error: 0.2721
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0502 - mean_absolute_error: 0.1768 - mean_absolute_percentage_error: 42.6602 - root_mean_squared_error: 0.2229 - val_loss: 0.0526 - val_mean_absolute_error: 0.1927 - val_mean_absolute_percentage_error: 14372920.0000 - val_root_mean_squared_error: 0.2293
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0259 - mean_absolute_error: 0.1255 - mean_absolute_percentage_error: 32.6235 - root_mean_squared_error: 0.1608 - val_loss: 0.0539 - val_mean_absolute_error: 0.1943 - val_mean_absolute_percentage_error: 14176785.0000 - val_root_mean_squared_error: 0.2322
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - lo

In [ ]:
y_pred = model_hybrid_20.predict(x_test)

evaluation_hybrid_20 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_20

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step


{'Mean Squared Error: ': np.float64(277.3340576643997),
 'Mean Absolute Error: ': np.float64(13.322164613373426),
 'Root Mean Absolute Error: ': 16.653349743051688,
 'Mean Absolute Percentage Error: ': np.float64(23.60073828402595)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_20.evaluate(x_test, y_test)
print("model hybrid 20")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0221 - mean_absolute_error: 0.1189 - mean_absolute_percentage_error: 29.7837 - root_mean_squared_error: 0.1487
model hybrid 20
Mean Square Error: 0.022108901292085648
Mean Absolute Error: 0.11894790083169937
Mean Absolute Percentage Error: 29.783672332763672
Root Mean Squared Error: 0.1486906260251999


## model 21 (CNN=300 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_21 = Sequential()
model_hybrid_21.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_21.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_21.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_21.add(Flatten())
model_hybrid_21.add(Dense(7))
model_hybrid_21.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_21.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_65"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_71 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_79 (LSTM)                  │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_80 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_65 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,531 (427.86 KB)

 Trainable params: 109,531 (427.86 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_21.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - loss: 0.3224 - mean_absolute_error: 0.5379 - mean_absolute_percentage_error: 94.9853 - root_mean_squared_error: 0.5673 - val_loss: 0.1589 - val_mean_absolute_error: 0.3476 - val_mean_absolute_percentage_error: 7296527.5000 - val_root_mean_squared_error: 0.3986
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0889 - mean_absolute_error: 0.2424 - mean_absolute_percentage_error: 46.9308 - root_mean_squared_error: 0.2960 - val_loss: 0.0622 - val_mean_absolute_error: 0.2119 - val_mean_absolute_percentage_error: 13124200.0000 - val_root_mean_squared_error: 0.2495
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0245 - mean_absolute_error: 0.1234 - mean_absolute_percentage_error: 28.4956 - root_mean_squared_error: 0.1565 - val_loss: 0.0570 - val_mean_absolute_error: 0.2027 - val_mean_absolute_percentage_error: 13877120.0000 - val_root_mean_squared_error: 0.2387
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - los

In [ ]:
y_pred = model_hybrid_21.predict(x_test)

evaluation_21 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_21

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step


{'Mean Squared Error: ': np.float64(336.418936132669),
 'Mean Absolute Error: ': np.float64(14.616225203689265),
 'Root Mean Absolute Error: ': 18.341726639895956,
 'Mean Absolute Percentage Error: ': np.float64(26.948818994343327)}

In [ ]:
# loss, mae, mape, rmse = model_hybrid_21.evaluate(x_test, y_test)
# print("model hybrid 21")
# print("Mean Square Error:", loss)
# print("Mean Absolute Error:", mae)
# print("Mean Absolute Percentage Error:", mape)
# print("Root Mean Squared Error:", rmse)

## model 22 (CNN=350 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_22 = Sequential()
model_hybrid_22.add(Conv1D(filters=350, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_22.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_22.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_22.add(Flatten())
model_hybrid_22.add(Dense(7))
model_hybrid_22.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_22.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_66"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_72 (Conv1D)              │ (None, 7, 350)         │         2,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_81 (LSTM)                  │ (None, 7, 64)          │       106,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_82 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_66 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,681 (479.22 KB)

 Trainable params: 122,681 (479.22 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_22.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 93ms/step - loss: 0.2506 - mean_absolute_error: 0.4580 - mean_absolute_percentage_error: 80.6196 - root_mean_squared_error: 0.4975 - val_loss: 0.0709 - val_mean_absolute_error: 0.2184 - val_mean_absolute_percentage_error: 17048698.0000 - val_root_mean_squared_error: 0.2663
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0326 - mean_absolute_error: 0.1425 - mean_absolute_percentage_error: 32.5169 - root_mean_squared_error: 0.1803 - val_loss: 0.0544 - val_mean_absolute_error: 0.1964 - val_mean_absolute_percentage_error: 15718237.0000 - val_root_mean_squared_error: 0.2332
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0242 - mean_absolute_error: 0.1215 - mean_absolute_percentage_error: 30.4622 - root_mean_squared_error: 0.1555 - val_loss: 0.0508 - val_mean_absolute_error: 0.1884 - val_mean_absolute_percentage_error: 15614125.0000 - val_root_mean_squared_error: 0.2255
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - lo

In [ ]:
y_pred = model_hybrid_22.predict(x_test)

evaluation_hybrid_22 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_22

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 511ms/step


{'Mean Squared Error: ': np.float64(368.86136982231534),
 'Mean Absolute Error: ': np.float64(14.678571389645947),
 'Root Mean Absolute Error: ': 19.205763973930203,
 'Mean Absolute Percentage Error: ': np.float64(27.41966421844953)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_22.evaluate(x_test, y_test)
print("model hybrid 22")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0294 - mean_absolute_error: 0.1311 - mean_absolute_percentage_error: 34.8716 - root_mean_squared_error: 0.1715
model hybrid 22
Mean Square Error: 0.02940540201961994
Mean Absolute Error: 0.13105866312980652
Mean Absolute Percentage Error: 34.871620178222656
Root Mean Squared Error: 0.17148002982139587


## model 23 (CNN=320 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_23 = Sequential()
model_hybrid_23.add(Conv1D(filters=320, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_23.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_23.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_23.add(Flatten())
model_hybrid_23.add(Dense(7))
model_hybrid_23.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_23.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_67"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_73 (Conv1D)              │ (None, 7, 320)         │         2,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_83 (LSTM)                  │ (None, 7, 64)          │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_84 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_67 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 114,791 (448.40 KB)

 Trainable params: 114,791 (448.40 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_23.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - loss: 0.2832 - mean_absolute_error: 0.4965 - mean_absolute_percentage_error: 87.3345 - root_mean_squared_error: 0.5305 - val_loss: 0.0970 - val_mean_absolute_error: 0.2469 - val_mean_absolute_percentage_error: 16577844.0000 - val_root_mean_squared_error: 0.3115
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0477 - mean_absolute_error: 0.1697 - mean_absolute_percentage_error: 37.5052 - root_mean_squared_error: 0.2164 - val_loss: 0.0548 - val_mean_absolute_error: 0.1942 - val_mean_absolute_percentage_error: 15145208.0000 - val_root_mean_squared_error: 0.2341
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0268 - mean_absolute_error: 0.1281 - mean_absolute_percentage_error: 33.2597 - root_mean_squared_error: 0.1636 - val_loss: 0.0530 - val_mean_absolute_error: 0.1936 - val_mean_absolute_percentage_error: 14512120.0000 - val_root_mean_squared_error: 0.2302
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - lo

In [ ]:
y_pred = model_hybrid_23.predict(x_test)

evaluation_hybrid_23 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_23

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step


{'Mean Squared Error: ': np.float64(345.3015008851733),
 'Mean Absolute Error: ': np.float64(14.241107434642558),
 'Root Mean Absolute Error: ': 18.5822899795793,
 'Mean Absolute Percentage Error: ': np.float64(26.651464530052593)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_23.evaluate(x_test, y_test)
print("model hybrid 23")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0275 - mean_absolute_error: 0.1272 - mean_absolute_percentage_error: 33.9161 - root_mean_squared_error: 0.1659
model hybrid 23
Mean Square Error: 0.0275272224098444
Mean Absolute Error: 0.12715274095535278
Mean Absolute Percentage Error: 33.916099548339844
Root Mean Squared Error: 0.16591329872608185


## model 24 gridsearch & early stopping (CNN=300, LSTM= 64, 32)

In [ ]:
# from keras.callbacks import EarlyStopping
# from sklearn.model_selection import GridSearchCV
# from scikeras.wrappers import KerasRegressor
# from keras.optimizers import Adam

# # 1. Define your model builder function
# def build_model(filters1=300, units1=64, units2=32, kernel_size=1, lr=0.001):
#     model = Sequential()
#     model.add(Input(shape=(7, 11)))
#     model.add(Conv1D(filters=filters1, kernel_size=kernel_size, activation='relu'))
#     model.add(LSTM(units=units1, activation='relu', return_sequences=True))
#     model.add(LSTM(units=units2, activation='relu'))
#     model.add(Flatten())
#     model.add(Dense(7))
#     optimizer = Adam(learning_rate=lr)
#     model.compile(loss='mse', optimizer=optimizer,
#                   metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
#     return model

# # 2. Set up KerasRegressor
# model = KerasRegressor(model=build_model, verbose=0)

# # 3. Define early stopping
# early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # 4. Grid of parameters
# param_grid = {
#     'model__filters1': [300],
#     'model__units1': [64],
#     'model__units2': [32],
#     'model__kernel_size': [1],
#     'model__lr': [0.0002, 0.0003, 0.0004],
#     'batch_size': [32, 42, 64],
#     'epochs': [50]  # set high epochs so early stopping can interrupt early
# }

# # 5. GridSearchCV with early stopping passed via fit_params
# grid = GridSearchCV(estimator=model, param_grid=param_grid,
#                     scoring='neg_mean_squared_error',
#                     cv=3,
#                     refit=True)

# # 6. Fit with early stopping
# grid_result_cnn = grid.fit(x_train, y_train, callbacks=[early_stop], validation_data=(x_val, y_val))


In [ ]:
# print("Best parameters found: ", grid_result_cnn.best_params_)
# # print("Best MSE: ", -grid_result.best_score_)

In [ ]:
# best_model = grid_result_cnn.best_estimator_
# loss, mae, mape, rmse = best_model.model_.evaluate(x_test, y_test)
# print("Test MSE:", loss)
# print("Test MAE:", mae)
# print("Test MAPE:", mape)
# print("Test RMSE:", rmse)


## model 25 (CNN=300 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_25 = Sequential()
model_hybrid_25.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_25.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_25.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_25.add(Flatten())
model_hybrid_25.add(Dense(7))
model_hybrid_25.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_25.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_68"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_74 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_85 (LSTM)                  │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_86 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_68 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,531 (427.86 KB)

 Trainable params: 109,531 (427.86 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_25.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=30,
    batch_size=64
)

Epoch 1/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - loss: 0.2788 - mean_absolute_error: 0.4960 - mean_absolute_percentage_error: 86.8094 - root_mean_squared_error: 0.5269 - val_loss: 0.0854 - val_mean_absolute_error: 0.2482 - val_mean_absolute_percentage_error: 10852179.0000 - val_root_mean_squared_error: 0.2923
Epoch 2/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0526 - mean_absolute_error: 0.1810 - mean_absolute_percentage_error: 40.4381 - root_mean_squared_error: 0.2291 - val_loss: 0.0653 - val_mean_absolute_error: 0.2175 - val_mean_absolute_percentage_error: 12180525.0000 - val_root_mean_squared_error: 0.2555
Epoch 3/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0327 - mean_absolute_error: 0.1472 - mean_absolute_percentage_error: 30.8538 - root_mean_squared_error: 0.1807 - val_loss: 0.0537 - val_mean_absolute_error: 0.1957 - val_mean_absolute_percentage_error: 14968516.0000 - val_root_mean_squared_error: 0.2316
Epoch 4/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - lo

In [ ]:
y_pred = model_hybrid_25.predict(x_test)

evaluation_hybrid_25 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_25

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step


{'Mean Squared Error: ': np.float64(393.01347593562076),
 'Mean Absolute Error: ': np.float64(15.011197187462633),
 'Root Mean Absolute Error: ': 19.824567484200525,
 'Mean Absolute Percentage Error: ': np.float64(28.4413327335505)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_25.evaluate(x_test, y_test)
print("model hybrid 25")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0313 - mean_absolute_error: 0.1340 - mean_absolute_percentage_error: 36.2400 - root_mean_squared_error: 0.1770
model hybrid 25
Mean Square Error: 0.03133079409599304
Mean Absolute Error: 0.13402853906154633
Mean Absolute Percentage Error: 36.23999786376953
Root Mean Squared Error: 0.17700506746768951


## model 26 (CNN=300 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_26 = Sequential()
model_hybrid_26.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_26.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_26.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_26.add(Flatten())
model_hybrid_26.add(Dense(7))
model_hybrid_26.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_26.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_69"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_75 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_87 (LSTM)                  │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_88 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_69 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,531 (427.86 KB)

 Trainable params: 109,531 (427.86 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_26.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=64
)

Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - loss: 0.3218 - mean_absolute_error: 0.5381 - mean_absolute_percentage_error: 94.8600 - root_mean_squared_error: 0.5670 - val_loss: 0.2085 - val_mean_absolute_error: 0.4027 - val_mean_absolute_percentage_error: 5694891.0000 - val_root_mean_squared_error: 0.4566
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.1447 - mean_absolute_error: 0.3130 - mean_absolute_percentage_error: 58.5948 - root_mean_squared_error: 0.3797 - val_loss: 0.0743 - val_mean_absolute_error: 0.2284 - val_mean_absolute_percentage_error: 12697804.0000 - val_root_mean_squared_error: 0.2726
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0364 - mean_absolute_error: 0.1542 - mean_absolute_percentage_error: 35.1181 - root_mean_squared_error: 0.1905 - val_loss: 0.0562 - val_mean_absolute_error: 0.1975 - val_mean_absolute_percentage_error: 14836739.0000 - val_root_mean_squared_error: 0.2370
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - los

In [ ]:
y_pred = model_hybrid_26.predict(x_test)

evaluation_hybrid_26 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_26

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step


{'Mean Squared Error: ': np.float64(409.9505542328133),
 'Mean Absolute Error: ': np.float64(15.554531175263076),
 'Root Mean Absolute Error: ': 20.247235718310126,
 'Mean Absolute Percentage Error: ': np.float64(29.115850689134337)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_26.evaluate(x_test, y_test)
print("model hybrid 26")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0327 - mean_absolute_error: 0.1389 - mean_absolute_percentage_error: 37.0046 - root_mean_squared_error: 0.1808
model hybrid 26
Mean Square Error: 0.03268100693821907
Mean Absolute Error: 0.13887974619865417
Mean Absolute Percentage Error: 37.00458908081055
Root Mean Squared Error: 0.1807788908481598


## model 27 (CNN=300 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_27 = Sequential()
model_hybrid_27.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_27.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_27.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_27.add(Flatten())
model_hybrid_27.add(Dense(7))
model_hybrid_27.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_27.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_70"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_76 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_89 (LSTM)                  │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_90 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_70 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,531 (427.86 KB)

 Trainable params: 109,531 (427.86 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_27.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=40,
    batch_size=64
)

Epoch 1/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 0.3279 - mean_absolute_error: 0.5437 - mean_absolute_percentage_error: 95.5064 - root_mean_squared_error: 0.5723 - val_loss: 0.2058 - val_mean_absolute_error: 0.4055 - val_mean_absolute_percentage_error: 4603568.5000 - val_root_mean_squared_error: 0.4537
Epoch 2/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.1287 - mean_absolute_error: 0.3032 - mean_absolute_percentage_error: 55.3897 - root_mean_squared_error: 0.3568 - val_loss: 0.0649 - val_mean_absolute_error: 0.2180 - val_mean_absolute_percentage_error: 13868003.0000 - val_root_mean_squared_error: 0.2547
Epoch 3/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0319 - mean_absolute_error: 0.1407 - mean_absolute_percentage_error: 32.3439 - root_mean_squared_error: 0.1786 - val_loss: 0.0559 - val_mean_absolute_error: 0.1985 - val_mean_absolute_percentage_error: 15086513.0000 - val_root_mean_squared_error: 0.2364
Epoch 4/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - los

In [ ]:
y_pred = model_hybrid_27.predict(x_test)

evaluation_hybrid_27 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_27

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step


{'Mean Squared Error: ': np.float64(277.32177746283327),
 'Mean Absolute Error: ': np.float64(12.84862239993348),
 'Root Mean Absolute Error: ': 16.65298103832564,
 'Mean Absolute Percentage Error: ': np.float64(23.80616432817356)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_27.evaluate(x_test, y_test)
print("model hybrid 27")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0221 - mean_absolute_error: 0.1147 - mean_absolute_percentage_error: 30.3095 - root_mean_squared_error: 0.1487
model hybrid 27
Mean Square Error: 0.0221079234033823
Mean Absolute Error: 0.11471984535455704
Mean Absolute Percentage Error: 30.309520721435547
Root Mean Squared Error: 0.14868733286857605


## model 28 (CNN=300 LSTM=64, 32)

In [ ]:
lr = 0.0003

model_hybrid_28 = Sequential()
model_hybrid_28.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_28.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_28.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_28.add(Flatten())
model_hybrid_28.add(Dense(7))
model_hybrid_28.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_28.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_71"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_77 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_91 (LSTM)                  │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_92 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_71 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,531 (427.86 KB)

 Trainable params: 109,531 (427.86 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_28.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=40,
    batch_size=42
)

Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - loss: 0.3022 - mean_absolute_error: 0.5163 - mean_absolute_percentage_error: 91.6659 - root_mean_squared_error: 0.5490 - val_loss: 0.1209 - val_mean_absolute_error: 0.2888 - val_mean_absolute_percentage_error: 13564192.0000 - val_root_mean_squared_error: 0.3476
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0651 - mean_absolute_error: 0.1979 - mean_absolute_percentage_error: 41.8110 - root_mean_squared_error: 0.2531 - val_loss: 0.0528 - val_mean_absolute_error: 0.1925 - val_mean_absolute_percentage_error: 15921965.0000 - val_root_mean_squared_error: 0.2299
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0243 - mean_absolute_error: 0.1218 - mean_absolute_percentage_error: 30.2422 - root_mean_squared_error: 0.1560 - val_loss: 0.0528 - val_mean_absolute_error: 0.1917 - val_mean_absolute_percentage_error: 15383220.0000 - val_root_mean_squared_error: 0.2298
Epoch 4/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - lo

In [ ]:
y_pred = model_hybrid_28.predict(x_test)

evaluation_hybrid_28 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_28

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step


{'Mean Squared Error: ': np.float64(315.36387699583975),
 'Mean Absolute Error: ': np.float64(13.398618737045599),
 'Root Mean Absolute Error: ': 17.758487463628196,
 'Mean Absolute Percentage Error: ': np.float64(25.422659256965158)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_28.evaluate(x_test, y_test)
print("model hybrid 28")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0251 - mean_absolute_error: 0.1196 - mean_absolute_percentage_error: 32.4592 - root_mean_squared_error: 0.1586
model hybrid 28
Mean Square Error: 0.025140613317489624
Mean Absolute Error: 0.11963052302598953
Mean Absolute Percentage Error: 32.459205627441406
Root Mean Squared Error: 0.1585579216480255


## model 29 (CNN=300 LSTM=32, 32)

In [ ]:
lr = 0.0003

model_hybrid_29 = Sequential()
model_hybrid_29.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_29.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_29.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_29.add(Flatten())
model_hybrid_29.add(Dense(7))
model_hybrid_29.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_29.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_72"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_78 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_93 (LSTM)                  │ (None, 7, 32)          │        42,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_94 (LSTM)                  │ (None, 7, 32)          │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_72 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,619 (213.36 KB)

 Trainable params: 54,619 (213.36 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_29.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - loss: 0.3142 - mean_absolute_error: 0.5310 - mean_absolute_percentage_error: 93.9163 - root_mean_squared_error: 0.5600 - val_loss: 0.1263 - val_mean_absolute_error: 0.3136 - val_mean_absolute_percentage_error: 7957017.5000 - val_root_mean_squared_error: 0.3554
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0604 - mean_absolute_error: 0.1973 - mean_absolute_percentage_error: 40.2846 - root_mean_squared_error: 0.2449 - val_loss: 0.0699 - val_mean_absolute_error: 0.2266 - val_mean_absolute_percentage_error: 11634530.0000 - val_root_mean_squared_error: 0.2644
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0293 - mean_absolute_error: 0.1380 - mean_absolute_percentage_error: 29.5946 - root_mean_squared_error: 0.1710 - val_loss: 0.0530 - val_mean_absolute_error: 0.1943 - val_mean_absolute_percentage_error: 14499971.0000 - val_root_mean_squared_error: 0.2303
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - los

In [ ]:
y_pred = model_hybrid_29.predict(x_test)

evaluation_hybrid_29 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_29

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step


{'Mean Squared Error: ': np.float64(297.27619761704227),
 'Mean Absolute Error: ': np.float64(13.860039964014168),
 'Root Mean Absolute Error: ': 17.24169938309569,
 'Mean Absolute Percentage Error: ': np.float64(24.62770722976006)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_29.evaluate(x_test, y_test)
print("model hybrid 29")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0237 - mean_absolute_error: 0.1238 - mean_absolute_percentage_error: 31.0907 - root_mean_squared_error: 0.1539
model hybrid 29
Mean Square Error: 0.023698674514889717
Mean Absolute Error: 0.12375034391880035
Mean Absolute Percentage Error: 31.09072494506836
Root Mean Squared Error: 0.153943732380867


## model 30 (CNN=300 LSTM=16, 32)

In [ ]:
lr = 0.0003

model_hybrid_30 = Sequential()
model_hybrid_30.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_30.add(LSTM(16, activation='relu', return_sequences=True))
model_hybrid_30.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_30.add(Flatten())
model_hybrid_30.add(Dense(7))
model_hybrid_30.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_30.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_73"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_79 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_95 (LSTM)                  │ (None, 7, 16)          │        20,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_96 (LSTM)                  │ (None, 7, 32)          │         6,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_73 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,235 (118.11 KB)

 Trainable params: 30,235 (118.11 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_30.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 0.3324 - mean_absolute_error: 0.5498 - mean_absolute_percentage_error: 97.1339 - root_mean_squared_error: 0.5764 - val_loss: 0.2397 - val_mean_absolute_error: 0.4469 - val_mean_absolute_percentage_error: 3553183.7500 - val_root_mean_squared_error: 0.4896
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.1508 - mean_absolute_error: 0.3372 - mean_absolute_percentage_error: 59.7356 - root_mean_squared_error: 0.3846 - val_loss: 0.0553 - val_mean_absolute_error: 0.1947 - val_mean_absolute_percentage_error: 16212682.0000 - val_root_mean_squared_error: 0.2353
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0254 - mean_absolute_error: 0.1257 - mean_absolute_percentage_error: 30.1462 - root_mean_squared_error: 0.1595 - val_loss: 0.0531 - val_mean_absolute_error: 0.1932 - val_mean_absolute_percentage_error: 15285109.0000 - val_root_mean_squared_error: 0.2304
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - los

In [ ]:
y_pred = model_hybrid_30.predict(x_test)

evaluation_hybrid_30 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_30

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step


{'Mean Squared Error: ': np.float64(382.7946677114502),
 'Mean Absolute Error: ': np.float64(15.25731592762227),
 'Root Mean Absolute Error: ': 19.565139092565893,
 'Mean Absolute Percentage Error: ': np.float64(27.925584648514512)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_30.evaluate(x_test, y_test)
print("model hybrid 30")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0305 - mean_absolute_error: 0.1362 - mean_absolute_percentage_error: 35.3614 - root_mean_squared_error: 0.1747
model hybrid 30
Mean Square Error: 0.030516156926751137
Mean Absolute Error: 0.13622602820396423
Mean Absolute Percentage Error: 35.361427307128906
Root Mean Squared Error: 0.17468874156475067


## model 31 (CNN=300 LSTM=8, 32)

In [ ]:
lr = 0.0003

model_hybrid_31 = Sequential()
model_hybrid_31.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_31.add(LSTM(8, activation='relu', return_sequences=True))
model_hybrid_31.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_31.add(Flatten())
model_hybrid_31.add(Dense(7))
model_hybrid_31.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_31.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_74"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_80 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_97 (LSTM)                  │ (None, 7, 8)           │         9,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_98 (LSTM)                  │ (None, 7, 32)          │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_74 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,811 (73.48 KB)

 Trainable params: 18,811 (73.48 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_31.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss: 0.3085 - mean_absolute_error: 0.5233 - mean_absolute_percentage_error: 92.2402 - root_mean_squared_error: 0.5548 - val_loss: 0.1381 - val_mean_absolute_error: 0.3273 - val_mean_absolute_percentage_error: 8457777.0000 - val_root_mean_squared_error: 0.3716
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0834 - mean_absolute_error: 0.2330 - mean_absolute_percentage_error: 45.3722 - root_mean_squared_error: 0.2870 - val_loss: 0.0555 - val_mean_absolute_error: 0.2007 - val_mean_absolute_percentage_error: 13660956.0000 - val_root_mean_squared_error: 0.2356
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0264 - mean_absolute_error: 0.1279 - mean_absolute_percentage_error: 30.5216 - root_mean_squared_error: 0.1625 - val_loss: 0.0535 - val_mean_absolute_error: 0.1917 - val_mean_absolute_percentage_error: 14420988.0000 - val_root_mean_squared_error: 0.2313
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - los

In [ ]:
y_pred = model_hybrid_31.predict(x_test)

evaluation_hybrid_31 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_31

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step


{'Mean Squared Error: ': np.float64(325.0771063030899),
 'Mean Absolute Error: ': np.float64(14.741983024441467),
 'Root Mean Absolute Error: ': 18.029894794565216,
 'Mean Absolute Percentage Error: ': np.float64(25.575844323843413)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_31.evaluate(x_test, y_test)
print("model hybrid 31")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0259 - mean_absolute_error: 0.1316 - mean_absolute_percentage_error: 32.0986 - root_mean_squared_error: 0.1610
model hybrid 31
Mean Square Error: 0.02591494843363762
Mean Absolute Error: 0.13162484765052795
Mean Absolute Percentage Error: 32.098609924316406
Root Mean Squared Error: 0.1609812080860138


## model 32 (CNN=300 LSTM=80, 32)

In [ ]:
lr = 0.0003

model_hybrid_32 = Sequential()
model_hybrid_32.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_32.add(LSTM(80, activation='relu', return_sequences=True))
model_hybrid_32.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_32.add(Flatten())
model_hybrid_32.add(Dense(7))
model_hybrid_32.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_32.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_75"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_81 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_99 (LSTM)                  │ (None, 7, 80)          │       121,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_100 (LSTM)                 │ (None, 7, 32)          │        14,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_75 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_75 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140,059 (547.11 KB)

 Trainable params: 140,059 (547.11 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_32.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - loss: 0.2934 - mean_absolute_error: 0.5085 - mean_absolute_percentage_error: 90.9654 - root_mean_squared_error: 0.5409 - val_loss: 0.1005 - val_mean_absolute_error: 0.2632 - val_mean_absolute_percentage_error: 13807096.0000 - val_root_mean_squared_error: 0.3170
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0573 - mean_absolute_error: 0.1887 - mean_absolute_percentage_error: 40.2236 - root_mean_squared_error: 0.2381 - val_loss: 0.0549 - val_mean_absolute_error: 0.1944 - val_mean_absolute_percentage_error: 15721834.0000 - val_root_mean_squared_error: 0.2344
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0286 - mean_absolute_error: 0.1315 - mean_absolute_percentage_error: 34.2827 - root_mean_squared_error: 0.1690 - val_loss: 0.0517 - val_mean_absolute_error: 0.1900 - val_mean_absolute_percentage_error: 15020937.0000 - val_root_mean_squared_error: 0.2274
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - lo

In [ ]:
y_pred = model_hybrid_32.predict(x_test)

evaluation_hybrid_32 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_32

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step


{'Mean Squared Error: ': np.float64(293.4747813972425),
 'Mean Absolute Error: ': np.float64(13.871171698278308),
 'Root Mean Absolute Error: ': 17.13110566768072,
 'Mean Absolute Percentage Error: ': np.float64(23.885885503652602)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_32.evaluate(x_test, y_test)
print("model hybrid 32")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0234 - mean_absolute_error: 0.1238 - mean_absolute_percentage_error: 29.9915 - root_mean_squared_error: 0.1530
model hybrid 32
Mean Square Error: 0.023395629599690437
Mean Absolute Error: 0.12384974956512451
Mean Absolute Percentage Error: 29.991456985473633
Root Mean Squared Error: 0.1529563069343567


## model 33 (CNN=300 LSTM=80, 32)

In [ ]:
lr = 0.0003

model_hybrid_33 = Sequential()
model_hybrid_33.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_33.add(LSTM(100, activation='relu', return_sequences=True))
model_hybrid_33.add(LSTM(32, activation='relu', return_sequences=True))
model_hybrid_33.add(Flatten())
model_hybrid_33.add(Dense(7))
model_hybrid_33.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_33.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_76"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_82 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_101 (LSTM)                 │ (None, 7, 100)         │       160,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_102 (LSTM)                 │ (None, 7, 32)          │        17,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_76 (Flatten)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ (None, 7)              │         1,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 181,099 (707.42 KB)

 Trainable params: 181,099 (707.42 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_33.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - loss: 0.2831 - mean_absolute_error: 0.4936 - mean_absolute_percentage_error: 87.1445 - root_mean_squared_error: 0.5303 - val_loss: 0.0698 - val_mean_absolute_error: 0.2139 - val_mean_absolute_percentage_error: 16321793.0000 - val_root_mean_squared_error: 0.2642
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0346 - mean_absolute_error: 0.1450 - mean_absolute_percentage_error: 33.9234 - root_mean_squared_error: 0.1853 - val_loss: 0.0535 - val_mean_absolute_error: 0.1923 - val_mean_absolute_percentage_error: 15277342.0000 - val_root_mean_squared_error: 0.2313
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0235 - mean_absolute_error: 0.1205 - mean_absolute_percentage_error: 29.1756 - root_mean_squared_error: 0.1532 - val_loss: 0.0537 - val_mean_absolute_error: 0.1935 - val_mean_absolute_percentage_error: 14542503.0000 - val_root_mean_squared_error: 0.2316
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - lo

In [ ]:
y_pred = model_hybrid_33.predict(x_test)

evaluation_hybrid_33 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_33

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step


{'Mean Squared Error: ': np.float64(418.45545035074355),
 'Mean Absolute Error: ': np.float64(15.595797324667172),
 'Root Mean Absolute Error: ': 20.456183670243664,
 'Mean Absolute Percentage Error: ': np.float64(29.401618414276047)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_33.evaluate(x_test, y_test)
print("model hybrid 33")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0334 - mean_absolute_error: 0.1392 - mean_absolute_percentage_error: 37.3973 - root_mean_squared_error: 0.1826
model hybrid 33
Mean Square Error: 0.03335901349782944
Mean Absolute Error: 0.13924819231033325
Mean Absolute Percentage Error: 37.39730453491211
Root Mean Squared Error: 0.18264450132846832


## model 34 (CNN=300 LSTM=64, 8)

In [ ]:
lr = 0.0003

model_hybrid_34 = Sequential()
model_hybrid_34.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_34.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_34.add(LSTM(8, activation='relu', return_sequences=True))
model_hybrid_34.add(Flatten())
model_hybrid_34.add(Dense(7))
model_hybrid_34.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_34.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_77"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_83 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_103 (LSTM)                 │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_104 (LSTM)                 │ (None, 7, 8)           │         2,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_77 (Flatten)            │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_77 (Dense)                │ (None, 7)              │           399 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,275 (383.89 KB)

 Trainable params: 98,275 (383.89 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_34.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - loss: 0.3107 - mean_absolute_error: 0.5242 - mean_absolute_percentage_error: 93.6148 - root_mean_squared_error: 0.5571 - val_loss: 0.1801 - val_mean_absolute_error: 0.3543 - val_mean_absolute_percentage_error: 8663014.0000 - val_root_mean_squared_error: 0.4244
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.1098 - mean_absolute_error: 0.2581 - mean_absolute_percentage_error: 48.9312 - root_mean_squared_error: 0.3291 - val_loss: 0.0734 - val_mean_absolute_error: 0.2325 - val_mean_absolute_percentage_error: 12466362.0000 - val_root_mean_squared_error: 0.2710
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0308 - mean_absolute_error: 0.1376 - mean_absolute_percentage_error: 31.0891 - root_mean_squared_error: 0.1753 - val_loss: 0.0548 - val_mean_absolute_error: 0.1951 - val_mean_absolute_percentage_error: 15935480.0000 - val_root_mean_squared_error: 0.2340
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - los

In [ ]:
y_pred = model_hybrid_34.predict(x_test)

evaluation_hybrid_34 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_34

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step


{'Mean Squared Error: ': np.float64(306.3274912982016),
 'Mean Absolute Error: ': np.float64(13.860484901739627),
 'Root Mean Absolute Error: ': 17.502213897053185,
 'Mean Absolute Percentage Error: ': np.float64(24.29744241692736)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_34.evaluate(x_test, y_test)
print("model hybrid 34")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0244 - mean_absolute_error: 0.1238 - mean_absolute_percentage_error: 30.5712 - root_mean_squared_error: 0.1563
model hybrid 34
Mean Square Error: 0.0244202371686697
Mean Absolute Error: 0.12375432252883911
Mean Absolute Percentage Error: 30.571216583251953
Root Mean Squared Error: 0.15626975893974304


## model 35 (CNN=300 LSTM=64, 16)

In [ ]:
lr = 0.0003

model_hybrid_35 = Sequential()
model_hybrid_35.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_35.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_35.add(LSTM(8, activation='relu', return_sequences=True))
model_hybrid_35.add(Flatten())
model_hybrid_35.add(Dense(7))
model_hybrid_35.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_35.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_78"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_84 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_105 (LSTM)                 │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_106 (LSTM)                 │ (None, 7, 8)           │         2,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_78 (Flatten)            │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_78 (Dense)                │ (None, 7)              │           399 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,275 (383.89 KB)

 Trainable params: 98,275 (383.89 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_35.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - loss: 0.3245 - mean_absolute_error: 0.5368 - mean_absolute_percentage_error: 94.5208 - root_mean_squared_error: 0.5692 - val_loss: 0.2047 - val_mean_absolute_error: 0.3816 - val_mean_absolute_percentage_error: 7995347.0000 - val_root_mean_squared_error: 0.4524
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1536 - mean_absolute_error: 0.3147 - mean_absolute_percentage_error: 58.2897 - root_mean_squared_error: 0.3905 - val_loss: 0.1004 - val_mean_absolute_error: 0.2651 - val_mean_absolute_percentage_error: 10353694.0000 - val_root_mean_squared_error: 0.3169
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0589 - mean_absolute_error: 0.1880 - mean_absolute_percentage_error: 38.8347 - root_mean_squared_error: 0.2423 - val_loss: 0.0701 - val_mean_absolute_error: 0.2254 - val_mean_absolute_percentage_error: 11955863.0000 - val_root_mean_squared_error: 0.2647
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - los

In [ ]:
y_pred = model_hybrid_35.predict(x_test)

evaluation_hybrid_35 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_35

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step


{'Mean Squared Error: ': np.float64(337.43783792733757),
 'Mean Absolute Error: ': np.float64(14.289937895171496),
 'Root Mean Absolute Error: ': 18.369481155637946,
 'Mean Absolute Percentage Error: ': np.float64(26.462778739628412)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_35.evaluate(x_test, y_test)
print("model hybrid 35")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0269 - mean_absolute_error: 0.1276 - mean_absolute_percentage_error: 33.6132 - root_mean_squared_error: 0.1640
model hybrid 35
Mean Square Error: 0.026900338008999825
Mean Absolute Error: 0.12758873403072357
Mean Absolute Percentage Error: 33.61317825317383
Root Mean Squared Error: 0.16401322185993195


## model 36 (CNN=300 LSTM=64, 64)

In [ ]:
lr = 0.0003

model_hybrid_36 = Sequential()
model_hybrid_36.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_36.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_36.add(LSTM(64, activation='relu'))
model_hybrid_36.add(Flatten())
model_hybrid_36.add(Dense(7))
model_hybrid_36.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_36.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_79"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_85 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_107 (LSTM)                 │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_108 (LSTM)                 │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_79 (Flatten)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 129,019 (503.98 KB)

 Trainable params: 129,019 (503.98 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_36.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - loss: 0.2702 - mean_absolute_error: 0.4791 - mean_absolute_percentage_error: 84.0928 - root_mean_squared_error: 0.5168 - val_loss: 0.0761 - val_mean_absolute_error: 0.2308 - val_mean_absolute_percentage_error: 14413624.0000 - val_root_mean_squared_error: 0.2759
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0391 - mean_absolute_error: 0.1582 - mean_absolute_percentage_error: 34.2537 - root_mean_squared_error: 0.1973 - val_loss: 0.0535 - val_mean_absolute_error: 0.1975 - val_mean_absolute_percentage_error: 14342047.0000 - val_root_mean_squared_error: 0.2313
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0249 - mean_absolute_error: 0.1233 - mean_absolute_percentage_error: 30.6456 - root_mean_squared_error: 0.1576 - val_loss: 0.0519 - val_mean_absolute_error: 0.1946 - val_mean_absolute_percentage_error: 14610999.0000 - val_root_mean_squared_error: 0.2278
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - lo

In [ ]:
y_pred = model_hybrid_36.predict(x_test)

evaluation_hybrid_36 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_36

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step


{'Mean Squared Error: ': np.float64(339.03365748635804),
 'Mean Absolute Error: ': np.float64(14.711404761489556),
 'Root Mean Absolute Error: ': 18.412866628701735,
 'Mean Absolute Percentage Error: ': np.float64(26.490348768924264)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_36.evaluate(x_test, y_test)
print("model hybrid 36")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0270 - mean_absolute_error: 0.1314 - mean_absolute_percentage_error: 33.4467 - root_mean_squared_error: 0.1644
model hybrid 36
Mean Square Error: 0.027027552947402
Mean Absolute Error: 0.13135182857513428
Mean Absolute Percentage Error: 33.446720123291016
Root Mean Squared Error: 0.1644005924463272


## model 37 (CNN=300 LSTM=64, 128)

In [ ]:
lr = 0.0003

model_hybrid_37 = Sequential()
model_hybrid_37.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_37.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_37.add(LSTM(128, activation='relu'))
model_hybrid_37.add(Flatten())
model_hybrid_37.add(Dense(7))
model_hybrid_37.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_37.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_80"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_86 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_109 (LSTM)                 │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_110 (LSTM)                 │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_80 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_80 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 195,259 (762.73 KB)

 Trainable params: 195,259 (762.73 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_37.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - loss: 0.2745 - mean_absolute_error: 0.4862 - mean_absolute_percentage_error: 87.3706 - root_mean_squared_error: 0.5222 - val_loss: 0.0689 - val_mean_absolute_error: 0.2061 - val_mean_absolute_percentage_error: 17897054.0000 - val_root_mean_squared_error: 0.2625
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0387 - mean_absolute_error: 0.1576 - mean_absolute_percentage_error: 36.6051 - root_mean_squared_error: 0.1958 - val_loss: 0.0554 - val_mean_absolute_error: 0.1932 - val_mean_absolute_percentage_error: 16241372.0000 - val_root_mean_squared_error: 0.2354
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0259 - mean_absolute_error: 0.1242 - mean_absolute_percentage_error: 31.3863 - root_mean_squared_error: 0.1608 - val_loss: 0.0517 - val_mean_absolute_error: 0.1873 - val_mean_absolute_percentage_error: 15888552.0000 - val_root_mean_squared_error: 0.2273
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - lo

In [ ]:
y_pred = model_hybrid_37.predict(x_test)

evaluation_hybrid_37 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_37

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step


{'Mean Squared Error: ': np.float64(354.4829828682073),
 'Mean Absolute Error: ': np.float64(14.625278648065061),
 'Root Mean Absolute Error: ': 18.82771847219432,
 'Mean Absolute Percentage Error: ': np.float64(27.09405022614645)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_37.evaluate(x_test, y_test)
print("model hybrid 37")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0283 - mean_absolute_error: 0.1306 - mean_absolute_percentage_error: 34.3921 - root_mean_squared_error: 0.1681
model hybrid 37
Mean Square Error: 0.0282591599971056
Mean Absolute Error: 0.13058283925056458
Mean Absolute Percentage Error: 34.392127990722656
Root Mean Squared Error: 0.1681046038866043


## model 38 (CNN=300 LSTM=64, 256)

In [ ]:
lr = 0.0003

model_hybrid_38 = Sequential()
model_hybrid_38.add(Conv1D(filters=300, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_38.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_38.add(LSTM(256, activation='relu'))
model_hybrid_38.add(Flatten())
model_hybrid_38.add(Dense(7))
model_hybrid_38.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_38.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_81"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_87 (Conv1D)              │ (None, 7, 300)         │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_111 (LSTM)                 │ (None, 7, 64)          │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_112 (LSTM)                 │ (None, 256)            │       328,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_81 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 426,043 (1.63 MB)

 Trainable params: 426,043 (1.63 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_38.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 96ms/step - loss: 0.2676 - mean_absolute_error: 0.4764 - mean_absolute_percentage_error: 85.3828 - root_mean_squared_error: 0.5146 - val_loss: 0.0506 - val_mean_absolute_error: 0.1861 - val_mean_absolute_percentage_error: 16014190.0000 - val_root_mean_squared_error: 0.2249
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0328 - mean_absolute_error: 0.1476 - mean_absolute_percentage_error: 33.1290 - root_mean_squared_error: 0.1810 - val_loss: 0.0504 - val_mean_absolute_error: 0.1837 - val_mean_absolute_percentage_error: 17379506.0000 - val_root_mean_squared_error: 0.2245
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0219 - mean_absolute_error: 0.1154 - mean_absolute_percentage_error: 28.6297 - root_mean_squared_error: 0.1477 - val_loss: 0.0509 - val_mean_absolute_error: 0.1854 - val_mean_absolute_percentage_error: 15837313.0000 - val_root_mean_squared_error: 0.2256
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - lo

In [ ]:
y_pred = model_hybrid_38.predict(x_test)

evaluation_hybrid_38 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_38

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step


{'Mean Squared Error: ': np.float64(355.4543093060602),
 'Mean Absolute Error: ': np.float64(14.533910362087951),
 'Root Mean Absolute Error: ': 18.85349594388426,
 'Mean Absolute Percentage Error: ': np.float64(27.01287068115001)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_38.evaluate(x_test, y_test)
print("model hybrid 38")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0283 - mean_absolute_error: 0.1298 - mean_absolute_percentage_error: 34.3201 - root_mean_squared_error: 0.1683
model hybrid 38
Mean Square Error: 0.028336599469184875
Mean Absolute Error: 0.129767045378685
Mean Absolute Percentage Error: 34.320064544677734
Root Mean Squared Error: 0.16833478212356567


## model 39 (CNN=32 LSTM=64, 16)

In [ ]:
lr = 0.0003

model_hybrid_39 = Sequential()
model_hybrid_39.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_39.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_39.add(LSTM(16, activation='relu'))
model_hybrid_39.add(Flatten())
model_hybrid_39.add(Dense(7))
model_hybrid_39.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_39.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_82"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_88 (Conv1D)              │ (None, 7, 32)          │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_113 (LSTM)                 │ (None, 7, 64)          │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_114 (LSTM)                 │ (None, 16)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_82 (Flatten)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,359 (118.59 KB)

 Trainable params: 30,359 (118.59 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_39.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 0.3240 - mean_absolute_error: 0.5402 - mean_absolute_percentage_error: 96.8131 - root_mean_squared_error: 0.5691 - val_loss: 0.2368 - val_mean_absolute_error: 0.4405 - val_mean_absolute_percentage_error: 3376017.5000 - val_root_mean_squared_error: 0.4867
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.1879 - mean_absolute_error: 0.3826 - mean_absolute_percentage_error: 67.2289 - root_mean_squared_error: 0.4325 - val_loss: 0.1212 - val_mean_absolute_error: 0.2881 - val_mean_absolute_percentage_error: 10910440.0000 - val_root_mean_squared_error: 0.3481
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0863 - mean_absolute_error: 0.2363 - mean_absolute_percentage_error: 44.7377 - root_mean_squared_error: 0.2937 - val_loss: 0.0869 - val_mean_absolute_error: 0.2430 - val_mean_absolute_percentage_error: 12814337.0000 - val_root_mean_squared_error: 0.2948
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - los

In [ ]:
y_pred = model_hybrid_39.predict(x_test)

evaluation_hybrid_39 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_39

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step


{'Mean Squared Error: ': np.float64(472.83891055577885),
 'Mean Absolute Error: ': np.float64(17.531496787557796),
 'Root Mean Absolute Error: ': 21.744859405288846,
 'Mean Absolute Percentage Error: ': np.float64(32.15752125252407)}

In [ ]:
loss, mae, mape, rmse = model_hybrid_39.evaluate(x_test, y_test)
print("model hybrid 39")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0377 - mean_absolute_error: 0.1565 - mean_absolute_percentage_error: 40.6359 - root_mean_squared_error: 0.1942
model hybrid 39
Mean Square Error: 0.03769442439079285
Mean Absolute Error: 0.1565312147140503
Mean Absolute Percentage Error: 40.635860443115234
Root Mean Squared Error: 0.19415052235126495


## model 40 (CNN=32 LSTM=64, 64)

In [ ]:
lr = 0.0003

model_hybrid_40 = Sequential()
model_hybrid_40.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_40.add(LSTM(64, activation='relu', return_sequences=True))
model_hybrid_40.add(LSTM(64, activation='relu'))
model_hybrid_40.add(Flatten())
model_hybrid_40.add(Dense(7))
model_hybrid_40.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_40.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_83"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_89 (Conv1D)              │ (None, 7, 32)          │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_115 (LSTM)                 │ (None, 7, 64)          │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_116 (LSTM)                 │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_83 (Flatten)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,535 (228.65 KB)

 Trainable params: 58,535 (228.65 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_40.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20


In [ ]:
y_pred = model_hybrid_40.predict(x_test)

evaluation_hybrid_40 = evaluate_preds(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred))
evaluation_hybrid_40

In [ ]:
loss, mae, mape, rmse = model_hybrid_40.evaluate(x_test, y_test)
print("model hybrid 40")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

## model 24 gridsearch & early stopping (CNN=300, LSTM= 64, 32)

In [ ]:
# from keras.callbacks import EarlyStopping
# from sklearn.model_selection import GridSearchCV
# from scikeras.wrappers import KerasRegressor
# from keras.optimizers import Adam

# # 1. Define your model builder function
# def build_model(filters1=300, units1=64, units2=32, kernel_size=1, lr=0.001):
#     model = Sequential()
#     model.add(Input(shape=(7, 11)))
#     model.add(Conv1D(filters=filters1, kernel_size=kernel_size, activation='relu'))
#     model.add(LSTM(units=units1, activation='relu', return_sequences=True))
#     model.add(LSTM(units=units2, activation='relu'))
#     model.add(Flatten())
#     model.add(Dense(7))
#     optimizer = Adam(learning_rate=lr)
#     model.compile(loss='mse', optimizer=optimizer,
#                   metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
#     return model

# # 2. Set up KerasRegressor
# model = KerasRegressor(model=build_model, verbose=0)

# # 3. Define early stopping
# early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # 4. Grid of parameters
# param_grid = {
#     'model__filters1': [128, 300],
#     'model__units1': [64, 128],
#     'model__units2': [32, 64, 128, 256],
#     'model__kernel_size': [1],
#     'model__lr': [0.0003],
#     'batch_size': [42],
#     'epochs': [20]  # set high epochs so early stopping can interrupt early
# }

# # 5. GridSearchCV with early stopping passed via fit_params
# grid = GridSearchCV(estimator=model, param_grid=param_grid,
#                     scoring='neg_mean_squared_error',
#                     cv=3,
#                     refit=True)

# # 6. Fit with early stopping
# grid_result_cnn = grid.fit(x_train, y_train, callbacks=[early_stop], validation_data=(x_val, y_val))


In [ ]:
# print("Best parameters found: ", grid_result_cnn.best_params_)
# # print("Best MSE: ", -grid_result.best_score_)

In [ ]:
# best_model = grid_result_cnn.best_estimator_
# loss, mae, mape, rmse = best_model.model_.evaluate(x_test, y_test)
# print("Test MSE:", loss)
# print("Test MAE:", mae)
# print("Test MAPE:", mape)
# print("Test RMSE:", rmse)


In [ ]:
evaluation_cnn_1, evaluation_cnn_2, evaluation_cnn_3, evaluation_cnn_4, evaluation_cnn_5, evaluation_cnn_6, evaluation_cnn_7, evaluation_cnn_8, evaluation_cnn_9, evaluation_cnn_10, evaluation_cnn_11, evaluation_cnn_12, evaluation_cnn_13, evaluation_cnn_14, evaluation_cnn_15, evaluation_cnn_16, evaluation_cnn_17, evaluation_cnn_18, evaluation_cnn_19, evaluation_cnn_20

({'Mean Squared Error: ': np.float64(308.99187142125805),
  'Mean Absolute Error: ': np.float64(13.681984979279187),
  'Root Mean Absolute Error: ': 17.578164620382246,
  'Mean Absolute Percentage Error: ': np.float64(24.58539616498866)},
 {'Mean Squared Error: ': np.float64(308.7728270587435),
  'Mean Absolute Error: ': np.float64(13.709325790405273),
  'Root Mean Absolute Error: ': 17.571932934618875,
  'Mean Absolute Percentage Error: ': np.float64(24.22289758217828)},
 {'Mean Squared Error: ': np.float64(323.158230720397),
  'Mean Absolute Error: ': np.float64(14.176996386781031),
  'Root Mean Absolute Error: ': 17.976602313017803,
  'Mean Absolute Percentage Error: ': np.float64(25.360192021301597)},
 {'Mean Squared Error: ': np.float64(313.0978174914591),
  'Mean Absolute Error: ': np.float64(13.376624593929368),
  'Root Mean Absolute Error: ': 17.694570282757905,
  'Mean Absolute Percentage Error: ': np.float64(24.57933044941292)},
 {'Mean Squared Error: ': np.float64(355.963328

In [ ]:
evaluation_lstm_1, evaluation_lstm_2, evaluation_lstm_3, evaluation_lstm_4, evaluation_lstm_5, evaluation_lstm_6, evaluation_lstm_7, evaluation_lstm_8, evaluation_lstm_9, evaluation_lstm_10, evaluation_lstm_11, evaluation_lstm_12, evaluation_lstm_13, evaluation_lstm_14, evaluation_lstm_15, evaluation_lstm_16, evaluation_lstm_17, evaluation_lstm_18, evaluation_lstm_19, evaluation_lstm_20, evaluation_lstm_21, evaluation_lstm_22, evaluation_lstm_23, evaluation_lstm_24, evaluation_lstm_25

({'Mean Squared Error: ': np.float64(340.11774836036926),
  'Mean Absolute Error: ': np.float64(14.84447903535804),
  'Root Mean Absolute Error: ': 18.44228153890861,
  'Mean Absolute Percentage Error: ': np.float64(23.803440264229536)},
 {'Mean Squared Error: ': np.float64(392.0863969822591),
  'Mean Absolute Error: ': np.float64(16.510735317152374),
  'Root Mean Absolute Error: ': 19.801171606302972,
  'Mean Absolute Percentage Error: ': np.float64(27.492623217690426)},
 {'Mean Squared Error: ': np.float64(395.8629015229882),
  'Mean Absolute Error: ': np.float64(15.105303316700216),
  'Root Mean Absolute Error: ': 19.896303715087086,
  'Mean Absolute Percentage Error: ': np.float64(28.91552608709616)},
 {'Mean Squared Error: ': np.float64(385.0995006265619),
  'Mean Absolute Error: ': np.float64(14.782673660589724),
  'Root Mean Absolute Error: ': 19.623952217292057,
  'Mean Absolute Percentage Error: ': np.float64(28.017642150445106)},
 {'Mean Squared Error: ': np.float64(324.77212

In [ ]:
evaluation_hybrid_1, evaluation_hybrid_2, evaluation_hybrid_3, evaluation_hybrid_4, evaluation_hybrid_5, evaluation_hybrid_6, evaluation_hybrid_7, evaluation_hybrid_8,  evaluation_hybrid_9, evaluation_hybrid_10, evaluation_hybrid_11, evaluation_hybrid_12, evaluation_hybrid_13, evaluation_hybrid_14, evaluation_hybrid_15, evaluation_hybrid_16,  evaluation_hybrid_17, evaluation_hybrid_18, evaluation_hybrid_19, evaluation_hybrid_20, evaluation_21, evaluation_hybrid_22, evaluation_hybrid_23,

({'Mean Squared Error: ': np.float64(368.07719860158966),
  'Mean Absolute Error: ': np.float64(14.585360215634719),
  'Root Mean Absolute Error: ': 19.185338115383573,
  'Mean Absolute Percentage Error: ': np.float64(27.464497393851005)},
 {'Mean Squared Error: ': np.float64(327.6028594153596),
  'Mean Absolute Error: ': np.float64(14.298462225466361),
  'Root Mean Absolute Error: ': 18.099802745205807,
  'Mean Absolute Percentage Error: ': np.float64(25.68796351144772)},
 {'Mean Squared Error: ': np.float64(315.60906852084565),
  'Mean Absolute Error: ': np.float64(14.145307190564212),
  'Root Mean Absolute Error: ': 17.76538962479702,
  'Mean Absolute Percentage Error: ': np.float64(25.504562203188776)},
 {'Mean Squared Error: ': np.float64(409.30828164831024),
  'Mean Absolute Error: ': np.float64(16.674587288681344),
  'Root Mean Absolute Error: ': 20.231368753703006,
  'Mean Absolute Percentage Error: ': np.float64(27.921256943160245)},
 {'Mean Squared Error: ': np.float64(278.67

In [ ]:
evaluation_hybrid_25, evaluation_hybrid_26, evaluation_hybrid_27, evaluation_hybrid_28, evaluation_hybrid_29, evaluation_hybrid_30, evaluation_hybrid_31, evaluation_hybrid_32, evaluation_hybrid_33, evaluation_hybrid_34, evaluation_hybrid_35, evaluation_hybrid_36, evaluation_hybrid_37, evaluation_hybrid_38, evaluation_hybrid_39, evaluation_hybrid_40


({'Mean Squared Error: ': np.float64(393.01347593562076),
  'Mean Absolute Error: ': np.float64(15.011197187462633),
  'Root Mean Absolute Error: ': 19.824567484200525,
  'Mean Absolute Percentage Error: ': np.float64(28.4413327335505)},
 {'Mean Squared Error: ': np.float64(409.9505542328133),
  'Mean Absolute Error: ': np.float64(15.554531175263076),
  'Root Mean Absolute Error: ': 20.247235718310126,
  'Mean Absolute Percentage Error: ': np.float64(29.115850689134337)},
 {'Mean Squared Error: ': np.float64(277.32177746283327),
  'Mean Absolute Error: ': np.float64(12.84862239993348),
  'Root Mean Absolute Error: ': 16.65298103832564,
  'Mean Absolute Percentage Error: ': np.float64(23.80616432817356)},
 {'Mean Squared Error: ': np.float64(315.36387699583975),
  'Mean Absolute Error: ': np.float64(13.398618737045599),
  'Root Mean Absolute Error: ': 17.758487463628196,
  'Mean Absolute Percentage Error: ': np.float64(25.422659256965158)},
 {'Mean Squared Error: ': np.float64(297.27619

In [ ]:
model_hybrid_12.save("Pol_Hybrid_Model_12.keras")